In [ ]:
##############################################################################
# Created by: Wesley DA SILVA COELHO
##############################################################################
#This file contains the main script for NSDP model execution
#: https://hal.archives-ouvertes.fr/hal-02448028v3


##############################################################################
#                   INCLUDE PACKAGES
##############################################################################
using LightGraphs #Package to construct simple graphs",
using MetaGraphs #Package to construct complex graphs",
using SimpleWeightedGraphs
import LightGraphs.Parallel
using CPLEX
using LinearAlgebra
using StatsBase
import LightGraphs.weights 
import LightGraphs.edges 


In [ ]:
function max_Int(a::Any,b::Any)
    if a< b
        return b
    else
        return a
    end
end

In [ ]:
function get_minimal_number_of_CUs(my_instance::Instance)
     
    ram = Inf
    cpu = Inf
    storage = Inf
    dim_f = []
    
    for u in my_instance.non_access
        if ram > props(my_instance.physical_network,u)[:ram_capacity] 
            ram = deepcopy(props(my_instance.physical_network,u)[:ram_capacity]) 
        end
        if cpu > props(my_instance.physical_network,u)[:cpu_capacity] 
            cpu = deepcopy(props(my_instance.physical_network,u)[:cpu_capacity] )
        end
        if storage > props(my_instance.physical_network,u)[:storage_capacity] 
            storage = deepcopy(props(my_instance.physical_network,u)[:storage_capacity]) 
        end
    end
    my_instance.max_cpu = deepcopy(cpu)
    my_instance.max_ram = deepcopy(ram)
    my_instance.max_storage = deepcopy(storage)

     for f in 1:length(my_instance.set_VNFs)
        dim = 0.0 
        for s in 1:length(my_instance.setSlices)
            if f!=1 && my_instance.set_VNFs[f].typee != :cp 
                dim += sum(my_instance.set_VNFs[f-1].compression*k["volume_of_data"]  for k in my_instance.setSlices[s].set_commodities)/my_instance.set_VNFs[f].treatment_capacity                
            elseif f== 1  && my_instance.set_VNFs[f].typee != :cp 
                dim += sum(k["volume_of_data"] for  k in my_instance.setSlices[s].set_commodities)/my_instance.set_VNFs[f].treatment_capacity                
            else
                dim += my_instance.set_VNFs[f].dataVolPerUE*my_instance.setSlices[s].TotalAmountUE/my_instance.set_VNFs[f].treatment_capacity
            end
        end
        push!(dim_f,(maximum([dim*my_instance.set_VNFs[f].ram_request/ram,dim*my_instance.set_VNFs[f].cpu_request/cpu,dim*my_instance.set_VNFs[f].storage_request/storage])))    
    end
    
    my_instance.number_of_hosts = Int(ceil(sum(dim_f[i] for i in 1:length(dim_f))))
end

In [ ]:
function get_CUs(my_instance::Instance, number_of_CUs_to_be_selected::Int64 )
    
    selected_CUs = Dict()
    selected_CUs["hosts"] = []
    

    for i in 1:min(length(my_instance.non_access),number_of_CUs_to_be_selected*my_instance.tries_choose_CUs)
        push!(selected_CUs["hosts"],my_instance.non_access[i])
    end

    
    my_instance.tries_choose_CUs += 1

    my_instance.selected_CUs = deepcopy(selected_CUs)
    
end

In [1]:
function get_paths(my_instance::Instance, s::Int64)
    cent_split_paths = []
    #for s in 1:length(my_instance.setSlices)
     my_instance.setSlices[s].cent_split_paths  = ""
     for k in 1:length(my_instance.setSlices[s].set_commodities)
            if my_instance.tries_choose_CUs ==1 
                yen =  yen_k_shortest_paths(my_instance.physical_network, my_instance.setSlices[s].set_commodities[k]["origin_node"], my_instance.setSlices[s].set_commodities[k]["target_node"], LightGraphs.weights(my_instance.physical_network), 100, maxdist=my_instance.setSlices[s].maxLatencyDataLayer)
                for y in 1:length(yen.paths)
                    push!(my_instance.setSlices[s].set_commodities[k]["paths"],yen.paths[y])
                    push!(my_instance.setSlices[s].set_commodities[k]["dists"],yen.dists[y])
                end #end for y 
            end # end if choose_ce
            if  my_instance.setSlices[s].cent_split_paths != "path_not_found"
                for u in my_instance.selected_CUs["hosts"], ratio in 0.2:0.3:0.8   
                    first_half =  yen_k_shortest_paths(my_instance.physical_network, my_instance.setSlices[s].set_commodities[k]["origin_node"], u, LightGraphs.weights(my_instance.physical_network), 30, maxdist=my_instance.setSlices[s].maxLatencyDataLayer*ratio)
                    second_half = yen_k_shortest_paths(my_instance.physical_network,  u, my_instance.setSlices[s].set_commodities[k]["target_node"], LightGraphs.weights(my_instance.physical_network), 30, maxdist=my_instance.setSlices[s].maxLatencyDataLayer*(1-ratio))
                     for p in 1:length(first_half.paths), q in 1:length(second_half.paths)
                        if length(first_half.paths[p])*length(second_half.paths[q]) !=0
                            deleteat!(second_half.paths[q], 1)
                            whole_path = vcat(first_half.paths[p],second_half.paths[q])
                            dist = first_half.dists[p] + second_half.dists[q]
                            already_found = false
                            for r in my_instance.setSlices[s].set_commodities[k]["paths"]
                                if r==whole_path
                                    already_found = true
                                     my_instance.setSlices[s].cent_split_paths  = "path_found"
                                    break
                                end #end if r
                            end #end for r
                            if already_found == false    
                                push!(my_instance.setSlices[s].set_commodities[k]["paths"],whole_path)
                                push!(my_instance.setSlices[s].set_commodities[k]["dists"],dist)
                                 my_instance.setSlices[s].cent_split_paths  = "path_found"
                            end #end if already-found
                        end#end lenght
                    end #end for p
                end #end for u
            end #end if path_not_found
           if my_instance.tries_choose_CUs ==1 
                if  my_instance.setSlices[s].cent_split_paths  != "path_found"
                     my_instance.setSlices[s].cent_split_paths = "path_not_found"
                end  #end if path_found
            end #end if tries
        end  #end k
    #end
    #return cent_split_paths
end

UndefVarError: UndefVarError: Instance not defined

In [ ]:
function first_split_setting(my_instance::Instance)
 
   Threads.@threads for ss in 1:length(my_instance.setSlices)
        if my_instance.setSlices[ss].cent_split_paths == "path_not_found"
            for f in 1:my_instance.number_of_AN_based_NFs
                my_instance.solution.var_z[ss,f] = 0
            end
        else
            for f in 1:my_instance.number_of_AN_based_NFs
                my_instance.solution.var_z[ss,f] = 1
            end
        end          
    end
end

In [ ]:
function choose_paths(my_instance::Instance)
    
 
   #creating ILP
    my_model =Model(CPLEX.Optimizer)
    set_optimizer_attributes(my_model,  "CPX_PARAM_THREADS"=>12)
    set_silent(my_model)
    @variable(my_model, x[s in 1:length(my_instance.setSlices), k in 1:length(my_instance.setSlices[s].set_commodities), p in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"])], Bin)
    # @variable(my_model, y[u in my_instance.non_access] >=0)
    @variable(my_model, z[u in my_instance.selected_CUs["hosts"],v in my_instance.selected_CUs["hosts"]] >=0)
    
    for s in 1:length(my_instance.setSlices)
        choosen = false
        for k in 1:length(my_instance.setSlices[s].set_commodities)
            @constraint(my_model, sum(x[s,k,p] for p in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"])) == 1)
            if length(my_instance.setSlices[s].set_commodities[k]["paths"]) > 1  && choosen == false
                for  pp in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"])
                    if rand() > 0.5 && choosen ==false
                        @constraint(my_model, x[s,k,pp] == 0)
                        choosen = true  
                         break
                    end
                end
            end
        end
     end


   #for u in my_instance.non_access
      #  expression = zero(AffExpr)
      #  for s in 1:length(my_instance.setSlices),  k in 1:length(my_instance.setSlices[s].set_commodities), p in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"]) 
      #      if u ∈ my_instance.setSlices[s].set_commodities[k]["paths"][p]
      #          add_to_expression!(expression, x[s,k,p])   
      #      end    
     #   end
        #@constraint(my_model, expression == y[u])
   # end    
    
    
   # for s in 1:length(my_instance.setSlices)
      #  if my_instance.solution.var_z[s,my_instance.number_of_AN_based_NFs] == 1
        #    for k in 1:length(my_instance.setSlices[s].set_commodities)-1,l in k+1:length(my_instance.setSlices[s].set_commodities), p in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"]), q in 1:length(my_instance.setSlices[s].set_commodities[l]["paths"])
        #         if length(isect2(my_instance.selected_CUs["hosts"],isect2(my_instance.setSlices[s].set_commodities[k]["paths"][p],my_instance.setSlices[s].set_commodities[l]["paths"][q]))) < 1
        #             @constraint(my_model, x[s,k,p] + x[s,l,q] <= 1)
        #        end 
        #    end
      #  end
   # end


    for u in my_instance.selected_CUs["hosts"], v in my_instance.selected_CUs["hosts"]
        U_V = zero(AffExpr)
        V_U = zero(AffExpr)
        if u!= v 
            for s in 1:length(my_instance.setSlices),  k in 1:length(my_instance.setSlices[s].set_commodities), p in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"])
                position_U = 0
                position_V = 0   
                    if u ∈ my_instance.setSlices[s].set_commodities[k]["paths"][p]
                     for node in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"][p])
                        if my_instance.setSlices[s].set_commodities[k]["paths"][p][node] == u 
                            position_U = node
                        end
                    end   
                end

                if v ∈ my_instance.setSlices[s].set_commodities[k]["paths"][p]
                     for node in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"][p])
                        if my_instance.setSlices[s].set_commodities[k]["paths"][p][node] == v 
                            position_V = node
                        end
                    end   
                end
                if position_U*position_V>0.0
                   if position_U < position_V
                        add_to_expression!(U_V, x[s,k,p])  
                    else
                        add_to_expression!(V_U, x[s,k,p])   
                    end

                end
            end
        end
        @constraint(my_model, U_V == z[u,v])
        @constraint(my_model, V_U == z[v,u])  
    end   
        
        
        
        
        #(10-log(sqrt(u)))*
    @objective(my_model, Max, sum(z[u,v] for u in my_instance.selected_CUs["hosts"], v in my_instance.selected_CUs["hosts"])) 

       

    #Solving ILP
    optimize!(my_model)
  

    #Getting Solution 
    sol_x = JuMP.value.(getindex(my_model,:x)) 
    #sol_z = JuMP.value.(getindex(my_model,:z)) 
     for s in 1:length(my_instance.setSlices), k in 1:length(my_instance.setSlices[s].set_commodities), p in 1:length(my_instance.setSlices[s].set_commodities[k]["paths"])
        if sol_x[s,k,p] >= 0.1 
            my_instance.setSlices[s].set_commodities[k]["chosen_path"]  = p
        end
    end
    my_model = nothing
   # return sol_z

end

In [ ]:
function random_choose_paths(my_instance::Instance)
    
     for s in 1:length(my_instance.setSlices)
       Threads.@threads for k in 1:length(my_instance.setSlices[s].set_commodities)
            p = rand(1:length(my_instance.setSlices[s].set_commodities[k]["paths"]))
            my_instance.setSlices[s].set_commodities[k]["chosen_path"]  = p 
        end
    end
   # return sol_z

end

In [ ]:
function get_solution_cost(my_instance::Instance)
    my_instance.solution.solution_cost =  sum((props(my_instance.physical_network,u)[:ram_cost]*my_instance.set_VNFs[f].ram_request+props(my_instance.physical_network,u)[:storage_cost]*my_instance.set_VNFs[f].storage_request+props(my_instance.physical_network,u)[:cpu_cost]*my_instance.set_VNFs[f].cpu_request)*my_instance.solution.var_y[f,n,u] for f in 1:length(my_instance.set_VNFs),  n in 1:my_instance.number_of_NFs, u in 1: props(my_instance.physical_network)[:number_nodes])                    
    my_instance.solution.installed_NFSs =  sum(my_instance.solution.var_y[f,n,u] for f in 1:length(my_instance.set_VNFs),  n in 1:my_instance.number_of_NFs, u in 1: props(my_instance.physical_network)[:number_nodes])                    
end
    


In [ ]:
function select_split_setting(my_instance::Instance)

    inter = Vector{Array{Any,1}}()
    for s in 1:length(my_instance.setSlices)
        push!(inter,my_instance.non_access)
    end
    choose_slices = sample(1:length(my_instance.setSlices),rand(0:min(length(my_instance.setSlices),my_instance.number_solution_tries)),replace=false, ordered=false)
    
    Threads.@threads for s in 1:length(my_instance.setSlices)
         for k in 1:length(my_instance.setSlices[s].set_commodities)
            inter[s] =  isect2(inter[s],my_instance.setSlices[s].set_commodities[k]["paths"][my_instance.setSlices[s].set_commodities[k]["chosen_path"]])
        end
        if length(inter[s]) < 1
            #my_instance.setSlices[s].cent_split_paths = "path_not_found"
            for f in 1:my_instance.number_of_AN_based_NFs
                my_instance.solution.var_z[s,f] = 0
            end 
        elseif s in choose_slices
          #my_instance.setSlices[s].cent_split_paths = "path_found"
            
          splitting_f = rand(0:my_instance.number_of_AN_based_NFs)
            if splitting_f < my_instance.number_of_AN_based_NFs
                for f in splitting_f+1:my_instance.number_of_AN_based_NFs
                    my_instance.solution.var_z[s,f] = 1
                 end
            end
            if splitting_f >= 1
                 for f in 1:splitting_f
                    my_instance.solution.var_z[s,f] = 0
                end 
            end
            #normalmente, é aqui que decido que temos que roodar novamente os choose_path e nao deixar escolher os mesmos para esse silice
        else
            for f in 1:my_instance.number_of_AN_based_NFs
              my_instance.solution.var_z[s,f] = 1
            end
        end
            
       my_instance.setSlices[s].potential_centralized_hosts = inter[s]
    end
end

In [ ]:
function verify_centralized_capacity(my_instance::Instance)
    aux_s = []   
    for s in 1:length(my_instance.setSlices)
        required_number_f = []
        for f in 1:my_instance.number_of_AN_based_NFs
            if my_instance.solution.var_z[s,f] == 1
                if  f!=1
                    push!(required_number_f,sum(my_instance.set_VNFs[f-1].compression*k["volume_of_data"] for k in my_instance.setSlices[s].set_commodities)/my_instance.set_VNFs[f].treatment_capacity)                
                else 
                    push!(required_number_f,sum(k["volume_of_data"] for k in my_instance.setSlices[s].set_commodities))
                end
            else
                if  f!=1
                    push!(required_number_f,my_instance.set_VNFs[f-1].compression*my_instance.setSlices[s].set_commodities[1]["volume_of_data"]/my_instance.set_VNFs[f].treatment_capacity)                
                else 
                    push!(required_number_f,my_instance.setSlices[s].set_commodities[1]["volume_of_data"])
                end   
            end
        end
        push!(aux_s,required_number_f)            
    end
    
    return aux_s
end

    

In [ ]:
function color_centralized_graph(my_instance::Instance)
    my_instance.centralized_coloring_solution["tries"] = 0
    while my_instance.centralized_coloring_solution["gap_maxClique"] > 0.0 && my_instance.centralized_coloring_solution["tries"] < 1000
        lets_color = Parallel.greedy_color(my_instance.centralized_coloring_solution["centralized_graph"] ,sort_degree=false,reps = 1000)

        if my_instance.centralized_coloring_solution["tries"] >0  && my_instance.centralized_coloring_solution["coloring"].num_colors >lets_color.num_colors
            my_instance.centralized_coloring_solution["coloring"] = deepcopy(lets_color)
            my_instance.centralized_coloring_solution["gap_maxClique"] = (my_instance.centralized_coloring_solution["coloring"].num_colors -  my_instance.centralized_coloring_solution["max_clique_size"])/my_instance.centralized_coloring_solution["coloring"].num_colors 
            my_instance.centralized_coloring_solution["best_try"] = my_instance.centralized_coloring_solution["tries"] + 1
        elseif my_instance.centralized_coloring_solution["tries"] ==0
            my_instance.centralized_coloring_solution["coloring"] = deepcopy(lets_color)
            my_instance.centralized_coloring_solution["gap_maxClique"] = (my_instance.centralized_coloring_solution["coloring"].num_colors -  my_instance.centralized_coloring_solution["max_clique_size"])/my_instance.centralized_coloring_solution["coloring"].num_colors 
            my_instance.centralized_coloring_solution["best_try"] = my_instance.centralized_coloring_solution["tries"] + 1
        end

        my_instance.centralized_coloring_solution["tries"]  += 1
    end 
end

In [ ]:
function get_centralized_conflit_graph(my_instance::Instance)
       
    min_CPU = Inf
    min_storage = Inf
    min_RAM = Inf
    
    for u in my_instance.selected_CUs["hosts"] 
        if props(my_instance.physical_network, u)[:cpu_capacity] <= min_CPU
            min_CPU = deepcopy(props(my_instance.physical_network, u)[:cpu_capacity])
        end
        if props(my_instance.physical_network, u)[:storage_capacity] <= min_storage
            min_storage = deepcopy(props(my_instance.physical_network, u)[:storage_capacity])
        end
        if props(my_instance.physical_network, u)[:ram_capacity] <= min_RAM
            min_RAM = deepcopy(props(my_instance.physical_network, u)[:ram_capacity])
        end
        
    end
    #creating conflit graph
    aux_vector = Vector{Dict}()
    for s in 1:length(my_instance.setSlices), f in 1:length(my_instance.set_VNFs)
        if (my_instance.set_VNFs[f].typee == :cp && my_instance.setSlices[s].set_VNFs_to_install[f] > 0.5) || (my_instance.set_VNFs[f].typee == :dp && my_instance.solution.var_z[s,f] > 0.5) 
            aux_dict = Dict()
            aux_dict["id"] = length(aux_vector)+1
            aux_dict["slice"] = s
            aux_dict["NFS"] = f
            aux_dict["number_of_NFS_copies"] = get_number_of_centralized_NFS_copies(my_instance,s,f)
            aux_dict["NF"] = 0
            aux_dict["phy_node"] = 0
            push!(aux_vector, aux_dict)
        end
    end 
    conflitGraphn = SimpleGraph(length(aux_vector))
     for u in 1:length(aux_vector)-1
        for v in u+1:length(aux_vector)
            f = aux_vector[u]["NFS"]
            g = aux_vector[v]["NFS"]
            s = aux_vector[u]["slice"]
            t = aux_vector[v]["slice"]


            if rand() < 0.25
                add_edge!(conflitGraphn, u, v)

            elseif  my_instance.setSlices[t].nodeSharing[s]*my_instance.setSlices[s].nodeSharing[t]*my_instance.setSlices[s].VNF_sharing["$(t)"][f][g]*my_instance.setSlices[t].VNF_sharing["$(s)"][f][g] ==0  
                add_edge!(conflitGraphn, u, v)

            else
                dim_f = 0.0
                dim_g = 0.0         
                if f!=1 && f<=my_instance.number_of_AN_based_NFs
                    dim_f = sum(my_instance.set_VNFs[f-1].compression*k["volume_of_data"] for k in my_instance.setSlices[s].set_commodities)/my_instance.set_VNFs[f].treatment_capacity    

                elseif f==1
                     dim_f = sum(k["volume_of_data"] for k in my_instance.setSlices[s].set_commodities)/ my_instance.set_VNFs[f].treatment_capacity    

                else
                   dim_f = my_instance.set_VNFs[f].dataVolPerUE*my_instance.setSlices[s].TotalAmountUE /my_instance.set_VNFs[f].treatment_capacity     
                end


                if g!=1 && g<=my_instance.number_of_AN_based_NFs
                    dim_g = sum(my_instance.set_VNFs[g-1].compression*k["volume_of_data"] for k in my_instance.setSlices[t].set_commodities)/my_instance.set_VNFs[g].treatment_capacity    

                elseif g==1
                     dim_g = sum(k["volume_of_data"] for k in my_instance.setSlices[t].set_commodities)/ my_instance.set_VNFs[g].treatment_capacity    

                else
                   dim_g = my_instance.set_VNFs[g].dataVolPerUE*my_instance.setSlices[t].TotalAmountUE /my_instance.set_VNFs[g].treatment_capacity      
                end


                if (dim_f+dim_g)*my_instance.set_VNFs[f].ram_request > min_RAM || (dim_f+dim_g)*my_instance.set_VNFs[f].cpu_request > min_CPU|| (dim_f+dim_g)*my_instance.set_VNFs[f].storage_request > min_storage
                    add_edge!(conflitGraphn, u,v )

                else 
                   for k in 1:length(my_instance.setSlices[s].set_commodities)
                      test = false                    
                    Threads.@threads  for l in 1:length(my_instance.setSlices[t].set_commodities)
                           if length(isect2(isect2(my_instance.setSlices[s].set_commodities[k]["paths"][my_instance.setSlices[s].set_commodities[k]["chosen_path"]],my_instance.setSlices[t].set_commodities[l]["paths"][my_instance.setSlices[t].set_commodities[l]["chosen_path"]]),my_instance.non_access)) ==0
                               add_edge!(conflitGraphn, u, v)
                                test = true
                                break
                            end
                        end
                        if test==true
                            break
                        end
                    end
                end
            end
        end
    end                        
    return conflitGraphn,aux_vector                        
    
end    

In [ ]:
using Base.Threads
function Parallel_color_cenntralized_graph(my_instance::Instance,tries::Int64)
    my_colorings = Vector{Dict}()
    for i in 1:tries
        push!(my_colorings,my_instance.centralized_coloring_solution)
        my_colorings[i]["tries"] = 0
    end

    Threads.@threads for shot in 1:tries
        while my_colorings[shot]["gap_maxClique"] > 0.0 && my_colorings[shot]["tries"] < 1000
            lets_color = greedy_color(my_colorings[shot]["centralized_graph"] ,sort_degree=false,reps = 20)

            if my_colorings[shot]["tries"] >0  && my_colorings[shot]["coloring"].num_colors >lets_color.num_colors
                my_colorings[shot]["coloring"] = deepcopy(lets_color)
                my_colorings[shot]["gap_maxClique"] = (my_colorings[shot]["coloring"].num_colors -  my_colorings[shot]["max_clique_size"])/my_colorings[shot]["coloring"].num_colors 
                my_colorings[shot]["best_try"] = my_colorings[shot]["tries"] + 1
            elseif my_colorings[shot]["tries"] ==0
                my_colorings[shot]["coloring"] = deepcopy(lets_color)
                my_colorings[shot]["gap_maxClique"] = (my_colorings[shot]["coloring"].num_colors -  my_colorings[shot]["max_clique_size"])/my_colorings[shot]["coloring"].num_colors 
                my_colorings[shot]["best_try"] = my_colorings[shot]["tries"] + 1
            end

            my_colorings[shot]["tries"]  += 1
        end 
    end
    best_shot = 0
    best_gap = Inf
    for shot in 1:tries
        if my_colorings[shot]["gap_maxClique"] < best_gap
           best_shot = shot
            best_gap = my_colorings[shot]["gap_maxClique"]
        end
    end
    my_instance.centralized_coloring_solution=deepcopy(my_colorings[best_shot])
end

In [ ]:
function pack_centralized_NFSs(my_instance::Instance)
    cpu = zeros(Float16, my_instance.centralized_coloring_solution["coloring"].num_colors, 1)
    ram = zeros(Float16, my_instance.centralized_coloring_solution["coloring"].num_colors, 1)
    storage = zeros(Float16, my_instance.centralized_coloring_solution["coloring"].num_colors, 1)
    for i in 1:length(my_instance.centralized_coloring_solution["centralized_packing"])
        n = my_instance.centralized_coloring_solution["coloring"].colors[my_instance.centralized_coloring_solution["centralized_packing"][i]["id"]]
        f = my_instance.centralized_coloring_solution["centralized_packing"][i]["NFS"]
        my_instance.centralized_coloring_solution["centralized_packing"][i]["NF"] = n
       cpu[n]+= my_instance.set_VNFs[f].cpu_request*my_instance.centralized_coloring_solution["centralized_packing"][i]["number_of_NFS_copies"]
       ram[n]+= my_instance.set_VNFs[f].ram_request*my_instance.centralized_coloring_solution["centralized_packing"][i]["number_of_NFS_copies"]
       storage[n]+= my_instance.set_VNFs[f].storage_request*my_instance.centralized_coloring_solution["centralized_packing"][i]["number_of_NFS_copies"]
    end    
    for n in 1:my_instance.centralized_coloring_solution["coloring"].num_colors
        if cpu[n]>my_instance.max_cpu ||  ram[n] > my_instance.max_ram || storage[n] > my_instance.max_storage
           my_instance.condition_NF_centr  = false 
           return 
        end
    end
    
    my_instance.condition_NF_centr = true

end

In [ ]:
function embbed_centralized_NFs(my_instance::Instance)
    cpu = zeros(Float16, props(my_instance.physical_network)[:number_nodes], 1)
    ram = zeros(Float16,props(my_instance.physical_network)[:number_nodes], 1)
    storage = zeros(Float16, props(my_instance.physical_network)[:number_nodes], 1)
    embbed_NF = zeros(Int64, my_instance.number_of_NFs, 1)
    for m in 1:my_instance.number_of_NFs
        embbed_NF[m] = rand(my_instance.selected_CUs["hosts"])
    end
    for i in 1:length(my_instance.centralized_coloring_solution["centralized_packing"])
        s =  my_instance.centralized_coloring_solution["centralized_packing"][i]["slice"]
        f =  my_instance.centralized_coloring_solution["centralized_packing"][i]["NFS"]
        
        #if my_instance.setSlices[s].cent_split_paths != "path_not_found"
             n =  my_instance.centralized_coloring_solution["centralized_packing"][i]["NF"]
            my_instance.centralized_coloring_solution["centralized_packing"][i]["phy_node"] = embbed_NF[n]
            u =  my_instance.centralized_coloring_solution["centralized_packing"][i]["phy_node"]
            nb_copies =  my_instance.centralized_coloring_solution["centralized_packing"][i]["number_of_NFS_copies"]
            my_instance.solution.var_x[s,f,n,u] = 1.0
            my_instance.solution.var_w[s,f,n,u] = nb_copies*my_instance.solution.var_x[s,f,n,u]
            my_instance.solution.var_y[f,n,u] = my_instance.solution.var_y[f,n,u]+ my_instance.solution.var_w[s,f,n,u] 
            cpu[u]+=my_instance.solution.var_w[s,f,n,u]*my_instance.set_VNFs[f].cpu_request
            ram[u]+=my_instance.solution.var_w[s,f,n,u]*my_instance.set_VNFs[f].ram_request
            storage[u]+=my_instance.solution.var_w[s,f,n,u]*my_instance.set_VNFs[f].storage_request
             if cpu[u]>props(my_instance.physical_network,u)[:cpu_capacity]  ||  ram[u] > props(my_instance.physical_network,u)[:ram_capacity] || storage[u] > props(my_instance.physical_network,u)[:storage_capacity] 
               my_instance.condition_centr_emb   =  false
                return 
            end
       # end 
    end
    check_physical_isolation(my_instance)
end


In [ ]:
function get_number_of_centralized_NFS_copies(my_instance::Instance,s::Int64,f::Int64)
    dim = 0.0
    if my_instance.set_VNFs[f].typee == :cp
      dim += my_instance.set_VNFs[f].dataVolPerUE*my_instance.setSlices[s].TotalAmountUE/my_instance.set_VNFs[f].treatment_capacity

    else
        if f!=1 
            dim += sum(my_instance.set_VNFs[f-1].compression*k["volume_of_data"]  for k in my_instance.setSlices[s].set_commodities)/my_instance.set_VNFs[f].treatment_capacity                
        else
            dim += sum(k["volume_of_data"] for  k in my_instance.setSlices[s].set_commodities)/my_instance.set_VNFs[f].treatment_capacity                
        end
    end
     return dim   

end


In [ ]:
function get_number_of_distributed_NFS_copies(my_instance::Instance,s::Int64,f::Int64,k::Int64)
    dim = 0.0
   
    if f!=1 
        dim += my_instance.set_VNFs[f-1].compression*my_instance.setSlices[s].set_commodities[k]["volume_of_data"] / my_instance.set_VNFs[f].treatment_capacity                
    else
        dim += my_instance.setSlices[s].set_commodities[k]["volume_of_data"] /my_instance.set_VNFs[f].treatment_capacity                
    end

     return dim   

end

In [ ]:
function isect2(g,m)
    is = []
    gset = Set(g)
    for i in m
        if i in gset
            push!(is,i)
        end
    end
    return is
end

In [ ]:
function round_up_y(my_instance::Instance)
    my_instance.solution.installed_NFSs = 0.0 
    count_nf = zeros(my_instance.number_of_NFs)
   Threads.@threads for n in 1:my_instance.number_of_NFs
        for f in 1:length(my_instance.set_VNFs), u in 1:props(my_instance.physical_network)[:number_nodes]
             my_instance.solution.var_y[f,n,u] = ceil(my_instance.solution.var_y[f,n,u])
             count_nf[n] += my_instance.solution.var_y[f,n,u]
        end
    end
    
    my_instance.solution.installed_NFSs = sum(count_nf[n] for n in 1:my_instance.number_of_NFs)
    
end

In [ ]:
function get_maxclique(conflitGraphn::SimpleGraph)
    my_model =Model(CPLEX.Optimizer)
    set_optimizer_attributes(my_model, "CPX_PARAM_TILIM" => 60*10)

    @variable(my_model,  x[u in 1:nv(conflitGraphn)],Bin)
    
    for u in 1:nv(conflitGraphn)-1, v in u +1:nv(conflitGraphn)
        if has_edge(conflitGraphn, u, v) == false
            @constraint(my_model, x[u] + x[v] <=1)
        end
    end
    
    A = zeros(Int8, nv(conflitGraphn), nv(conflitGraphn))
    for u in 1:nv(conflitGraphn)-1, v in u+1:nv(conflitGraphn)
        if has_edge(conflitGraphn, u, v) == false
          A[u,v] = 1
          A[v,u] = 1
        end    
    end
    
    UB = nv(conflitGraphn)-rank(A)/2
    #@constraint(my_model, sum(x[u] for u in 1:nv(conflitGraphn))<=maximum(degree(conflitGraphn))) 
    @constraint(my_model, sum(x[u] for u in 1:nv(conflitGraphn))<=UB) 
    
    @objective(my_model, Max, sum(x[u] for u in 1:nv(conflitGraphn))) 
    #Solving ILP
    mockModel = nothing
    mockModel=JuMP.backend(my_model).optimizer.model
    CPLEX.set_logfile(mockModel.inner.env,joinpath(result_folder,"LOG_MILP.log"))
    optimize!(my_model)

    return (round(objective_value(my_model)))

    
end

In [ ]:
function get_HEURISTIC_maxclique(conflitGraphn::SimpleGraph)
    
    max_tries = 10000
    my_cliques = Vector{ Vector{Int64} }() 
   for a in 1:max_tries                  
       a_clique = Vector{Int64}() 
       push!(my_cliques,a_clique)
    end
    
   Threads.@threads for a in 1:max_tries 
        
       for u in 1:nv(conflitGraphn)*10
            rand_vert = rand(1:nv(conflitGraphn)) 
            if length(my_cliques[a]) == 0
                push!(my_cliques[a],rand_vert)
            elseif (rand_vert ∈  my_cliques[a]) == false 
               test = true 
                for v in my_cliques[a]
                    if has_edge(conflitGraphn, rand_vert, v) == false && has_edge(conflitGraphn, v,rand_vert) == false
                        test = false
                    end
                end
                if test == true
                    push!(my_cliques[a],rand_vert)
                end
            end
        end
    end
    max_clique_size = 0
    for a in 1:max_tries                  
       if length(my_cliques[a]) > max_clique_size
          max_clique_size = length(my_cliques[a]) 
        end
    end
    return max_clique_size
end

In [ ]:
function pack_distributed_NFSs(my_instance::Instance)
  cpu = zeros(Float16, props(my_instance.physical_network)[:number_nodes], 1)
    ram = zeros(Float16,props(my_instance.physical_network)[:number_nodes], 1)
    storage = zeros(Float16, props(my_instance.physical_network)[:number_nodes], 1)   
    
    for i in 1:length(my_instance.distributed_coloring_solution["distributed_packing"])
        my_instance.distributed_coloring_solution["distributed_packing"][i]["NF"] = my_instance.distributed_coloring_solution["coloring"].colors[my_instance.distributed_coloring_solution["distributed_packing"][i]["id"]]
        s =  my_instance.distributed_coloring_solution["distributed_packing"][i]["slice"]
        f =  my_instance.distributed_coloring_solution["distributed_packing"][i]["function"]
        n =  my_instance.distributed_coloring_solution["distributed_packing"][i]["NF"]
        u =  my_instance.distributed_coloring_solution["distributed_packing"][i]["DU"]
        nb_copies =  my_instance.distributed_coloring_solution["distributed_packing"][i]["number_of_NFS_copies"]
        my_instance.solution.var_x[s,f,n,u] = 1.0
        my_instance.solution.var_w[s,f,n,u] = nb_copies*my_instance.solution.var_x[s,f,n,u]
        my_instance.solution.var_y[f,n,u] = my_instance.solution.var_y[f,n,u]+ my_instance.solution.var_w[s,f,n,u] 
        cpu[u]+=my_instance.solution.var_w[s,f,n,u]*my_instance.set_VNFs[f].cpu_request
        ram[u]+=my_instance.solution.var_w[s,f,n,u]*my_instance.set_VNFs[f].ram_request
        storage[u]+=my_instance.solution.var_w[s,f,n,u]*my_instance.set_VNFs[f].storage_request
        if cpu[u]>props(my_instance.physical_network,u)[:cpu_capacity]  ||  ram[u] > props(my_instance.physical_network,u)[:ram_capacity] || storage[u] > props(my_instance.physical_network,u)[:storage_capacity] 
            my_instance.condition_NF_dis =   false 
            return 
        end
    end 

        my_instance.condition_NF_dis =  true
end

In [ ]:
function get_conflit_distributed_graph(my_instance::Instance)
       
    #creating conflit graph
    aux_vector = Vector{Dict}()
    for s in 1:length(my_instance.setSlices),  k in my_instance.setSlices[s].set_commodities, u in 1:nv(my_instance.physical_network)
        if u == k["origin_node"]
            for f in 1:length(my_instance.set_VNFs)
                if  (my_instance.set_VNFs[f].typee == :dp && my_instance.solution.var_z[s,f] == 0) 
                    aux_dict = Dict()
                    aux_dict["id"] = length(aux_vector)+1
                    aux_dict["slice"] = s
                    aux_dict["function"] = f
                    aux_dict["DU"] = u
                    aux_dict["number_of_NFS_copies"] = get_number_of_distributed_NFS_copies(my_instance,s,f,k["commodity_id"])
                    aux_dict["commodity"] = k["commodity_id"]
                    aux_dict["NF"] = 0
                    push!(aux_vector, aux_dict)
                end
            end
        end
    end 
    distributedConflitGraphn = SimpleGraph(length(aux_vector))
     for u in 1:length(aux_vector)-1, v in u+1:length(aux_vector)
        f = aux_vector[u]["function"]
        g = aux_vector[v]["function"]
        s = aux_vector[u]["slice"]
        t = aux_vector[v]["slice"]
        DU_u = aux_vector[u]["DU"]
        DU_v = aux_vector[v]["DU"]

        #if f!=g && rand() < 0.5
           # add_edge!(distributedConflitGraphn, u, v)      
        if DU_u!=DU_v
            add_edge!(distributedConflitGraphn, u, v) 
        elseif  my_instance.setSlices[t].nodeSharing[s]*my_instance.setSlices[s].nodeSharing[t]*my_instance.setSlices[s].VNF_sharing["$(t)"][f][g]*my_instance.setSlices[t].VNF_sharing["$(s)"][f][g] ==0  
            add_edge!(distributedConflitGraphn, u, v)
        end
    end                        
    return distributedConflitGraphn,aux_vector                        
    
end    

In [ ]:
function color_distributed_graph(my_instance::Instance)
    my_instance.distributed_coloring_solution["tries"] = 0
    while my_instance.distributed_coloring_solution["gap_maxClique"] > 0.0 && my_instance.distributed_coloring_solution["tries"] < 1000
        lets_color = Parallel.greedy_color(my_instance.distributed_coloring_solution["distributed_graph"],sort_degree=false,reps =100)

        if my_instance.distributed_coloring_solution["tries"] >0  && my_instance.distributed_coloring_solution["coloring"].num_colors >lets_color.num_colors
            my_instance.distributed_coloring_solution["coloring"] = deepcopy(lets_color)
            my_instance.distributed_coloring_solution["gap_maxClique"] = (my_instance.distributed_coloring_solution["coloring"].num_colors -  my_instance.distributed_coloring_solution["max_clique_size"])/my_instance.distributed_coloring_solution["coloring"].num_colors 
            my_instance.distributed_coloring_solution["best_try"] = my_instance.distributed_coloring_solution["tries"] + 1
        else
            my_instance.distributed_coloring_solution["coloring"] =  deepcopy(lets_color)
            my_instance.distributed_coloring_solution["gap_maxClique"] = (my_instance.distributed_coloring_solution["coloring"].num_colors -  my_instance.distributed_coloring_solution["max_clique_size"])/my_instance.distributed_coloring_solution["coloring"].num_colors 
            my_instance.distributed_coloring_solution["best_try"] = my_instance.distributed_coloring_solution["tries"] + 1
        end

        my_instance.distributed_coloring_solution["tries"]  += 1
    end 
end

In [ ]:
using Base.Threads
function Parallel_color_distributed_graph(my_instance::Instance,tries::Int64)
    my_colorings = Vector{Dict}()
    for i in 1:tries
        push!(my_colorings,my_instance.distributed_coloring_solution)
        my_colorings[i]["tries"] = 0
    end

    Threads.@threads for shot in 1:tries
        while my_colorings[shot]["gap_maxClique"] > 0.0 && my_colorings[shot]["tries"] < 1000
            lets_color = greedy_color(my_colorings[shot]["distributed_graph"] ,sort_degree=false,reps = 20)

            if my_colorings[shot]["tries"] >0  && my_colorings[shot]["coloring"].num_colors >lets_color.num_colors
                my_colorings[shot]["coloring"] =  deepcopy(lets_color)
                my_colorings[shot]["gap_maxClique"] = (my_colorings[shot]["coloring"].num_colors -  my_colorings[shot]["max_clique_size"])/my_colorings[shot]["coloring"].num_colors 
                my_colorings[shot]["best_try"] = my_colorings[shot]["tries"] + 1
            elseif my_colorings[shot]["tries"] ==0
                my_colorings[shot]["coloring"] =  deepcopy(lets_color)
                my_colorings[shot]["gap_maxClique"] = (my_colorings[shot]["coloring"].num_colors -  my_colorings[shot]["max_clique_size"])/my_colorings[shot]["coloring"].num_colors 
                my_colorings[shot]["best_try"] = my_colorings[shot]["tries"] + 1
            end

            my_colorings[shot]["tries"]  += 1
        end 
    end
    best_shot = 0
    best_gap = Inf
    for shot in 1:tries
        if my_colorings[shot]["gap_maxClique"] < best_gap
           best_shot = shot
           best_gap = my_colorings[shot]["gap_maxClique"]
        end
    end
    my_instance.distributed_coloring_solution= deepcopy(my_colorings[best_shot])
end

In [ ]:
function get_bounds(my_instance::Instance)
    
    ram = 0
    cpu = 0
    storage = 0
    
    for u in 1:props(my_instance.physical_network)[:number_nodes]
        if ram < props(my_instance.physical_network,u)[:ram_cost] && props(my_instance.physical_network, u)[:node_type] != "app"
            ram = deepcopy(props(my_instance.physical_network,u)[:ram_cost]) 
        end
        if cpu < props(my_instance.physical_network,u)[:cpu_cost] && props(my_instance.physical_network, u)[:node_type] != "app"
            cpu = deepcopy(props(my_instance.physical_network,u)[:cpu_cost] )
        end
        if storage < props(my_instance.physical_network,u)[:storage_cost] && props(my_instance.physical_network, u)[:node_type] != "app"
            storage = deepcopy(props(my_instance.physical_network,u)[:storage_cost]) 
        end
    end
    
    
    cost_f_LB = []
    cost_f_UB = []
    dim_f_LB = []
    dim_f_UB = []
    for f in 1:length(my_instance.set_VNFs)
        dim1 = 0.0 
        dim2 = 0.0
        for  s in 1:length(my_instance.setSlices)
            if f!=1 && my_instance.set_VNFs[f].typee != :cp 
                dim1 += sum(my_instance.set_VNFs[f-1].compression*k["volume_of_data"]  for k in my_instance.setSlices[s].set_commodities)/my_instance.set_VNFs[f].treatment_capacity                
                dim2 += sum(ceil(my_instance.set_VNFs[f-1].compression*k["volume_of_data"]/my_instance.set_VNFs[f].treatment_capacity)  for k in my_instance.setSlices[s].set_commodities)                
            elseif f== 1  && my_instance.set_VNFs[f].typee != :cp 
                dim1 += sum(k["volume_of_data"] for  k in my_instance.setSlices[s].set_commodities)/my_instance.set_VNFs[f].treatment_capacity                                
                dim2 += sum(ceil(k["volume_of_data"]/my_instance.set_VNFs[f].treatment_capacity) for  k in my_instance.setSlices[s].set_commodities)                
            else
                dim1 += my_instance.setSlices[s].set_VNFs_to_install[f]*my_instance.set_VNFs[f].dataVolPerUE*my_instance.setSlices[s].TotalAmountUE/my_instance.set_VNFs[f].treatment_capacity
                dim2 += ceil(my_instance.setSlices[s].set_VNFs_to_install[f]*my_instance.set_VNFs[f].dataVolPerUE*my_instance.setSlices[s].TotalAmountUE/my_instance.set_VNFs[f].treatment_capacity)

            end
        end
        push!(cost_f_LB,ceil(dim1)*(ram+cpu+storage)) 
        push!(cost_f_UB,dim2*(ram+cpu+storage)) 
         push!(dim_f_LB,ceil(dim1)) 
        push!(dim_f_UB,dim2) 
    end

    return (sum(dim_f_LB[i] for i in 1:length(dim_f_LB)))
   # my_instance.solution.lower_bound =  sum(dim_f_LB[i] for i in 1:length(dim_f_LB))
    #my_instance.solution.upper_bound =  sum(dim_f_UB[i] for i in 1:length(dim_f_UB))

end

In [ ]:
function initiate_z_variables(my_instance::Instance)

    my_instance.solution.var_z = zeros(length(my_instance.setSlices),my_instance.number_of_AN_based_NFs)


end

In [ ]:
function initiate_x_y_w_variables(my_instance::Instance)

    my_instance.solution.var_x =   zeros(length(my_instance.setSlices),length(my_instance.set_VNFs), my_instance.number_of_NFs, props(my_instance.physical_network)[:number_nodes])
    my_instance.solution.var_y = zeros(length(my_instance.set_VNFs), my_instance.number_of_NFs,props(my_instance.physical_network)[:number_nodes])
    my_instance.solution.var_w = zeros(length(my_instance.setSlices),length(my_instance.set_VNFs), my_instance.number_of_NFs, props(my_instance.physical_network)[:number_nodes])
    

end

In [ ]:
function initiate_gamma_variables(my_instance::Instance)

    my_instance.solution.var_gamma = zeros(length(my_instance.setSlices), length(my_instance.setSlices[1].set_commodities),props(my_instance.physical_network)[:number_of_arcs],length(my_instance.set_VNFs)+1,length(my_instance.set_VNFs)+1)

end

In [ ]:
function build_paths_CP(my_instance::Instance, s::Int64)
    my_paths = Vector{Dict}()
    for f in 1:length(my_instance.set_VNFs), g in 1:length(my_instance.set_VNFs)
      #control plane
       if (f>my_instance.number_of_AN_based_NFs ||  g>my_instance.number_of_AN_based_NFs) && my_instance.VNF_connection["$(s)"][f][g] == true      
            for u in 1:props(my_instance.physical_network)[:number_nodes], v in 1:props(my_instance.physical_network)[:number_nodes]
                if u!= v && sum(my_instance.solution.var_x[s,f,n,u] for n in 1:my_instance.number_of_NFs) > 0.5 &&  sum(my_instance.solution.var_x[s,g,m,v] for m in 1:my_instance.number_of_NFs) > 0.5 
                    aux_Dict = Dict()
                    aux_Dict["NFS_origin"] = f
                    aux_Dict["NFS_target"] = g
                    aux_Dict["origin_node"] = u
                    aux_Dict["target_node"] = v
                    aux_Dict["slice"] = s
                    aux_Dict["type"] = "control_plane"
                    aux_Dict["traffic"] = 1
                    aux_Dict["volume"] = my_instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*my_instance.setSlices[s].TotalAmountUE
                    yen = yen_k_shortest_paths(my_instance.physical_network, u, v, LightGraphs.weights(my_instance.physical_network), 10, maxdist=my_instance.maxLatencyBetweenFunctions[f][g])
                    aux_yen = Vector{Vector{Int64}}()
                    for path in 1:length(yen.paths)
                        my_instance.test_Yen = true
                        for link in 1:length(yen.paths[path])-1
                            if get_prop(my_instance.physical_network,yen.paths[path][link],yen.paths[path][link+1],:max_bandwidth) < aux_Dict["volume"]
                                my_instance.test_Yen  = false
                                break
                            end
                        end
                        if my_instance.test_Yen  == true
                            push!(aux_yen,yen.paths[path])
                        end
                    end

                    if length(aux_yen)<1
                        my_instance.test_find_paths_flow = false
                        return
                    else
                        aux_Dict["paths"] = aux_yen
                        aux_Dict["chosen_path"] = 0
                        push!(my_paths,aux_Dict)
                    end
                end
            end
        #data plane = split     
        elseif  f<=my_instance.number_of_AN_based_NFs &&  g<=my_instance.number_of_AN_based_NFs && f == g-1 && my_instance.solution.var_z[s,f] < 0.5 && my_instance.solution.var_z[s,g] > 0.5
            for u in 1:props(my_instance.physical_network)[:number_nodes], v in 1:props(my_instance.physical_network)[:number_nodes]
                if u!= v && sum(my_instance.solution.var_x[s,f,n,u] for n in 1:my_instance.number_of_NFs) > 0.5 &&  sum(my_instance.solution.var_x[s,g,m,v] for m in 1:my_instance.number_of_NFs) > 0.5 
                    for k in my_instance.setSlices[s].set_commodities
                        if k["origin_node"] == u
                            aux_Dict = Dict()
                            aux_Dict["NFS_origin"] = f
                            aux_Dict["NFS_target"] = g
                            aux_Dict["origin_node"] = u
                            aux_Dict["target_node"] = v
                            aux_Dict["slice"] = s
                            aux_Dict["type"] = "data_plane"
                            aux_Dict["traffic"] = k["commodity_id"]
                            aux_Dict["volume"] = k["volume_of_data"]*my_instance.set_VNFs[f].compression
                            yen = yen_k_shortest_paths(my_instance.physical_network, u, v, LightGraphs.weights(my_instance.physical_network), 10, maxdist=my_instance.maxLatencyBetweenFunctions[f][g])
                            aux_yen = Vector{Vector{Int64}}()
                            for path in 1:length(yen.paths)
                                my_instance.test_Yen = true
                                for link in 1:length(yen.paths[path])-1
                                    if get_prop(my_instance.physical_network,yen.paths[path][link],yen.paths[path][link+1],:max_bandwidth) < aux_Dict["volume"]
                                        my_instance.test_Yen  = false
                                        break
                                    end
                                end
                                if my_instance.test_Yen  == true
                                    push!(aux_yen,yen.paths[path])
                                end
                            end

                            if length(aux_yen)<1
                                my_instance.test_find_paths_flow = false
                                return
                            else
                                aux_Dict["paths"] = aux_yen
                                aux_Dict["chosen_path"] = 0
                                push!(my_paths,aux_Dict)
                            end
                        end
                    end
                end
            end
          #data plane = centralized     
        elseif  f<=my_instance.number_of_AN_based_NFs &&  g<=my_instance.number_of_AN_based_NFs && f == g-1 && my_instance.solution.var_z[s,f] > 0.5 && my_instance.solution.var_z[s,g] > 0.5
            for u in 1:props(my_instance.physical_network)[:number_nodes], v in 1:props(my_instance.physical_network)[:number_nodes]
                if u!= v && sum(my_instance.solution.var_x[s,f,n,u] for n in 1:my_instance.number_of_NFs) > 0.5 &&  sum(my_instance.solution.var_x[s,g,m,v] for m in 1:my_instance.number_of_NFs) > 0.5
                    aux_Dict = Dict()
                    aux_Dict["NFS_origin"] = f
                    aux_Dict["NFS_target"] = g
                    aux_Dict["origin_node"] = u
                    aux_Dict["target_node"] = v
                    aux_Dict["slice"] = s
                    aux_Dict["type"] = "data_plane"
                    aux_Dict["traffic"] = 1
                    aux_Dict["volume"] = sum(my_instance.set_VNFs[f].compression*k["volume_of_data"] for k in my_instance.setSlices[s].set_commodities)
                    yen = yen_k_shortest_paths(my_instance.physical_network, u, v, LightGraphs.weights(my_instance.physical_network), 10, maxdist=my_instance.maxLatencyBetweenFunctions[f][g])
                    aux_yen = Vector{Vector{Int64}}()
                    for path in 1:length(yen.paths)
                        my_instance.test_Yen  = true
                        for link in 1:length(yen.paths[path])-1
                            if get_prop(my_instance.physical_network,yen.paths[path][link],yen.paths[path][link+1],:max_bandwidth) < aux_Dict["volume"]
                                my_instance.test_Yen  = false
                                break
                            end
                        end
                        if my_instance.test_Yen  == true
                            push!(aux_yen,yen.paths[path])
                        end
                    end

                    if length(aux_yen)<1
                        my_instance.test_find_paths_flow = false
                        return
                    else
                        aux_Dict["paths"] = aux_yen
                        aux_Dict["chosen_path"] = 0
                        push!(my_paths,aux_Dict)
                    end
                end
            end
        end
    end
    #btw each o_k and first data plane NFS 
    if my_instance.solution.var_z[s,1] > 0.5
        node_found = 0
        for v in my_instance.non_access
            if sum(my_instance.solution.var_x[s,1,n,v] for n in 1:my_instance.number_of_NFs) > 0.5
                node_found = v
            end
        end
        for k in my_instance.setSlices[s].set_commodities
            aux_Dict = Dict()
            aux_Dict["NFS_origin"] = length(my_instance.set_VNFs)+1
            aux_Dict["NFS_target"] = 1
            aux_Dict["origin_node"] = k["origin_node"]
            aux_Dict["target_node"] = node_found
            aux_Dict["slice"] = s
            aux_Dict["type"] = "data_plane"
            aux_Dict["traffic"] = k["commodity_id"]
            aux_Dict["volume"] = k["volume_of_data"]
            yen= yen_k_shortest_paths(my_instance.physical_network, k["origin_node"], node_found, LightGraphs.weights(my_instance.physical_network), 10, maxdist=my_instance.setSlices[s].maxLatencyDataLayer)
            aux_yen = Vector{Vector{Int64}}()
            for path in 1:length(yen.paths)
                my_instance.test_Yen  = true
                for link in 1:length(yen.paths[path])-1
                    if get_prop(my_instance.physical_network,yen.paths[path][link],yen.paths[path][link+1],:max_bandwidth) < aux_Dict["volume"]
                        my_instance.test_Yen  = false
                        break
                    end
                end
                if my_instance.test_Yen  == true
                    push!(aux_yen,yen.paths[path])
                end
            end
            
            if length(aux_yen)<1
                my_instance.test_find_paths_flow = false
                return
            else
                aux_Dict["paths"] = aux_yen
                aux_Dict["chosen_path"] = 0
                push!(my_paths,aux_Dict)
            end
        end
    end
    #and the last data plane NFS and to each t_k
    if my_instance.solution.var_z[s,my_instance.number_of_AN_based_NFs] > 0.5
        node_found = 0
        for v in my_instance.non_access
            if sum(my_instance.solution.var_x[s,my_instance.number_of_AN_based_NFs,n,v] for n in 1:my_instance.number_of_NFs) > 0.5
                node_found = v
            end
        end
        for k in my_instance.setSlices[s].set_commodities
            aux_Dict = Dict()
            aux_Dict["NFS_origin"] = my_instance.number_of_AN_based_NFs 
            aux_Dict["NFS_target"] = length(my_instance.set_VNFs)+1
            aux_Dict["origin_node"] = node_found
            aux_Dict["target_node"] = k["target_node"] 
            aux_Dict["slice"] = s
            aux_Dict["type"] = "data_plane"
            aux_Dict["traffic"] = k["commodity_id"]
            aux_Dict["volume"] = k["volume_of_data"]*my_instance.set_VNFs[my_instance.number_of_AN_based_NFs].compression
            yen = yen_k_shortest_paths(my_instance.physical_network, node_found,k["target_node"], LightGraphs.weights(my_instance.physical_network), 10, maxdist=my_instance.setSlices[s].maxLatencyDataLayer)
            aux_yen = Vector{Vector{Int64}}()
            for path in 1:length(yen.paths)
                my_instance.test_Yen  = true
                for link in 1:length(yen.paths[path])-1
                    if get_prop(my_instance.physical_network,yen.paths[path][link],yen.paths[path][link+1],:max_bandwidth) < aux_Dict["volume"]
                        my_instance.test_Yen  = false
                        break
                    end
                end
                if my_instance.test_Yen  == true
                    push!(aux_yen,yen.paths[path])
                end
            end
            
            if length(aux_yen)<1
                my_instance.test_find_paths_flow = false
                return
            else
                aux_Dict["paths"] = aux_yen
                aux_Dict["chosen_path"] = 0
                push!(my_paths,aux_Dict)
            end
        end    
    else
        for k in my_instance.setSlices[s].set_commodities
            aux_Dict = Dict()
            aux_Dict["NFS_origin"] = my_instance.number_of_AN_based_NFs 
            aux_Dict["NFS_target"] = length(my_instance.set_VNFs)+1
            aux_Dict["origin_node"] = k["origin_node"] 
            aux_Dict["target_node"] = k["target_node"] 
            aux_Dict["slice"] = s
            aux_Dict["type"] = "data_plane"
            aux_Dict["traffic"] = k["commodity_id"]
            aux_Dict["volume"] = k["volume_of_data"]*my_instance.set_VNFs[my_instance.number_of_AN_based_NFs].compression
            yen = yen_k_shortest_paths(my_instance.physical_network, k["origin_node"], k["target_node"], LightGraphs.weights(my_instance.physical_network), 10, maxdist=my_instance.setSlices[s].maxLatencyDataLayer)
            aux_yen = Vector{Vector{Int64}}()
            for path in 1:length(yen.paths)
                my_instance.test_Yen  = true
                for link in 1:length(yen.paths[path])-1
                    if get_prop(my_instance.physical_network,yen.paths[path][link],yen.paths[path][link+1],:max_bandwidth) < aux_Dict["volume"]
                        my_instance.test_Yen  = false
                        break
                    end
                end
                if my_instance.test_Yen  == true
                    push!(aux_yen,yen.paths[path])
                end
            end
            
            if length(aux_yen)<1
                my_instance.test_find_paths_flow=  false
                return
            else
                aux_Dict["paths"] = aux_yen
                aux_Dict["chosen_path"] = 0
                push!(my_paths,aux_Dict)
            end
        end    
    end
    #verificar capacidade, se no fim nao tiver nenhum path, para e retorna erro
    
    
    
    my_instance.setSlices[s].paths_btw_active_flows = deepcopy(my_paths)
    

end



In [ ]:
function verify_capacity(my_instance::Instance) 
    
    band = zeros(Float16, length(edges(my_instance.physical_network)), 1)
    my_instance.check_capacity_flow = true
    for slice in 1:length(my_instance.setSlices), flow in 1:length(my_instance.setSlices[slice].paths_btw_active_flows)
        choosen_path = my_instance.setSlices[slice].paths_btw_active_flows[flow]["paths"][my_instance.setSlices[slice].paths_btw_active_flows[flow]["chosen_path"]]
        volume = my_instance.setSlices[slice].paths_btw_active_flows[flow]["volume"]
        for link in 1:length(choosen_path)-1
            if  band[get_prop(my_instance.physical_network,choosen_path[link],choosen_path[link+1],:edge_id)] + volume <= get_prop(my_instance.physical_network,choosen_path[link],choosen_path[link+1],:max_bandwidth)
                band[get_prop(my_instance.physical_network,choosen_path[link],choosen_path[link+1],:edge_id)] += volume
                k = my_instance.setSlices[slice].paths_btw_active_flows[flow]["traffic"]
                a = get_prop(my_instance.physical_network,choosen_path[link],choosen_path[link+1],:edge_id)
                f = my_instance.setSlices[slice].paths_btw_active_flows[flow]["NFS_origin"]
                g = my_instance.setSlices[slice].paths_btw_active_flows[flow]["NFS_target"]
                my_instance.solution.var_gamma[slice,k,a,f,g] = 1
            else
                my_instance.check_capacity_flow = false                
                return
            end
        end
    end
    
     my_instance.check_capacity_flow = true
end

In [ ]:
function verify_e2e_latency(my_instance::Instance) 
    for slice in 1:length(my_instance.setSlices)
        e2e_lat = zeros(Float16, length(my_instance.setSlices[slice].set_commodities), 1)
        Threads.@threads for k  in my_instance.setSlices[slice].set_commodities
            for flow in 1:length(my_instance.setSlices[slice].paths_btw_active_flows)
                if  my_instance.setSlices[slice].paths_btw_active_flows[flow]["traffic"] ==  k["commodity_id"]
                    if my_instance.setSlices[slice].paths_btw_active_flows[flow]["NFS_target"] <= my_instance.number_of_AN_based_NFs && my_instance.setSlices[slice].paths_btw_active_flows[flow]["NFS_target"] == my_instance.setSlices[slice].paths_btw_active_flows[flow]["NFS_origin"] + 1
                        choosen_path = my_instance.setSlices[slice].paths_btw_active_flows[flow]["paths"][my_instance.setSlices[slice].paths_btw_active_flows[flow]["chosen_path"]]
                        max_latency = my_instance.setSlices[slice].maxLatencyDataLayer
                        for link in 1:length(choosen_path)-1
                            if  get_prop(my_instance.physical_network,choosen_path[link],choosen_path[link+1],:delay) + e2e_lat[k["commodity_id"]]  <= max_latency
                                e2e_lat[k["commodity_id"]] += get_prop(my_instance.physical_network,choosen_path[link],choosen_path[link+1],:delay)
                                
                            else
                                my_instance.check_e2e_lat = false
                                return
                            end
                        end
                    end
                end
            end
        end
    end
    
    my_instance.check_e2e_lat = true
end

In [ ]:
function choose_path_flows(my_instance::Instance) 
    

    for slice in 1:length(my_instance.setSlices)
       Threads.@threads for flow in 1:length(my_instance.setSlices[slice].paths_btw_active_flows)
            my_instance.setSlices[slice].paths_btw_active_flows[flow]["chosen_path"] = rand(1:length(my_instance.setSlices[slice].paths_btw_active_flows[flow]["paths"]))
        end
    end
    
end   


In [ ]:
function check_solution(instance::Instance)
    
  
     
    for u in 1: props(instance.physical_network)[:number_nodes]
        if props(instance.physical_network, u)[:node_type] == "non_access"
           for  s in 1:length(instance.setSlices)-1, t in s+1:length(instance.setSlices), f in 1:length(instance.set_VNFs), g in 1:length(instance.set_VNFs)
                if  sum(instance.best_solution.var_x[s,f,n,u] for n in 1:instance.number_of_NFs) + sum(instance.best_solution.var_x[t,g,m,u] for m in 1:instance.number_of_NFs)>instance.setSlices[s].nodeSharing[t]*instance.setSlices[t].nodeSharing[s] +1.1 
                    println("$(sum(instance.best_solution.var_x[s,f,n,u] for n in 1:instance.number_of_NFs) + sum(instance.best_solution.var_x[t,g,m,u] for m in 1:instance.number_of_NFs)) et $(instance.setSlices[s].nodeSharing[t]*instance.setSlices[t].nodeSharing[s] +1 )")
                end
            end    
        end
    end



end

In [ ]:
function check_physical_isolation(my_instance::Instance)
    
       
    for u in 1: props(my_instance.physical_network)[:number_nodes]
        if props(my_instance.physical_network, u)[:node_type] == "non_access"
           for  s in 1:length(my_instance.setSlices)-1, t in s+1:length(my_instance.setSlices), f in 1:length(my_instance.set_VNFs), g in 1:length(my_instance.set_VNFs)
                if  sum(my_instance.solution.var_x[s,f,n,u] for n in 1:my_instance.number_of_NFs) + sum(my_instance.solution.var_x[t,g,m,u] for m in 1:my_instance.number_of_NFs)>my_instance.setSlices[s].nodeSharing[t]*my_instance.setSlices[t].nodeSharing[s] +1 
                    my_instance.condition_centr_emb   =  false
                    return
                end
            end    
        end
    end
  my_instance.condition_centr_emb   =  true

end

In [ ]:
function solve_NSDP_heuristic(input_folder::String,number_of_instances::Int64)
    my_instance = get_Instance(input_folder,number_of_instances)  
    solveTime = 0.0
    my_instance.stop_condition = false
    my_instance.number_solution_tries = 0
    my_instance.solution.number_solution = 0
    my_instance.solution.time_found = 0.0
    my_instance.solution.av_solution_cost = 0.0
    while  my_instance.stop_condition != true #1
        my_instance.number_solution_tries+= 1
        my_instance.path_condition = false
        my_instance.tries_choose_CUs = 1
        while my_instance.path_condition != true #2

            my_instance.solution.time_found += @elapsed begin #3
                initiate_z_variables(my_instance)
                #getting the minimal number of host CUs. This number is calculated only once
                if my_instance.tries_choose_CUs ==1 #4
                    get_minimal_number_of_CUs(my_instance)
                    get_CUs(my_instance,min(my_instance.number_of_hosts*2,length(my_instance.non_access)))
                end#4
                #getting host CUs
                if length(my_instance.selected_CUs) < length(my_instance.non_access) && my_instance.tries_choose_CUs > 1 #4
                    get_CUs(my_instance,min(my_instance.number_of_hosts*2,length(my_instance.non_access)))
                end#4
                Threads.@threads for ss in 1:length(my_instance.setSlices) #4
                    get_paths(my_instance,ss)
                end #4
                random_choose_paths(my_instance)
                select_split_setting(my_instance)
            end#3

            my_instance.solution.time_found += @elapsed begin #4
                centralized_graph = 0.0
                centralized_dict = 0.0
                centralized_graph,centralized_dict = get_centralized_conflit_graph(my_instance)
                aux_Dict = Dict()    
                aux_Dict["gap_maxClique"] = 1.0
                aux_Dict["coloring"] = 1
                aux_Dict["tries"]  = 0 
                aux_Dict["best_try"] = 1                   
                aux_Dict["max_clique_size"] =  0 #get_HEURISTIC_maxclique(centralized_graph)#get_maxclique(centralized_graph)
                aux_Dict["centralized_graph"] = centralized_graph
                aux_Dict["centralized_packing"] = centralized_dict
                my_instance.my_condition_packing_embedding_cent = false
                my_instance.my_tries_packing_embedding_cent = 0

                while (my_instance.my_tries_packing_embedding_cent <= 1000) && (my_instance.my_condition_packing_embedding_cent != true) #5
                    my_instance.centralized_coloring_solution = deepcopy(aux_Dict) 
                    initiate_x_y_w_variables(my_instance)       
                    Parallel_color_cenntralized_graph(my_instance,1)
                    pack_centralized_NFSs(my_instance)
                    if  my_instance.condition_NF_centr == true #6
                        embbed_centralized_NFs(my_instance)
                        if  my_instance.condition_centr_emb == true #7
                            my_instance.my_condition_packing_embedding_cent = true
                        end #7
                    end #6
                    my_instance.my_tries_packing_embedding_cent +=  1
                end#5

                centralized_graph = nothing
                centralized_dict = nothing
                aux_Dict = nothing

                distributed_graph = 0.0
                distributed_dict = 0.0
                my_instance.my_tries_packing_embedding_dist = 0
                my_instance.my_condition_packing_embedding_dist = false
                distributed_graph,distributed_dict = get_conflit_distributed_graph(my_instance)
                if nv( distributed_graph) > 0#5
                        aux_Dict = Dict()    
                        aux_Dict["gap_maxClique"] = 1.0
                        aux_Dict["coloring"] = 1
                        aux_Dict["tries"]  = 0 
                        aux_Dict["best_try"] = 1                   
                        aux_Dict["max_clique_size"] =  0#get_HEURISTIC_maxclique(distributed_graph) #get_maxclique(distributed_graph)
                        aux_Dict["distributed_graph"] = distributed_graph
                        aux_Dict["distributed_packing"] = distributed_dict

                        while my_instance.my_tries_packing_embedding_dist < 1000 && my_instance.my_condition_packing_embedding_dist  !=true#6
                            my_instance.distributed_coloring_solution = deepcopy(aux_Dict) 
                            Parallel_color_distributed_graph(my_instance,1)
                            pack_distributed_NFSs(my_instance)
                            if my_instance.condition_NF_dis  == true  #7
                                my_instance.my_condition_packing_embedding_dist  = true
                            else
                                my_instance.my_tries_packing_embedding_dist +=1
                            end#7
                        end#6
                        #freeing memory
                        aux_Dict = nothing
                        distributed_graph = nothing
                        distributed_dict = nothing
                else

                    aux_Dict = Dict()    
                    aux_Dict["gap_maxClique"] = 1.0
                    aux_Dict["coloring"] = 1
                    aux_Dict["tries"]  = 0 
                    aux_Dict["best_try"] = 1                   
                    aux_Dict["max_clique_size"] = -20
                    aux_Dict["distributed_graph"] = distributed_graph
                    aux_Dict["distributed_packing"] = distributed_dict
                    my_instance.distributed_coloring_solution = deepcopy(aux_Dict) 
                    #freeing memory
                    aux_Dict = nothing
                    distributed_graph = nothing
                    distributed_dict = nothing
                end#5

                my_instance.test_find_paths_flow =  true 
                Threads.@threads for slice in 1:length(my_instance.setSlices)
                    build_paths_CP(my_instance,slice) 
                    if my_instance.test_find_paths_flow == false
                        break
                    end 
                end  
            end# 4
            if my_instance.test_find_paths_flow  == true #4

                my_instance.path_tries = 1
                my_instance.path_condition = false

                while my_instance.path_condition != true && my_instance.path_tries < 1000 #5
                    my_instance.solution.time_found += @elapsed begin #6
                        initiate_gamma_variables(my_instance)
                        choose_path_flows(my_instance) 
                        verify_capacity(my_instance) 
                    end#6
                    if  my_instance.check_capacity_flow == true #6
                        my_instance.solution.time_found += @elapsed begin #7
                            verify_e2e_latency(my_instance) 
                        end #7
                        if my_instance.check_e2e_lat == true  #7

                            my_instance.path_condition =  true
                            my_instance.solution.number_solution+=1
                            my_instance.solution.time_found += @elapsed begin #8
                                round_up_y(my_instance)
                                get_solution_cost(my_instance)
                                my_instance.solution.av_solution_cost += my_instance.solution.solution_cost
                            end #8
                            if my_instance.solution.number_solution <= 1 #8
                                my_instance.best_solution = deepcopy(my_instance.solution)
                                #println("first solution $(my_instance.best_solution.solution_cost)")
                            elseif my_instance.solution.solution_cost < my_instance.best_solution.solution_cost
                                my_instance.best_solution = deepcopy(my_instance.solution)
                               # println("new best solution  $(my_instance.best_solution.solution_cost)")
                            end   #8
                            if  rand() < (1 - 60/my_instance.solution.time_found)#8
                                my_instance.stop_condition = true
                            end#8
                        end#7
                    end #6
                    my_instance.path_tries += 1      
                end # 5
            end # 4
            if my_instance.solution.time_found > 10000#4
                my_instance.stop_condition = true
                my_instance.solution.number_solution+=1
                my_instance.
                my_instance.solution.av_solution_cost = my_instance.solution.av_solution_cost/my_instance.solution.number_solution
                return my_instance
            end #4
        end#3
    end# 2

my_instance.solution.av_solution_cost = my_instance.solution.av_solution_cost/my_instance.solution.number_solution
return my_instance
end


In [ ]:
using JuMP,CPLEX,MathProgBase,GLPK
using NBInclude,GraphIO
using LightGraphs, MetaGraphs,LightGraphsFlows  

function get_solution_heu(my_instance::Instance,folder::String,lati::String,test::Int64,my_parameters::Parameters)
  
#-------------------- GETTING SOLUTION -----------------------------------      
    sol_z = my_instance.best_solution.var_z
    sol_x = my_instance.best_solution.var_x
    sol_y = my_instance.best_solution.var_y
    
    sol_w = my_instance.best_solution.var_w 
    sol_gamma = my_instance.best_solution.var_gamma 
   
    Objective_value =  my_instance.best_solution.solution_cost

#-------------------- GETTING NODE LOAD AND NUMBER OF NFS -----------------------------------      
    #auxiliary variables
    charge_node = Array{Float64}(undef, props(my_instance.physical_network)[:number_nodes])
    total_nodes_capacity = 0.0
    most_loaded = 0.0 
    total_load = 0.0
    number_NFSs = 0.0
    ppn = 0.0
    number_active_node = 0
    dis_NFS = 0
    cent_NFS = 0

    for u in 1: props(my_instance.physical_network)[:number_nodes]
        charge_node[u] = 0.0
        total_nodes_capacity+=props(my_instance.physical_network,u)[:cpu_capacity]
        if props(my_instance.physical_network, u)[:node_type] != "app"
            for f in 1:length(my_instance.set_VNFs),  m in 1:my_instance.number_of_NFs
                charge_node[u] += (sol_y[f,m,u]*my_instance.set_VNFs[f].cpu_request)/props(my_instance.physical_network,u)[:cpu_capacity]
                total_load += (sol_y[f,m,u]*my_instance.set_VNFs[f].cpu_request)
                number_NFSs += sol_y[f,m,u]
                if props(my_instance.physical_network, u)[:node_type] == "non_access"
                    cent_NFS += sol_y[f,m,u]
                else
                    dis_NFS += sol_y[f,m,u]
                end
            end
        end
        if charge_node[u] > 0
            number_active_node += 1
            ppn+=charge_node[u]
            if charge_node[u] > most_loaded
                most_loaded = charge_node[u]
            end
        end
    end
    total_load = total_load/total_nodes_capacity


  #-------------------- GETTING LINK metrics  -----------------------------------      
    #auxiliary variables
    charge_arc = Array{Float64}(undef, props(my_instance.physical_network)[:number_of_arcs])
    total_arc_capacity = 0.0
    most_loaded_arc = 0.0 
    total_arc_load = 0.0
    number_active_links = 0
    ppa = 0.0
    ver = 0.0
    
    for a in edges(my_instance.physical_network) 
        edge_id = get_prop(my_instance.physical_network,a,:edge_id)
        charge_arc[edge_id] = 0.0
        total_arc_capacity+= get_prop(my_instance.physical_network,a,:max_bandwidth)

        for s in 1:length(my_instance.setSlices), k in 1:length(my_instance.setSlices[s].set_commodities)

            charge_arc[edge_id] += sol_gamma[s,k,edge_id,length(my_instance.set_VNFs)+1,1]*my_instance.setSlices[s].set_commodities[k]["volume_of_data"]

            charge_arc[edge_id] += sol_gamma[s,k,edge_id,my_instance.number_of_AN_based_NFs,length(my_instance.set_VNFs)+1]*my_instance.setSlices[s].set_commodities[k]["volume_of_data"]*my_instance.set_VNFs[my_instance.number_of_AN_based_NFs].compression

            for f in 1:(my_instance.number_of_AN_based_NFs+my_instance.number_of_CN_based_NFs), g in 1:(my_instance.number_of_AN_based_NFs +my_instance.number_of_CN_based_NFs)
                if f!=g 
                    if f<=my_instance.number_of_AN_based_NFs&&g<=my_instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g] >0.9
                    charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*my_instance.setSlices[s].set_commodities[k]["volume_of_data"]*my_instance.set_VNFs[f].compression
                    end
                    if f>my_instance.number_of_AN_based_NFs && g>my_instance.number_of_AN_based_NFs && sol_gamma[s,1,edge_id,f,g]>=0.9
                        charge_arc[edge_id] += sol_gamma[s,1,edge_id,f,g]*my_instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(my_instance.setSlices[s].TotalAmountUE/length(my_instance.setSlices[s].set_commodities))
                    elseif f>my_instance.number_of_AN_based_NFs && g<=my_instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g]>=0.9
                     charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*my_instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(my_instance.setSlices[s].TotalAmountUE/length(my_instance.setSlices[s].set_commodities))
                    elseif f<=my_instance.number_of_AN_based_NFs && g>my_instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g]>=0.9
                        charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*my_instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(my_instance.setSlices[s].TotalAmountUE/length(my_instance.setSlices[s].set_commodities))

                    end
                end
            end
        end


        total_arc_load+= charge_arc[edge_id]
         charge_arc[edge_id] =  charge_arc[edge_id]/get_prop(my_instance.physical_network,a,:max_bandwidth) 
        if charge_arc[edge_id] > 0
            number_active_links += 1
            ppa+=charge_arc[edge_id]
            if charge_arc[edge_id]  > most_loaded_arc
                most_loaded_arc = charge_arc[edge_id] 
            end
        end
    end
    total_arc_load = total_arc_load/total_arc_capacity
    avr_lat = 0.0            
    lat = zeros(Float64, 32, 8)            
    for s in 1:length(my_instance.setSlices), k in 1:length(my_instance.setSlices[s].set_commodities)
          
      lat[s,k] += sum(get_prop(my_instance.physical_network,a,:delay) *sol_gamma[s,k,get_prop(my_instance.physical_network,a,:edge_id),f,f+1] for f in 1:my_instance.number_of_AN_based_NFs-1,a in edges(my_instance.physical_network))
      lat[s,k] += sum(get_prop(my_instance.physical_network,a,:delay)*sol_gamma[s,k,get_prop(my_instance.physical_network,a,:edge_id),my_instance.number_of_AN_based_NFs,length(my_instance.set_VNFs)+1] for f in 1:my_instance.number_of_AN_based_NFs,a in edges(my_instance.physical_network))
      lat[s,k] += sum(get_prop(my_instance.physical_network,a,:delay)*sol_gamma[s,k,get_prop(my_instance.physical_network,a,:edge_id),length(my_instance.set_VNFs)+1,1] for f in 1:my_instance.number_of_AN_based_NFs,a in edges(my_instance.physical_network))
    
      end
    for s in 1:length(my_instance.setSlices), k in 1:length(my_instance.setSlices[s].set_commodities)
      avr_lat += lat[s,k]/32          
    end          

#-------------------- PRINTING -----------------------------------      
        
    open(joinpath(result_folder,"load_on_arcs.csv"), "a") do iooo
        open(joinpath(result_folder,"final_statistics.csv"), "a") do io
            open(joinpath(result_folder,"load_on_nodes.csv"), "a") do ioo              

          
                 write(io,"$(test);Heuristic;$(my_parameters.class);$(my_parameters.graph_density);$(my_parameters.link_capacity_ratio);$(my_parameters.prob_sharing_NFS);")
                


                write(io,"$(Objective_value);")   
                write(io,"$(number_NFSs);")
                write(io,"$(dis_NFS);")
                write(io,"$(cent_NFS);")


                write(io,"$(props(my_instance.physical_network)[:number_nodes]);")
                write(io,"$(number_active_node);")
                write(io,"$(number_active_node/props(my_instance.physical_network)[:number_nodes]);")

                write(io,"$(total_nodes_capacity);")
                write(io,"$(total_load);")
                write(io,"$(ppn/number_active_node);")
                write(io,"$(most_loaded);")


                write(io,"$(props(my_instance.physical_network)[:number_of_arcs]);")
                write(io,"$(number_active_links);")
                write(io,"$(number_active_links/props(my_instance.physical_network)[:number_of_arcs]);")

                write(io,"$(total_arc_capacity);")
                write(io,"$(total_arc_load);")
                write(io,"$(ppa/number_active_links);")
                write(io,"$(most_loaded_arc);")
                write(io,"$(avr_lat)\n")

                for u in 1:props(my_instance.physical_network)[:number_nodes]
                  node_type = props(my_instance.physical_network, u)[:node_type] 
                        write(ioo,"$(test);Heuristic;$(u);$(node_type);$(my_parameters.class);$(my_parameters.graph_density);$(my_parameters.link_capacity_ratio);$(charge_node[u]);$(my_parameters.prob_sharing_NFS)\n")
                    
                end

                for a in edges(my_instance.physical_network) 
                    edge_id = get_prop(my_instance.physical_network,a,:edge_id)
                    edge_type = ""           
                    if props(my_instance.physical_network, src(a))[:node_type] == "access" ||  props(my_instance.physical_network, dst(a))[:node_type]  == "access"         
                        edge_type = "fronthaul"
                    else        
                        edge_type = "backhaul"
                    end               
                        write(iooo,"$(test);Heuristic;$(edge_id);$(edge_type);$(my_parameters.class);$(my_parameters.graph_density);$(my_parameters.link_capacity_ratio);$(charge_arc[edge_id]);$(my_parameters.prob_sharing_NFS)\n")
                end
            end 
        end   
    end

    
                                                    
        
end

function get_solution_mod(my_model::Model, instance::Instance,folder::String,lati::String,test::Int64,my_parameters::Parameters)
  
#-------------------- GETTING SOLUTION -----------------------------------      
    sol_z = JuMP.value.(getindex(my_model,:z)) 
    sol_x = JuMP.value.(getindex(my_model,:x)) 
    sol_y = JuMP.value.(getindex(my_model,:y)) 
    
    sol_w = JuMP.value.(getindex(my_model,:w)) 
    sol_gamma = JuMP.value.(getindex(my_model,:gamma)) 
   
    Objective_value =  JuMP.objective_value(my_model)

#-------------------- GETTING NODE LOAD AND NUMBER OF NFS -----------------------------------      
    #auxiliary variables
    charge_node = Array{Float64}(undef, props(instance.physical_network)[:number_nodes])
    total_nodes_capacity = 0.0
    most_loaded = 0.0 
    total_load = 0.0
    number_NFSs = 0.0
    ppn = 0.0
    number_active_node = 0
    dis_NFS = 0
    cent_NFS = 0

    for u in 1: props(instance.physical_network)[:number_nodes]
        charge_node[u] = 0.0
        total_nodes_capacity+=props(instance.physical_network,u)[:cpu_capacity]
        if props(instance.physical_network, u)[:node_type] != "app"
            for f in 1:length(instance.set_VNFs),  m in 1:instance.number_of_NFs
                charge_node[u] += (sol_y[f,m,u]*instance.set_VNFs[f].cpu_request)/props(instance.physical_network,u)[:cpu_capacity]
                total_load += (sol_y[f,m,u]*instance.set_VNFs[f].cpu_request)
                number_NFSs += sol_y[f,m,u]
                if props(instance.physical_network, u)[:node_type] == "non_access"
                    cent_NFS += sol_y[f,m,u]
                else
                    dis_NFS += sol_y[f,m,u]
                end
            end
        end
        if charge_node[u] > 0
            number_active_node += 1
            ppn+=charge_node[u]
            if charge_node[u] > most_loaded
                most_loaded = charge_node[u]
            end
        end
    end
    total_load = total_load/total_nodes_capacity


  #-------------------- GETTING LINK metrics  -----------------------------------      
    #auxiliary variables
    charge_arc = Array{Float64}(undef, props(instance.physical_network)[:number_of_arcs])
    total_arc_capacity = 0.0
    most_loaded_arc = 0.0 
    total_arc_load = 0.0
    number_active_links = 0
    ppa = 0.0
    ver = 0.0
    
    for a in edges(instance.physical_network) 
        edge_id = get_prop(instance.physical_network,a,:edge_id)
        charge_arc[edge_id] = 0.0
        total_arc_capacity+= get_prop(instance.physical_network,a,:max_bandwidth)

        for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities)

            charge_arc[edge_id] += sol_gamma[s,k,edge_id,length(instance.set_VNFs)+1,1]*instance.setSlices[s].set_commodities[k]["volume_of_data"]

            charge_arc[edge_id] += sol_gamma[s,k,edge_id,instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1]*instance.setSlices[s].set_commodities[k]["volume_of_data"]*instance.set_VNFs[instance.number_of_AN_based_NFs].compression

            for f in 1:(instance.number_of_AN_based_NFs+instance.number_of_CN_based_NFs), g in 1:(instance.number_of_AN_based_NFs +instance.number_of_CN_based_NFs)
                if f!=g 
                    if f<=instance.number_of_AN_based_NFs&&g<=instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g] >0.9
                    charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*instance.setSlices[s].set_commodities[k]["volume_of_data"]*instance.set_VNFs[f].compression
                    end
                    if f>instance.number_of_AN_based_NFs && g>instance.number_of_AN_based_NFs && sol_gamma[s,1,edge_id,f,g]>=0.9
                        charge_arc[edge_id] += sol_gamma[s,1,edge_id,f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities))
                    elseif f>instance.number_of_AN_based_NFs && g<=instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g]>=0.9
                     charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities))
                    elseif f<=instance.number_of_AN_based_NFs && g>instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g]>=0.9
                        charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities))

                    end
                end
            end
        end


        total_arc_load+= charge_arc[edge_id]
         charge_arc[edge_id] =  charge_arc[edge_id]/get_prop(instance.physical_network,a,:max_bandwidth) 
        if charge_arc[edge_id] > 0
            number_active_links += 1
            ppa+=charge_arc[edge_id]
            if charge_arc[edge_id]  > most_loaded_arc
                most_loaded_arc = charge_arc[edge_id] 
            end
        end
    end
    total_arc_load = total_arc_load/total_arc_capacity
    avr_lat = 0.0            
    lat = zeros(Float64, 32, 8)            
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities)
          
      lat[s,k] += sum(get_prop(instance.physical_network,a,:delay) *sol_gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:instance.number_of_AN_based_NFs-1,a in edges(instance.physical_network))
      lat[s,k] += sum(get_prop(instance.physical_network,a,:delay)*sol_gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] for f in 1:instance.number_of_AN_based_NFs,a in edges(instance.physical_network))
      lat[s,k] += sum(get_prop(instance.physical_network,a,:delay)*sol_gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] for f in 1:instance.number_of_AN_based_NFs,a in edges(instance.physical_network))
    
      end
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities)
      avr_lat += lat[s,k]/32          
    end          

#-------------------- PRINTING -----------------------------------      
        
    open(joinpath(result_folder,"load_on_arcs.csv"), "a") do iooo
        open(joinpath(result_folder,"final_statistics.csv"), "a") do io
            open(joinpath(result_folder,"load_on_nodes.csv"), "a") do ioo              

          
                 write(io,"$(test);MILP; $(my_parameters.class);$(my_parameters.graph_density);$(my_parameters.link_capacity_ratio);$(my_parameters.prob_sharing_NFS);")
                


                write(io,"$(Objective_value);")   
                write(io,"$(number_NFSs);")
                write(io,"$(dis_NFS);")
                write(io,"$(cent_NFS);")


                write(io,"$(props(instance.physical_network)[:number_nodes]);")
                write(io,"$(number_active_node);")
                write(io,"$(number_active_node/props(instance.physical_network)[:number_nodes]);")

                write(io,"$(total_nodes_capacity);")
                write(io,"$(total_load);")
                write(io,"$(ppn/number_active_node);")
                write(io,"$(most_loaded);")


                write(io,"$(props(instance.physical_network)[:number_of_arcs]);")
                write(io,"$(number_active_links);")
                write(io,"$(number_active_links/props(instance.physical_network)[:number_of_arcs]);")

                write(io,"$(total_arc_capacity);")
                write(io,"$(total_arc_load);")
                write(io,"$(ppa/number_active_links);")
                write(io,"$(most_loaded_arc);")
                write(io,"$(avr_lat)\n")

                for u in 1:props(instance.physical_network)[:number_nodes]
                  node_type = props(instance.physical_network, u)[:node_type] 
                        write(ioo,"$(test);MILP;$(u);$(node_type);$(my_parameters.class);$(my_parameters.graph_density);$(my_parameters.link_capacity_ratio);$(charge_node[u]);$(my_parameters.prob_sharing_NFS)\n")
                    
                end

                for a in edges(instance.physical_network) 
                    edge_id = get_prop(instance.physical_network,a,:edge_id)
                    edge_type = ""           
                    if props(instance.physical_network, src(a))[:node_type] == "access" ||  props(instance.physical_network, dst(a))[:node_type]  == "access"         
                        edge_type = "fronthaul"
                    else        
                        edge_type = "backhaul"
                    end               
                        write(iooo,"$(test);MILP;$(edge_id);$(edge_type);$(my_parameters.class);$(my_parameters.graph_density);$(my_parameters.link_capacity_ratio);$(charge_arc[edge_id]);$(my_parameters.prob_sharing_NFS)\n")
                end
            end 
        end   
    end

    
                                                    
        
end

function warm_up(mmy_instance::Instance,file_warm::String)
    
    mmy_parameters = Parameters()
    mmy_parameters.instance_name = file_warm
    mmy_parameters.valid_inequalities = "none"
    mmy_parameters.cut_class = "none"
    mmy_parameters.boost = true
    mmy_parameters.solver = "CPLEX"
    mmy_parameters.instance_type = "type_$(1)"
    mmy_parameters.instance_number = 1

    mmy_parameters.number_of_phy_nodes = 10
    mmy_parameters.graph_density = 0.1
    mmy_parameters.link_capacity_ratio = "4.0"
    mmy_parameters.node_capacity_ratio =""
    mmy_parameters.number_of_slice_requests =1
    mmy_parameters.number_of_commodities_per_NS = 1
    mmy_parameters.number_of_CP_NFSs = 1 
    mmy_parameters.number_of_DP_NFSs = 1
    mmy_parameters.prob_sharing_NFS = 1.0
    mmy_parameters.prob_sharing_node = 1.0
    mmy_parameters.warming_up = true
    mmy_parameters.objf = "minLINKload"
    mmy_parameters.formulation = "complete"


    #creating model
    mmodel = create_NSDP_complete_model(mmy_instance,mmy_parameters)
    mmy_parameters.relaxation=false
    # calling solver for LP
    solve_NSDP_model(mmodel,mmy_parameters,mmy_instance,file_warm)
    
    wp = ""
    open("$(file_warm).txt") do file
        #we read line by line
        for ln in eachline(file)
            aux_string = split(ln)
            #getting number of slice_requests 
            if length(aux_string[1])>0 && aux_string[1] == "wp" 
               wp = aux_string[2]
            end
        end
    end
    mmodel = nothing
    return wp
end

function export_stat_instance_reduced(parameters::Parameters,test::Int64)    
   
    open(joinpath(result_folder,"quantitative_analyses_test$(test).csv"), "a") do io

    
        
        write(io,"$(parameters.number_of_lazy_cuts);")
        write(io,"$(parameters.cut_time);")
        write(io,"$(parameters.number_of_cuts);")
        
        write(io,"$((parameters.MILP_solution - parameters.LP_solution)/parameters.MILP_solution);")

        
       

        write(io,"$(parameters.total_MILP_time);")
        write(io,"$((parameters.first_upper_bound - parameters.LP_solution)/parameters.first_upper_bound);")
        write(io,"$(parameters.root_node_processing_time);")
        write(io,"$((parameters.final_upper_bound - parameters.root_gap)/parameters.final_upper_bound);")
        write(io,"$((parameters.final_upper_bound - parameters.start_branching_upper_bound)/parameters.final_upper_bound);")
        write(io,"$(parameters.branch_and_bound_time-parameters.root_node_processing_time);")
        write(io,"$(parameters.tree_size);")
        write(io,"$(parameters.number_of_int_solutions);")
        write(io,"$(parameters.node_last_int_sol);")
        write(io,"$((parameters.final_upper_bound - parameters.final_lower_bound)/parameters.final_upper_bound);")
        write(io,"$(parameters.final_upper_bound); $(parameters.final_lower_bound);")
        write(io,"$(parameters.final_status)\n")

    end
end

function export_stat_instance(parameters::Parameters,test::Int64,model::String)    
   
    open(joinpath(result_folder,"quantitative_analyses_test$(test).csv"), "a") do io

        write(io,"instance_$(parameters.instance_number);")
        write(io,"$(parameters.instance_type);")
        write(io,"$(parameters.objf);")
        write(io,"$(model);")
        write(io,"$(parameters.class);")
        

        
        write(io,"$(parameters.number_of_phy_nodes);") 
        write(io,"$(parameters.graph_density);") 
        write(io,"$(parameters.link_capacity_ratio);") 
        write(io,"$(parameters.node_capacity_ratio);") 
        write(io,"$(parameters.number_of_slice_requests);")
        write(io,"$(parameters.number_of_commodities_per_NS);")
        write(io,"$(parameters.number_of_CP_NFSs);")
        write(io,"$(parameters.number_of_DP_NFSs);")
        write(io,"$(parameters.prob_sharing_NFS);")
        write(io,"$(parameters.prob_sharing_node);")

        
        write(io,"$(parameters.solver);")
        write(io,"$(parameters.boost);")
        write(io,"$(parameters.valid_inequalities);")
        write(io,"$(parameters.vi_creation_time);")
        write(io,"$(parameters.number_constraints_before_VI);")
        write(io,"$(parameters.number_constraints_after_VI);")
        write(io,"$((parameters.number_constraints_after_VI-parameters.number_constraints_before_VI)/parameters.number_constraints_after_VI);")
        write(io,"$(parameters.number_of_variables_gamma+parameters.number_of_variables_x+parameters.number_of_variables_z+parameters.number_of_variables_y+parameters.number_of_variables_w);")
        
        write(io,"$(parameters.number_of_lazy_cuts);")
          write(io,"$(parameters.lazy_time);")
        write(io,"$(parameters.cut_class);")
        write(io,"$(parameters.cut_time);")
        write(io,"$(parameters.number_of_cuts);")
        
        write(io,"$(parameters.total_LP_time);")
        write(io,"$((parameters.MILP_solution - parameters.LP_solution)/parameters.MILP_solution);")
        write(io,"$(parameters.simplex_iterations);")

        
        write(io,"$(parameters.number_of_variables_gamma);")
        write(io,"$(parameters.number_of_base_relax_variables_gamma/parameters.number_of_variables_gamma);")
        write(io,"$(parameters.number_of_frac_variables_gamma/parameters.number_of_base_relax_variables_gamma);")
        write(io,"$(parameters.number_of_base_final_variables_gamma/parameters.number_of_variables_gamma);")

        write(io,"$(parameters.number_of_variables_x);")
        write(io,"$(parameters.number_of_base_relax_variables_x/parameters.number_of_variables_x);")
        write(io,"$(parameters.number_of_frac_variables_x/parameters.number_of_base_relax_variables_x);")
        write(io,"$(parameters.number_of_base_final_variables_x/parameters.number_of_variables_x);")
        
        write(io,"$(parameters.number_of_variables_y);")
        write(io,"$(parameters.number_of_base_relax_variables_y/parameters.number_of_variables_y);")
        write(io,"$(parameters.number_of_frac_variables_y/parameters.number_of_base_relax_variables_y);")
        write(io,"$(parameters.number_of_base_final_variables_y/parameters.number_of_variables_y);")
        
        write(io,"$(parameters.number_of_variables_z);")
        write(io,"$(parameters.number_of_base_relax_variables_z/parameters.number_of_variables_z);")
        write(io,"$(parameters.number_of_frac_variables_z/parameters.number_of_base_relax_variables_z);")
        write(io,"$(parameters.number_of_base_final_variables_z/parameters.number_of_variables_z);")
        
        write(io,"$(parameters.number_of_variables_w);")
        write(io,"$(parameters.number_of_base_relax_variables_w/parameters.number_of_variables_w);")
        write(io,"$(parameters.number_of_base_final_variables_w/parameters.number_of_variables_w);")
        

        write(io,"$(parameters.total_MILP_time);")
        write(io,"$((parameters.first_upper_bound - parameters.LP_solution)/parameters.first_upper_bound);")
        write(io,"$(parameters.root_node_processing_time);")
        write(io,"$((parameters.final_upper_bound - parameters.root_gap)/parameters.final_upper_bound);")
        write(io,"$((parameters.final_upper_bound - parameters.start_branching_upper_bound)/parameters.final_upper_bound);")
        write(io,"$(parameters.branch_and_bound_time-parameters.root_node_processing_time);")
        write(io,"$(parameters.tree_size);")
        write(io,"$(parameters.number_of_int_solutions);")
        write(io,"$(parameters.node_last_int_sol);")
        write(io,"$((parameters.final_upper_bound - parameters.final_lower_bound)/parameters.final_upper_bound);")
        write(io,"$(parameters.final_upper_bound); $(parameters.final_lower_bound);")
        write(io,"$(parameters.final_status)\n")

    end
end

function export_stat_instance_heu(parameters::Parameters,test::Int64,solution_heuristic::Instance,bv::Float16)    
   
    open(joinpath(result_folder,"quantitative_analyses_test_HEU$(test).csv"), "a") do io

        write(io,"instance_$(parameters.instance_number);")
        write(io,"$(parameters.instance_type);")
        write(io,"$(parameters.objf);")
        write(io,"heuristic;")
        write(io,"$(parameters.class);")
        

        
        write(io,"$(parameters.number_of_phy_nodes);") 
        write(io,"$(parameters.graph_density);") 
        write(io,"$(parameters.link_capacity_ratio);") 
        write(io,"$(parameters.node_capacity_ratio);") 
        write(io,"$(parameters.number_of_slice_requests);")
        write(io,"$(parameters.number_of_commodities_per_NS);")
        write(io,"$(parameters.number_of_CP_NFSs);")
        write(io,"$(parameters.number_of_DP_NFSs);")
        write(io,"$(parameters.prob_sharing_NFS);")
        write(io,"$(parameters.prob_sharing_node);")

        write(io,"$(solution_heuristic.solution.time_found);")
        write(io,"$(solution_heuristic.solution.number_solution);")
        write(io,"$(solution_heuristic.solution.time_found/solution_heuristic.solution.number_solution);")
        write(io,"$(solution_heuristic.best_solution.solution_cost);")
        write(io,"$(solution_heuristic.best_solution.number_solution);")
        write(io,"$(solution_heuristic.best_solution.time_found);")
        write(io,"$(solution_heuristic.solution.av_solution_cost);")
        write(io,"$(max(0.0,(solution_heuristic.best_solution.solution_cost - bv)/bv));")
        write(io,"$(max(0.0,(solution_heuristic.solution.av_solution_cost - bv)/bv))\n")
        
        
     
       

    end
end

function create_NSDP_reduced_model(instance::Instance,parameters::Parameters)
    
    my_model = 1
    #my_model = Model(CPLEX.Optimizer
            
    if parameters.solver=="CPLEX"
        my_model =Model(CPLEX.Optimizer)
        set_optimizer_attributes(my_model, "CPX_PARAM_TILIM" => 60*30*2)
       if parameters.warming_up == true
            set_optimizer_attributes(my_model,  "CPX_PARAM_THREADS"=>15)
            set_optimizer_attributes(my_model,  "CPX_PARAM_EPGAP"=>1.0)
         else 
            set_optimizer_attributes(my_model,  "CPX_PARAM_REDUCE"=>0)
            set_optimizer_attributes(my_model,  "CPX_PARAM_PRELINEAR"=>0)
            set_optimizer_attributes(my_model,  "CPX_PARAM_THREADS"=>1)

         end 
            
        set_optimizer_attributes(my_model,  "CPX_PARAM_MIPDISPLAY"=>4)  
        set_optimizer_attributes(my_model, "CPX_PARAM_MIPINTERVAL" => 100000)
        set_optimizer_attributes(my_model,  "CPX_PARAM_WORKMEM"=>1024*10)
        set_optimizer_attributes(my_model,  "CPX_PARAM_NODEFILEIND"=>2)
        
        if parameters.boost == false 
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_BQP"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Cliques"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Covers"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Disjunctive"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_FlowCovers"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_PathCut"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Gomory"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_GUBCovers"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Implied"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_LocalImplied"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_LiftProj"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_MIRCut"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_RLT"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_ZeroHalfCut"=>-1)
        end
    end 
        



    #-------------------------Variables--------------------
    @variable(my_model, gamma[s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities), a in 1:props(instance.physical_network)[:number_of_arcs],f in 1:length(instance.set_VNFs)+1, g in 1:length(instance.set_VNFs)+1], Bin)
    @variable(my_model, z[s in 1:length(instance.setSlices),f in 1:instance.number_of_AN_based_NFs],Bin)
    @variable(my_model, x[s in 1:length(instance.setSlices),f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]], Bin)
    @variable(my_model, w[s in 1:length(instance.setSlices),f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]] >= 0)
    @variable(my_model, y[f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]] >=0, Int)
    if parameters.objf== "minLINKload"                                
        @variable(my_model, 0 <= U <= 1)
    end

      test = 0
     #---------------------------------------------
    # SPLIT SELECTION
    #---------------------------------------------
    for s in 1:length(instance.setSlices),  f in 1:(instance.number_of_AN_based_NFs-1)
        @constraint(my_model, z[s,f]<=z[s,f+1])
    end
    
   
    #---------------------------------------------
    # DIMENSIONING - CP NFSs
    #---------------------------------------------
    for s in 1:length(instance.setSlices),  f in (instance.number_of_AN_based_NFs+1):(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),u in 1: props(instance.physical_network)[:number_nodes],n in 1:instance.number_of_NFs
        if instance.setSlices[s].set_VNFs_to_install[f] > 0.5
            @constraint(my_model, w[s,f,n,u]  == instance.set_VNFs[f].dataVolPerUE*instance.setSlices[s].TotalAmountUE*x[s,f,n,u] /instance.set_VNFs[f].treatment_capacity)
        else
            @constraint(my_model, w[s,f,n,u]  == 0)
        end
    end 
    #---------------------------------------------
    # DIMENSIONING - DISTRIBUTED DP NFSs
    #---------------------------------------------
     for s in 1:length(instance.setSlices),  k in instance.setSlices[s].set_commodities, f in 1:instance.number_of_AN_based_NFs,n in 1:instance.number_of_NFs
         if f!=1
            @constraint(my_model, w[s,f,n,k["origin_node"]]  == instance.set_VNFs[f-1].compression*k["volume_of_data"]*x[s,f,n,k["origin_node"]]/instance.set_VNFs[f].treatment_capacity)
    
        else
            @constraint(my_model, w[s,f,n,k["origin_node"]] == k["volume_of_data"]*x[s,f,n,k["origin_node"]]/instance.set_VNFs[f].treatment_capacity)
        end
    end
    #---------------------------------------------
    # DIMENSIONING - CENTRALIZED DP NFSs
    #---------------------------------------------
     for s in 1:length(instance.setSlices),  f in 1:instance.number_of_AN_based_NFs, u in 1: props(instance.physical_network)[:number_nodes],n in 1:instance.number_of_NFs
        if props(instance.physical_network, u)[:node_type] == "non_access" && f!=1
            @constraint(my_model, w[s,f,n,u] == instance.setSlices[s].set_VNFs_to_install[f]*(sum(instance.set_VNFs[f-1].compression*k["volume_of_data"] for k in instance.setSlices[s].set_commodities)*x[s,f,n,u])/instance.set_VNFs[f].treatment_capacity)                
        elseif props(instance.physical_network, u)[:node_type] == "non_access" && f==1
            @constraint(my_model, w[s,f,n,u] ==  instance.setSlices[s].set_VNFs_to_install[f]*(sum(k["volume_of_data"] for k in instance.setSlices[s].set_commodities)*x[s,f,n,u])/instance.set_VNFs[f].treatment_capacity)                
         elseif props(instance.physical_network, u)[:node_type] == "app"
             @constraint(my_model, w[s,f,n,u] ==0)
        end
    end
                                           
    #---------------------------------------------
    # PLACEMENT - DISTRIBUTED DP NFSs
    #---------------------------------------------
    for s in 1:length(instance.setSlices) 
        O = Vector()   
        for k in instance.setSlices[s].set_commodities
             push!(O,k["origin_node"])                            
        end
        for f in 1:instance.number_of_AN_based_NFs,  u in 1: props(instance.physical_network)[:number_nodes]
            if u ∈ O 
                @constraint(my_model, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs)== instance.setSlices[s].set_VNFs_to_install[f]*(1 - z[s,f]))
            elseif props(instance.physical_network, u)[:node_type] != "non_access"
                @constraint(my_model, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs)== 0)
            end
        end
    end
            
  
    #---------------------------------------------
    # PLACEMENT - CENTRALIZED DP NFSs      --------------
    #---------------------------------------------
     for s in 1:length(instance.setSlices),  f in 1:instance.number_of_AN_based_NFs
        expression = zero(AffExpr)
        for u in 1: props(instance.physical_network)[:number_nodes]
            if props(instance.physical_network, u)[:node_type] == "non_access"
                add_to_expression!(expression, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs))                           
            end

        end
        @constraint(my_model, expression == instance.setSlices[s].set_VNFs_to_install[f]*z[s,f])  
                               
    end
                
    #---------------------------------------------
    # PLACEMENT - CP NFSs
    #---------------------------------------------
     for s in 1:length(instance.setSlices),  f in (1+instance.number_of_AN_based_NFs):(instance.number_of_AN_based_NFs + instance.number_of_CN_based_NFs)
        expression = zero(AffExpr)
         expression2 = zero(AffExpr)                            
        for u in 1: props(instance.physical_network)[:number_nodes]
            if props(instance.physical_network, u)[:node_type] == "non_access"
                add_to_expression!(expression, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs))
            else
                add_to_expression!(expression, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs))
            end
        end       
        @constraint(my_model, expression == instance.setSlices[s].set_VNFs_to_install[f]) 
        @constraint(my_model, expression2 ==0) 

     end


    
    
    #---------------------------------------------
    # PACKING
    #--------------------------------------------
    for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]
        @constraint(my_model, sum(w[s,f,m,u] for s in 1:length(instance.setSlices))  <= y[f,m,u])
        if parameters.objf== "minLINKload"                                
            @constraint(my_model, sum(w[s,f,m,u] for s in 1:length(instance.setSlices)) +0.99999999 >= y[f,m,u])
        end
    end
            
    #---------------------------------------------
    # NODE CAPACITY
    #---------------------------------------------
    for u in 1: props(instance.physical_network)[:number_nodes]
        @constraint(my_model,  sum(instance.set_VNFs[f].ram_request*y[f,m,u] for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs) <= props(instance.physical_network,u)[:ram_capacity])
        @constraint(my_model,  sum(instance.set_VNFs[f].storage_request*y[f,m,u] for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs) <= props(instance.physical_network,u)[:storage_capacity])
        @constraint(my_model,  sum(instance.set_VNFs[f].cpu_request*y[f,m,u] for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs) <= props(instance.physical_network,u)[:cpu_capacity])
     end  
    
    #---------------------------------------------
    # FLOW - DP
    #---------------------------------------------
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities), f in 1:(instance.number_of_AN_based_NFs-1),u in 1: props(instance.physical_network)[:number_nodes]
            
        ingoing = zero(AffExpr)
        outgoing = zero(AffExpr)
        ingoing_target= zero(AffExpr)
        outgoing_target = zero(AffExpr)
        ingoing_origin = zero(AffExpr)
        outgoing_origin = zero(AffExpr)
        conttt=0
        for a in edges(instance.physical_network)
            if src(a) == u
               add_to_expression!(outgoing_target, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1])
               add_to_expression!(outgoing_origin, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1])
               add_to_expression!(outgoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1])
            elseif dst(a) == u
                add_to_expression!(ingoing_target, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1])
                add_to_expression!(ingoing_origin, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] )
                add_to_expression!(ingoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1])
            end
        end
        if props(instance.physical_network, u)[:node_type] != "access"
            @constraint(my_model, outgoing - ingoing == sum(x[s,f,m,u] for m in 1:instance.number_of_NFs) -sum(x[s,f+1,m,u] for m in 1:instance.number_of_NFs))
        elseif u!=instance.setSlices[s].set_commodities[k]["origin_node"]
            @constraint(my_model, outgoing - ingoing == 0)
        else               
             @constraint(my_model, outgoing  == z[s,f+1] - z[s,f])
        end  
        if  u==instance.setSlices[s].set_commodities[k]["target_node"]
            @constraint(my_model, outgoing_target - ingoing_target == -1)
            @constraint(my_model, outgoing_origin - ingoing_origin == 0)        
        
        elseif  u==instance.setSlices[s].set_commodities[k]["origin_node"]
            @constraint(my_model, outgoing_target - ingoing_target == 1-z[s,instance.number_of_AN_based_NFs])
            @constraint(my_model, outgoing_origin - ingoing_origin == z[s,1])

        
        elseif  props(instance.physical_network, u)[:node_type] == "non_access" 
            @constraint(my_model, outgoing_target - ingoing_target == sum(x[s,instance.number_of_AN_based_NFs,m,u] for m in 1:instance.number_of_NFs))
            @constraint(my_model, outgoing_origin - ingoing_origin == -sum(x[s,1,m,u] for m in 1:instance.number_of_NFs))
        
        else 
            @constraint(my_model, outgoing_target - ingoing_target == 0)
            @constraint(my_model, outgoing_origin - ingoing_origin == 0)
        end
    end
                
    #---------------------------------------------
    # FLOW - BETWEEN CP NFSs
    #---------------------------------------------
    for s in 1:length(instance.setSlices),  f in (instance.number_of_AN_based_NFs+1):(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),g in (instance.number_of_AN_based_NFs+1):(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),u in 1: props(instance.physical_network)[:number_nodes]
        if f!=g && instance.VNF_connection["$(s)"][f][g] == true && instance.setSlices[s].set_VNFs_to_install[g] == true && instance.setSlices[s].set_VNFs_to_install[f] == true
            ingoing = zero(AffExpr)
            outgoing = zero(AffExpr)
            for a in edges(instance.physical_network)
              if src(a) == u
                   add_to_expression!(outgoing, gamma[s,1,get_prop(instance.physical_network,a,:edge_id),f,g])         
                end
                if dst(a) == u
                   add_to_expression!(ingoing,gamma[s,1,get_prop(instance.physical_network,a,:edge_id),f,g])          
                end
            end
            @constraint(my_model, outgoing - ingoing == sum(x[s,f,m,u] for m in 1:instance.number_of_NFs) - sum(x[s,g,m,u] for m in 1:instance.number_of_NFs))
        end
    end
                
    #---------------------------------------------
    # FLOW - BETWEEN DP AND CP NFSs
    #---------------------------------------------               
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities), f in (instance.number_of_AN_based_NFs+1):(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),g in 1:instance.number_of_AN_based_NFs,u in 1: props(instance.physical_network)[:number_nodes]
        if instance.VNF_connection["$(s)"][f][g] == true && instance.setSlices[s].set_VNFs_to_install[g] == true && instance.setSlices[s].set_VNFs_to_install[f] == true
            ingoing = zero(AffExpr)
            outgoing = zero(AffExpr)
            for a in edges(instance.physical_network)
              if src(a) == u
                   add_to_expression!(outgoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])          
                end
                if dst(a) == u
                   add_to_expression!(ingoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])         
                end
            end
            if props(instance.physical_network, u)[:node_type] == "non_access"           
                @constraint(my_model, outgoing - ingoing == sum(x[s,f,m,u] for m in 1:instance.number_of_NFs) - sum(x[s,g,m,u] for m in 1:instance.number_of_NFs))

            elseif props(instance.physical_network, u)[:node_type] == "access" && props(instance.physical_network, u)[:node_id] ==  instance.setSlices[s].set_commodities[k]["origin_node"]           
                @constraint(my_model, outgoing - ingoing == z[s,g]-1)
            else
               @constraint(my_model, outgoing - ingoing == 0)
            end
        end
    end        

                    
  for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities),f in 1:instance.number_of_AN_based_NFs,g in (instance.number_of_AN_based_NFs+1):instance.number_of_AN_based_NFs+instance.number_of_CN_based_NFs,u in 1: props(instance.physical_network)[:number_nodes]
        if instance.VNF_connection["$(s)"][f][g] == true && instance.setSlices[s].set_VNFs_to_install[g] == true && instance.setSlices[s].set_VNFs_to_install[f] == true
            ingoing = zero(AffExpr)
            outgoing = zero(AffExpr)
            for a in edges(instance.physical_network)
              if src(a) == u
                   add_to_expression!(outgoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])         
                end
                if dst(a) == u
                   add_to_expression!(ingoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])         
                end
            end
            if props(instance.physical_network, u)[:node_type] == "non_access"           
                @constraint(my_model, outgoing - ingoing == sum(x[s,f,m,u] for m in 1:instance.number_of_NFs) - sum(x[s,g,m,u] for m in 1:instance.number_of_NFs))
            
            elseif props(instance.physical_network, u)[:node_type] == "access" && props(instance.physical_network, u)[:node_id] ==  instance.setSlices[s].set_commodities[k]["origin_node"]           
                 @constraint(my_model, outgoing - ingoing == 1 - z[s,f])
            else
                 @constraint(my_model, outgoing - ingoing == 0)            
            end
        end
    end       
    #---------------------------------------------
    # LINK CAPACITY
                                                
    #---------------------------------------------
    for a in edges(instance.physical_network)           
        sum_gamma = zero(AffExpr)            
        for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities)
            add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]*instance.setSlices[s].set_commodities[k]["volume_of_data"])
            add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1]*instance.setSlices[s].set_commodities[k]["volume_of_data"]*instance.set_VNFs[instance.number_of_AN_based_NFs].compression)
            for f in 1:(instance.number_of_AN_based_NFs+instance.number_of_CN_based_NFs), g in 1:(instance.number_of_AN_based_NFs+instance.number_of_CN_based_NFs)
                if f!=g 
                    if f <= instance.number_of_AN_based_NFs && g <= instance.number_of_AN_based_NFs && f==g-1                      
                        add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]*instance.setSlices[s].set_commodities[k]["volume_of_data"]*instance.set_VNFs[f].compression)
                    end                            
                    if f>instance.number_of_AN_based_NFs && g>instance.number_of_AN_based_NFs
                        add_to_expression!(sum_gamma,gamma[s,1,get_prop(instance.physical_network,a,:edge_id),f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities)))
                    elseif f>instance.number_of_AN_based_NFs && g<=instance.number_of_AN_based_NFs
                        add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities)))
                    elseif f<=instance.number_of_AN_based_NFs && g>instance.number_of_AN_based_NFs
                        add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities)))

                    end
                end

            end
        end
        if parameters.objf== "minNFS"                                
            @constraint(my_model, sum_gamma <= get_prop(instance.physical_network,a,:max_bandwidth))
        else                               
            @constraint(my_model, sum_gamma <= U*get_prop(instance.physical_network,a,:max_bandwidth))                      
        end
    end                                                        
    parameters.number_constraints_before_VI = sum(num_constraints(my_model, F, S) for (F,S) in list_of_constraint_types(my_model))
      
  parameters.vi_creation_time = @elapsed begin                                    
          #-------------------------- VALID INEQUALITIES-----------------------------------------

     if parameters.valid_inequalities == "LB" || parameters.valid_inequalities=="all" || parameters.valid_inequalities=="both"
     try
        dim_f = []
         for f in 1:length(instance.set_VNFs)
            dim = 0.0 
            for  s in 1:length(instance.setSlices)
                if f!=1 && instance.set_VNFs[f].typee != :cp 
                    dim += sum(instance.set_VNFs[f-1].compression*k["volume_of_data"]  for k in instance.setSlices[s].set_commodities)/instance.set_VNFs[f].treatment_capacity                
                elseif f== 1  && instance.set_VNFs[f].typee != :cp 
                    dim += sum(k["volume_of_data"] for  k in instance.setSlices[s].set_commodities)/instance.set_VNFs[f].treatment_capacity                
                else
                    dim += instance.setSlices[s].set_VNFs_to_install[f]*instance.set_VNFs[f].dataVolPerUE*instance.setSlices[s].TotalAmountUE/instance.set_VNFs[f].treatment_capacity
                end
            end
            push!(dim_f,ceil(dim))    
        end
                         

         LB =  sum(dim_f[i] for i in 1:length(dim_f))
         my_Ys = zero(AffExpr)
        add_to_expression!(my_Ys, sum(y[f,m,u] for f in 1:length(instance.set_VNFs),  m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]))
         @constraint(my_model, my_Ys >= LB)                                                                                   
     

    catch
        b=1
    end
end 
                                                                                    
        if parameters.valid_inequalities == "SymmetryBreaking" || parameters.valid_inequalities=="all" || parameters.valid_inequalities=="both"
            #"SymmetryBreaking1" Inequalities
            try
                for n in 1:(instance.number_of_NFs - 1)
                    @constraint(my_model, sum(x[s,f,n+1,u] for s in 1:length(instance.setSlices),f in 1:length(instance.set_VNFs), u in 1:props(instance.physical_network)[:number_nodes]) <=instance.number_of_NFs*sum(x[t,g,n,v] for t in 1:length(instance.setSlices),g in 1:length(instance.set_VNFs), v in 1: props(instance.physical_network)[:number_nodes]))  
                end 
            catch
                b=1
            end
        end

                                                                            
        if parameters.valid_inequalities == "ShortestPath"|| parameters.valid_inequalities=="all"
            for a in edges(instance.physical_network)
                set_prop!(instance.physical_network, src(a) , dst(a) , :weight, get_prop(instance.physical_network,a,:delay))
            end
            try
                #"ShortestPath"-based Inequalities
                for s in 1:length(instance.setSlices),k in 1:length(instance.setSlices[s].set_commodities)
                    my_path = zero(AffExpr)
                    add_to_expression!(my_path,sum(get_prop(instance.physical_network,a,:delay)*(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] 
                            + gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]) for a in edges(instance.physical_network))) 
                    +sum(sum(get_prop(instance.physical_network,a,:delay)*gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for a in edges(instance.physical_network),f in 1:(instance.number_of_AN_based_NFs-1))) 

                     shortest_path_of_K = enumerate_paths(dijkstra_shortest_paths(instance.physical_network, instance.setSlices[s].set_commodities[k]["origin_node"]), instance.setSlices[s].set_commodities[k]["target_node"])    
                     shortest_latency = 0.0
                    for a in 1:length(shortest_path_of_K)-1
                         shortest_latency+= get_prop(instance.physical_network,shortest_path_of_K[a],shortest_path_of_K[a+1],:delay)
                    end
                    @constraint(my_model, my_path >= shortest_latency)
                end 
                                                                                
         catch
            b=1
        end
        end

        if parameters.valid_inequalities == "MinCut"|| parameters.valid_inequalities=="all"

           #"MinCut" inequalities
            numberNodes = props(instance.physical_network)[:number_nodes]
            flow_graph = DiGraph(numberNodes) 
            capacity_matrix = zeros(Float64, numberNodes, numberNodes) 
            for a in edges(instance.physical_network)
                add_edge!(flow_graph, src(a), dst(a))
                capacity_matrix[src(a),dst(a)] = get_prop(instance.physical_network,a,:max_bandwidth)
            end
            for s in 1:length(instance.setSlices),  k in length(instance.setSlices[s].set_commodities)

                f,F = LightGraphsFlows.maximum_flow(flow_graph, instance.setSlices[s].set_commodities[k]["origin_node"], instance.setSlices[s].set_commodities[k]["target_node"], capacity_matrix,LightGraphsFlows.EdmondsKarpAlgorithm()) # Run default maximum_flow with the capacity_matrix
                f2,F2 = LightGraphsFlows.maximum_flow(flow_graph, instance.setSlices[s].set_commodities[k]["origin_node"], instance.setSlices[s].set_commodities[k]["target_node"], capacity_matrix,LightGraphsFlows.DinicAlgorithm()) # Run default maximum_flow with the capacity_matrix
                f3,F3 = LightGraphsFlows.maximum_flow(flow_graph, instance.setSlices[s].set_commodities[k]["origin_node"], instance.setSlices[s].set_commodities[k]["target_node"], capacity_matrix,LightGraphsFlows.BoykovKolmogorovAlgorithm()) # Run default maximum_flow with the capacity_matrix

                     expression1 = zero(AffExpr)
                     expression2 = zero(AffExpr) 
                     expression3 = zero(AffExpr) 

                     expression1b = zero(AffExpr)
                     expression2b = zero(AffExpr) 
                     expression3b = zero(AffExpr)
                    for a in edges(instance.physical_network)
                        if F[src(a),dst(a)]!=0.0
                           add_to_expression!(expression1, sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] )
                           add_to_expression!( expression1b, get_prop(instance.physical_network,a,:max_bandwidth)*(sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]) )

                        end
                        if F2[src(a),dst(a)]!=0.0
                            add_to_expression!(expression2, sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]) 
                            add_to_expression!(expression2b, get_prop(instance.physical_network,a,:max_bandwidth)*(sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] ))

                        end
                        if F3[src(a),dst(a)]!=0.0
                           add_to_expression!( expression3, sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] )
                             add_to_expression!(expression3b, get_prop(instance.physical_network,a,:max_bandwidth)*(sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]))

                        end
                     end 
                @constraint(my_model, expression1>=1)
                @constraint(my_model, expression2>=1)
                @constraint(my_model, expression3>=1)

                @constraint(my_model, expression1b>=instance.set_VNFs[instance.number_of_AN_based_NFs].compression*instance.setSlices[s].set_commodities[k]["volume_of_data"])
                @constraint(my_model, expression2b>=instance.set_VNFs[instance.number_of_AN_based_NFs].compression*instance.setSlices[s].set_commodities[k]["volume_of_data"])
                @constraint(my_model, expression3b>=instance.set_VNFs[instance.number_of_AN_based_NFs].compression*instance.setSlices[s].set_commodities[k]["volume_of_data"])

            end
        end
    end

    parameters.number_constraints_after_VI = sum(num_constraints(my_model, F, S) for (F,S) in list_of_constraint_types(my_model))
    
                                        
    function my_callback_cuts_reduced(cb_data)
        #test_solution = 0.0                                                                                            
          # if parameters.objf== "minNFS"
            #   test_solution= sum((props(instance.physical_network,u)[:ram_cost]*instance.set_VNFs[f].ram_request+props(instance.physical_network,u)[:storage_cost]*instance.set_VNFs[f].storage_request+props(instance.physical_network,u)[:cpu_cost]*instance.set_VNFs[f].cpu_request)*callback_value(cb_data,y[f,m,u]) for f in 1:length(instance.set_VNFs),  m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes])                    
           # else
           #      test_solution = callback_value(cb_data, U) 
           # end   
        #
        if rand() < 0.75   
        time = @elapsed begin
                if parameters.cut_class=="isolation_cuts" ||  parameters.cut_class=="all" 
                  for physical_node in 1:props(instance.physical_network)[:number_nodes]
                        if props(instance.physical_network, physical_node)[:node_type] == "non_access" 
                             aux_vector = Vector{Dict}()
                            for s in 1:length(instance.setSlices), f in 1:length(instance.set_VNFs), n in 1:instance.number_of_NFs
                                if callback_value(cb_data, x[s,f,n,physical_node]) >0.000001
                                    aux_dict = Dict()
                                    aux_dict["id"] = length(aux_vector)+1
                                    aux_dict["slice"] = s
                                    aux_dict["NF"] = n
                                    aux_dict["function"] = f
                                    push!(aux_vector, aux_dict)
                                end
                            end

                            GraphNODEisolation = SimpleGraph(length(aux_vector))
                            for u in 1:length(aux_vector)-1, v in u+1:length(aux_vector)
                                if  aux_vector[v]["slice"] != aux_vector[u]["slice"] && instance.setSlices[aux_vector[v]["slice"]].nodeSharing[aux_vector[u]["slice"]]*instance.setSlices[aux_vector[u]["slice"]].nodeSharing[aux_vector[v]["slice"]] ==0
                                    add_edge!(GraphNODEisolation, u, v)
                                end
                            end

                            my_cliques = maximal_cliques(GraphNODEisolation)


                           for cl in my_cliques                
                                if length(cl) >= 2
                                    expres = zero(AffExpr)
                                     test = 0.0                                                               
                                    for node in cl
                                        add_to_expression!(expres,x[aux_vector[node]["slice"],aux_vector[node]["function"],aux_vector[node]["NF"],physical_node])
                                         test+=callback_value(cb_data,x[aux_vector[node]["slice"],aux_vector[node]["function"],aux_vector[node]["NF"],physical_node])
                                    end
                                     if test >1                                                               
                                         con = @build_constraint(expres<=1)
                                         MOI.submit(my_model, MOI.UserCut(cb_data), con)
                                         parameters.number_of_cuts += 1
                                    end                                                                   
                                end
                            end
                        end
                    end
                   for n in 1:instance.number_of_NFs
                    aux_vector = Vector{Dict}()
                    for s in 1:length(instance.setSlices), f in 1:length(instance.set_VNFs), u in 1:props(instance.physical_network)[:number_nodes]
                       if  props(instance.physical_network, u)[:node_type] != "app"&& callback_value(cb_data, x[s,f,n,u]) >0.000001 
                            aux_dict = Dict()
                            aux_dict["id"] = length(aux_vector)+1
                            aux_dict["slice"] = s
                            aux_dict["node"] = u
                            aux_dict["function"] = f
                            push!(aux_vector, aux_dict)
                       end
                    end

                    GraphNFIsolation = SimpleGraph(length(aux_vector))
                    for u in 1:length(aux_vector)-1, v in u+1:length(aux_vector)
                        if aux_vector[v]["slice"] != aux_vector[u]["slice"]                                
                            if instance.setSlices[aux_vector[v]["slice"]].VNF_sharing["$(aux_vector[u]["slice"])"][aux_vector[u]["function"]][aux_vector[v]["function"]]*instance.setSlices[aux_vector[u]["slice"]].VNF_sharing["$(aux_vector[v]["slice"])"][aux_vector[v]["function"]][aux_vector[u]["function"]] ==0
                                add_edge!(GraphNFIsolation, u, v)
                            end
                        end
                    end

                    my_cliques = maximal_cliques(GraphNFIsolation)
                     for cl in my_cliques
                        if length(cl) >= 2
                            expres = zero(AffExpr)
                            test = 0.0                                                                    
                            for node in cl
                                add_to_expression!(expres,x[aux_vector[node]["slice"],aux_vector[node]["function"],n,aux_vector[node]["node"]])
                                test +=  callback_value(cb_data, x[aux_vector[node]["slice"],aux_vector[node]["function"],n,aux_vector[node]["node"]])                                                                  
                            end
                            if test > 1                                                                   
                                 con = @build_constraint(expres<=1)
                                 MOI.submit(my_model, MOI.UserCut(cb_data), con)
                                 parameters.number_of_cuts += 1
                            end 
                        end    
                     end
                end                                                         
            end                                                                   
            parameters.cut_class=="capacity_cuts" ||  parameters.cut_class=="all" 
           Threads.@threads for aa in 1:length(edges(instance.physical_network))
                    a = nothing
                    for b in edges(instance.physical_network)
                        if get_prop(instance.physical_network,b,:edge_id) ==  aa
                            a = b
                            break
                        end
                    end                   
                    aux_vector1 = Vector{Dict}()
                        for f in 1:length(instance.set_VNFs)+1                                                          
                        for s in 1:length(instance.setSlices),g in 1:length(instance.set_VNFs)+1,k in length(instance.setSlices[s].set_commodities)
                           if  callback_value(cb_data, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]) > 0.000001 
                                 aux_dict = Dict()
                                aux_dict["id"] = length(aux_vector1)+1
                                aux_dict["slice"] = s
                                aux_dict["functionF"] = f
                                 aux_dict["functionG"] = g
                                if f > instance.number_of_AN_based_NFs &&  f <= length(instance.set_VNFs)                            
                                    aux_dict["commodity"] = 1
                                elseif g > instance.number_of_AN_based_NFs &&  g <= length(instance.set_VNFs)                            
                                    aux_dict["commodity"] = 1  
                                else 
                                     aux_dict["commodity"] = k                                       
                                end
                                push!(aux_vector1, aux_dict)
                            end
                        end
                    end

                    GraphLinkCap = SimpleGraph(length(aux_vector1))
                  Threads.@threads for u in 1:length(aux_vector1)-1
                       for  v in u+1:length(aux_vector1)
                            requestU = 0.0
                            requestV = 0.0
                                if  aux_vector1[u]["functionF"] < instance.number_of_AN_based_NFs && aux_vector1[u]["functionG"] ==  aux_vector1[u]["functionF"] + 1 
                                    requestU = instance.setSlices[aux_vector1[u]["slice"]].set_commodities[aux_vector1[u]["commodity"]]["volume_of_data"]*instance.set_VNFs[aux_vector1[u]["functionF"]].compression

                                elseif aux_vector1[u]["functionF"] == length(instance.set_VNFs)+1 && aux_vector1[u]["functionG"] == 1 
                                     requestU = instance.setSlices[aux_vector1[u]["slice"]].set_commodities[aux_vector1[u]["commodity"]]["volume_of_data"]

                                elseif aux_vector1[u]["functionF"] == instance.number_of_AN_based_NFs && aux_vector1[u]["functionG"] == length(instance.set_VNFs)+1
                                    requestU = instance.setSlices[aux_vector1[u]["slice"]].set_commodities[aux_vector1[u]["commodity"]]["volume_of_data"]*instance.set_VNFs[aux_vector1[u]["functionF"]].compression
                                else
                                    requestU =  instance.set_VNFs[aux_vector1[u]["functionF"]].amount_of_traffic_sent_to_g[aux_vector1[u]["functionG"]]*instance.setSlices[aux_vector1[u]["slice"]].TotalAmountUE                                             
                                end

                                if  aux_vector1[v]["functionF"] < instance.number_of_AN_based_NFs && aux_vector1[v]["functionG"] ==  aux_vector1[v]["functionF"] + 1 
                                    requestU = instance.setSlices[aux_vector1[v]["slice"]].set_commodities[aux_vector1[v]["commodity"]]["volume_of_data"]*instance.set_VNFs[aux_vector1[v]["functionF"]].compression

                                elseif aux_vector1[v]["functionF"] == length(instance.set_VNFs)+1 && aux_vector1[v]["functionG"] == 1 
                                     requestU = instance.setSlices[aux_vector1[v]["slice"]].set_commodities[aux_vector1[v]["commodity"]]["volume_of_data"]

                                elseif aux_vector1[v]["functionF"] == instance.number_of_AN_based_NFs && aux_vector1[v]["functionG"] == length(instance.set_VNFs)+1
                                    requestU = instance.setSlices[aux_vector1[v]["slice"]].set_commodities[aux_vector1[v]["commodity"]]["volume_of_data"]*instance.set_VNFs[aux_vector1[v]["functionF"]].compression
                                else
                                    requestU =  instance.set_VNFs[aux_vector1[v]["functionF"]].amount_of_traffic_sent_to_g[aux_vector1[v]["functionG"]]*instance.setSlices[aux_vector1[v]["slice"]].TotalAmountUE                                             
                                end                     

                             if (requestV+requestU) > get_prop(instance.physical_network,a,:max_bandwidth)       
                                add_edge!(GraphLinkCap, u, v)
                            end                 
                        end
                    end

                    my_cliques = maximal_cliques(GraphLinkCap)
                    for cl in my_cliques
                        if length(cl) >= 2
                            cliqueEx= zero(AffExpr)
                            test = 0.0                                                                                                    
                            for node in cl
                                test+=callback_value(cb_data,gamma[aux_vector1[node]["slice"],aux_vector1[node]["commodity"],get_prop(instance.physical_network,a,:edge_id),aux_vector1[node]["functionF"],aux_vector1[node]["functionG"]])                                                                                                         
                                add_to_expression!(cliqueEx,gamma[aux_vector1[node]["slice"],aux_vector1[node]["commodity"],get_prop(instance.physical_network,a,:edge_id),aux_vector1[node]["functionF"],aux_vector1[node]["functionG"]])          
                            end
                             if test > 1                                                                                                   
                                 con = @build_constraint(cliqueEx<=1)
                                 MOI.submit(my_model, MOI.UserCut(cb_data), con)
                                 parameters.number_of_cuts += 1

                            end
                        end
                    end
                end
                resources = ["ram_capacity","storage_capacity","cpu_capacity"]

                Threads.@threads for physical_node in 1:props(instance.physical_network)[:number_nodes] 
                    aux_vector = Vector{Dict}()
                    if  props(instance.physical_network, physical_node)[:node_type] != "app"     
                        for s in 1:length(instance.setSlices), f in 1:length(instance.set_VNFs), n in 1:instance.number_of_NFs
                            if callback_value(cb_data, x[s,f,n,physical_node]) >0.00001
                                aux_dict = Dict()
                                aux_dict["id"] = length(aux_vector)+1
                                aux_dict["slice"] = s
                                aux_dict["node"] = physical_node
                                aux_dict["NFS"] = f
                                aux_dict["NF"] = n                                                                
                                push!(aux_vector, aux_dict)
                            end
                        end
                    end

                    GraphNodeCap = SimpleGraph(length(aux_vector))
                    for u in 1:length(aux_vector)-1, v in u+1:length(aux_vector), resource in resources
                        cap_u = callback_value(cb_data, w[aux_vector[u]["slice"],aux_vector[u]["NFS"],aux_vector[u]["NF"],physical_node]) / callback_value(cb_data, x[aux_vector[u]["slice"],aux_vector[u]["NFS"],aux_vector[u]["NF"],physical_node])                                                                    
                        cap_v = callback_value(cb_data, w[aux_vector[v]["slice"],aux_vector[v]["NFS"],aux_vector[v]["NF"],physical_node]) / callback_value(cb_data, x[aux_vector[v]["slice"],aux_vector[v]["NFS"],aux_vector[v]["NF"],physical_node])                                                                    
                        if resource == "ram_capacity"
                            test = cap_u*instance.set_VNFs[aux_vector[u]["NFS"]].ram_request + cap_v*instance.set_VNFs[aux_vector[v]["NFS"]].ram_request
                            if test > props(instance.physical_network,physical_node)[:ram_capacity]
                                add_edge!(GraphNodeCap, u, v)
                            end 
                        end

                        if resource == "storage_capacity"
                            test = cap_u*instance.set_VNFs[aux_vector[u]["NFS"]].storage_request + cap_v*instance.set_VNFs[aux_vector[v]["NFS"]].storage_request
                            if test > props(instance.physical_network,physical_node)[:storage_capacity]
                                add_edge!(GraphNodeCap, u, v)
                            end 
                        end

                        if resource == "cpu_capacity"
                            test = cap_u*instance.set_VNFs[aux_vector[u]["NFS"]].cpu_request + cap_v*instance.set_VNFs[aux_vector[v]["NFS"]].cpu_request
                            if test > props(instance.physical_network,physical_node)[:cpu_capacity]
                                add_edge!(GraphNodeCap, u, v)
                            end 
                        end
                    end

                    my_cliques = maximal_cliques(GraphNodeCap)
                     for cl in my_cliques
                        if length(cl) >= 2
                            expres = zero(AffExpr)
                            test = 0.0                                                                                                    
                            for node in cl
                                add_to_expression!(expres,x[aux_vector[node]["slice"],aux_vector[node]["NFS"],aux_vector[node]["NF"],aux_vector[node]["node"]])
                                test+=callback_value(cb_data, x[aux_vector[node]["slice"],aux_vector[node]["NFS"],aux_vector[node]["NF"],aux_vector[node]["node"]])                                                                                                   
                            end
                            if test > 1                                                                                                   
                                 con = @build_constraint(expres<=1)
                                 MOI.submit(my_model, MOI.UserCut(cb_data), con)
                                 parameters.number_of_cuts += 1
                            end  
                        end    
                    end
                end 
    
        end
            parameters.cut_time += time  
        end
    end

        if parameters.cut_class !="none"
                                                                                                
            MOI.set(my_model, MOI.UserCutCallback(), my_callback_cuts_reduced)

        end                                                                                  
                                                                                            
                                                                                            
   function my_lazy_constraint_callback(cb_data)
        TOL = 1e-6                                                                                        
        time = @elapsed begin 
           test_solution = 0.0                                                                                            
          if parameters.objf== "minNFS"
              test_solution= sum((props(instance.physical_network,u)[:ram_cost]*instance.set_VNFs[f].ram_request+props(instance.physical_network,u)[:storage_cost]*instance.set_VNFs[f].storage_request+props(instance.physical_network,u)[:cpu_cost]*instance.set_VNFs[f].cpu_request)*callback_value(cb_data,y[f,m,u]) for f in 1:length(instance.set_VNFs),  m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes])                    
            else
                 test_solution = callback_value(cb_data, U) 
            end   
                
            if  test_solution  <= parameters.best_sol
                parameters.best_sol = test_solution  
             Threads.@threads for u in 1: props(instance.physical_network)[:number_nodes]
                if props(instance.physical_network, u)[:node_type] != "app"
                    for  s in 1:length(instance.setSlices)-1, t in s+1:length(instance.setSlices), f in 1:length(instance.set_VNFs), g in 1:length(instance.set_VNFs), n in 1:instance.number_of_NFs
                        #---------------------------------------------
                        # VIRTUAL ISOLATION
                        #---------------------------------------------
                        if instance.setSlices[s].VNF_sharing["$(t)"][f][g]*instance.setSlices[t].VNF_sharing["$(s)"][g][f]<=0+TOL && callback_value(cb_data, x[s,f,n,u])*callback_value(cb_data, x[t,g,n,u])>0 + TOL 
                            con = @build_constraint(x[s,f,n,u] + x[t,g,n,u]  <= 1)
                            MOI.submit(my_model, MOI.LazyConstraint(cb_data), con)

                            parameters.number_of_lazy_cuts += 1                                                                                            
                         end 
                        #---------------------------------------------
                        # Physical ISOLATION
                        #---------------------------------------------
                         if props(instance.physical_network, u)[:node_type] == "non_access"
                            for m in 1:instance.number_of_NFs
                                if instance.setSlices[s].nodeSharing[t]*instance.setSlices[t].nodeSharing[s]<=0+TOL && callback_value(cb_data, x[s,f,m,u])*callback_value(cb_data, x[t,g,m,u])>0 + TOL                                                                                             
                                    con = @build_constraint(x[s,f,m,u] + x[t,g,n,u]  <= 1)
                                    MOI.submit(my_model, MOI.LazyConstraint(cb_data), con) 
                                    parameters.number_of_lazy_cuts += 1  

                                end
                            end
                        end
                        #----------------------------------------------------------------------------                                                                                           
                        #packing   
                        #-------------------------------------                                                                                           
                        for v in 1:props(instance.physical_network)[:number_nodes] 
                            if props(instance.physical_network, v)[:node_type] != "app"&&  v == u+1 && callback_value(cb_data, x[s,f,n,u])*callback_value(cb_data, x[t,g,n,v])>0 + TOL
                                con = @build_constraint(x[s,f,n,u] + x[t,g,n,v]  <= 1)
                                MOI.submit(my_model, MOI.LazyConstraint(cb_data), con) 
                                parameters.number_of_lazy_cuts += 1 
                                                                                                                        
                            end
                        end                                                                                                
                    end 
                end # if
                 end # end of all u  

               #---------------------------------------------
                #  LATENCY
                #---------------------------------------------
            Threads.@threads  for s in 1:length(instance.setSlices)
                  Threads.@threads  for k in 1:length(instance.setSlices[s].set_commodities)
                    delay_e2e = zero(AffExpr)
                    testE2E = 0.0                                                                                         
                    for f in 1:(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),g in 1:(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs)
                        delay_pair = zero(AffExpr)
                        testPair = 0.0 
                        for a in edges(instance.physical_network)
                            delay_a = get_prop(instance.physical_network,a,:delay)
                            if f < instance.number_of_AN_based_NFs && g == f+1
                                test_delay= delay_a*callback_value(cb_data,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])                                                                                        
                                if test_delay > 0 + TOL
                                    add_to_expression!(delay_e2e, delay_a*gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])
                                    testE2E+=test_delay                                                                                       
                                end
                            end 
                            if f == 1 && g == 1
                                test_delay= delay_a*callback_value(cb_data,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1])                                                                                         
                                if test_delay > 0 + TOL
                                    add_to_expression!(delay_e2e, delay_a * gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1])
                                    testE2E+=test_delay                                                                                
                                end                                                                                        
                                test_delay= delay_a*callback_value(cb_data,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1])                                                                                         
                                if test_delay > 0 + TOL
                                    add_to_expression!(delay_e2e, delay_a * gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1])
                                    testE2E+=test_delay 
                                end 
                            end

                            if instance.VNF_connection["$(s)"][f][g] == true
                                test_delay= delay_a*callback_value(cb_data, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])                                                                                        
                                if test_delay > 0 + TOL
                                    add_to_expression!(delay_pair, delay_a * gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])
                                    testPair+=test_delay                                                                                       
                                end
                            end
                        end # end for arcs
                        if testPair > instance.maxLatencyBetweenFunctions[f][g]                                                                                        
                            con = @build_constraint(delay_pair <= instance.maxLatencyBetweenFunctions[f][g])
                            MOI.submit(my_model, MOI.LazyConstraint(cb_data), con) 

                            parameters.number_of_lazy_cuts += 1   
                        end
                    end # end for functions
                    if testE2E > instance.setSlices[s].maxLatencyDataLayer
                        con = @build_constraint( delay_e2e <= instance.setSlices[s].maxLatencyDataLayer)
                        MOI.submit(my_model, MOI.LazyConstraint(cb_data), con) 
                        parameters.number_of_lazy_cuts += 1

                                                                                            
                    end
                end
            end # end of latency  
           end                                                                                       
                                                                                                   
                                                                                                        
         end # end of time
         parameters.lazy_time += time 
    end # end of my_lazy_constraint_callback function

                                                                                                
                                                                           
    MOI.set(my_model, MOI.LazyConstraintCallback(), my_lazy_constraint_callback)                                                                                   
                                                                                            
                                                                                            
                                                                                            
    #-------------------------- objective-----------------------------------------
    #-------------------------- objective-----------------------------------------
    if parameters.objf== "minNFS"
        @objective(my_model, Min , sum((props(instance.physical_network,u)[:ram_cost]*instance.set_VNFs[f].ram_request+props(instance.physical_network,u)[:storage_cost]*instance.set_VNFs[f].storage_request+props(instance.physical_network,u)[:cpu_cost]*instance.set_VNFs[f].cpu_request)*y[f,m,u] for f in 1:length(instance.set_VNFs),  m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]))                    
    else
         @objective(my_model, Min, U) 
    end
    return my_model

end



function create_NSDP_complete_model(instance::Instance,parameters::Parameters)
    
    my_model = 1
    #my_model = Model(CPLEX.Optimizer
            
        if parameters.solver=="CPLEX"
        my_model =Model(CPLEX.Optimizer)
        #set_optimizer_attributes(my_model, "CPX_PARAM_TILIM" => 60*60*5)
       if parameters.warming_up == true
            set_optimizer_attributes(my_model,  "CPX_PARAM_THREADS"=>15)
            set_optimizer_attributes(my_model,  "CPX_PARAM_EPGAP"=>1.0)

         else 
            #set_optimizer_attributes(my_model, "CPX_PARAM_PREIND"=>0)
           #set_optimizer_attributes(my_model,   "CPX_PARAM_BNDSTRENIND"=>0)
           # set_optimizer_attributes(my_model,  "CPX_PARAM_COEREDIND"=>0)
           #set_optimizer_attributes(my_model,  "CPX_PARAM_RELAXPREIND"=>0)
           # set_optimizer_attributes(my_model,  "CPX_PARAM_REDUCE"=>0)
            #set_optimizer_attributes(my_model,  "CPX_PARAM_PREPASS"=>0)
            #set_optimizer_attributes(my_model,  "CPX_PARAM_PRELINEAR"=>0)
            set_optimizer_attributes(my_model,  "CPX_PARAM_THREADS"=>12)
                        set_optimizer_attributes(my_model, "CPX_PARAM_TILIM" => 60*60*3)



         end 
            
        set_optimizer_attributes(my_model,  "CPX_PARAM_MIPDISPLAY"=>4)  
        set_optimizer_attributes(my_model, "CPX_PARAM_MIPINTERVAL" => 100000)
        #set_optimizer_attributes(my_model,  "CPX_PARAM_MIPEMPHASIS"=>1)
        set_optimizer_attributes(my_model,  "CPX_PARAM_WORKMEM"=>1024*10)
        set_optimizer_attributes(my_model,  "CPX_PARAM_NODEFILEIND"=>2)
        
        if parameters.boost == false 
            set_optimizer_attributes(my_model, "CPX_PARAM_TILIM" => 60*60)
            set_optimizer_attributes(my_model, "CPX_PARAM_PREIND"=>0)
            #set_optimizer_attributes(my_model,   "CPX_PARAM_BNDSTRENIND"=>0)
            #set_optimizer_attributes(my_model,  "CPX_PARAM_COEREDIND"=>0)
           # set_optimizer_attributes(my_model,  "CPX_PARAM_RELAXPREIND"=>0)
            set_optimizer_attributes(my_model,  "CPX_PARAM_REDUCE"=>0)
            set_optimizer_attributes(my_model,  "CPX_PARAM_PREPASS"=>0)
            #set_optimizer_attributes(my_model,  "CPX_PARAM_REPEATPRESOLVE"=>0)
            set_optimizer_attributes(my_model,"CPX_PARAM_HEURFREQ" => -1)
            set_optimizer_attributes(my_model,   "CPX_PARAM_SYMMETRY" => 0)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_BQP"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Cliques"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Covers"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Disjunctive"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_FlowCovers"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_PathCut"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Gomory"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_GUBCovers"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_Implied"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_LocalImplied"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_LiftProj"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_MIRCut"=>-1)
            #set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cut_MCFCut"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_RLT"=>-1)
            set_optimizer_attributes(my_model,  "CPXPARAM_MIP_Cuts_ZeroHalfCut"=>-1)
        end
    end 
        

    #-------------------------Variables--------------------
    @variable(my_model, gamma[s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities), a in 1:props(instance.physical_network)[:number_of_arcs],f in 1:length(instance.set_VNFs)+1, g in 1:length(instance.set_VNFs)+1], Bin)
    @variable(my_model, z[s in 1:length(instance.setSlices),f in 1:instance.number_of_AN_based_NFs],Bin)
    @variable(my_model, x[s in 1:length(instance.setSlices),f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]], Bin)
    @variable(my_model, w[s in 1:length(instance.setSlices),f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]] >= 0)
    @variable(my_model, y[f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]] >=0, Int)
    if parameters.objf== "minLINKload"                                
        @variable(my_model, 0 <= U <= 1)
    end

    test = 0
 #---------------------------------------------
    # SPLIT SELECTION
    #---------------------------------------------
    for s in 1:length(instance.setSlices),  f in 1:(instance.number_of_AN_based_NFs-1)
        @constraint(my_model, z[s,f]<=z[s,f+1])
    end
    
   
    #---------------------------------------------
    # DIMENSIONING - CP NFSs
    #---------------------------------------------
    for s in 1:length(instance.setSlices),  f in (instance.number_of_AN_based_NFs+1):(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),u in 1: props(instance.physical_network)[:number_nodes],n in 1:instance.number_of_NFs
        @constraint(my_model, w[s,f,n,u]  == instance.set_VNFs[f].dataVolPerUE*instance.setSlices[s].TotalAmountUE*x[s,f,n,u] /instance.set_VNFs[f].treatment_capacity)
    end 
    #---------------------------------------------
    # DIMENSIONING - DISTRIBUTED DP NFSs
    #---------------------------------------------
     for s in 1:length(instance.setSlices),  k in instance.setSlices[s].set_commodities, f in 1:instance.number_of_AN_based_NFs,n in 1:instance.number_of_NFs
         if f!=1
            @constraint(my_model, w[s,f,n,k["origin_node"]]  == instance.set_VNFs[f-1].compression*k["volume_of_data"]*x[s,f,n,k["origin_node"]]/instance.set_VNFs[f].treatment_capacity)
    
        else
            @constraint(my_model, w[s,f,n,k["origin_node"]] == k["volume_of_data"]*x[s,f,n,k["origin_node"]]/instance.set_VNFs[f].treatment_capacity)
        end
    end
    #---------------------------------------------
    # DIMENSIONING - CENTRALIZED DP NFSs
    #---------------------------------------------
     for s in 1:length(instance.setSlices),  f in 1:instance.number_of_AN_based_NFs, u in 1: props(instance.physical_network)[:number_nodes],n in 1:instance.number_of_NFs
        if props(instance.physical_network, u)[:node_type] == "non_access" && f!=1
            @constraint(my_model, w[s,f,n,u] == instance.setSlices[s].set_VNFs_to_install[f]*(sum(instance.set_VNFs[f-1].compression*k["volume_of_data"] for k in instance.setSlices[s].set_commodities)*x[s,f,n,u])/instance.set_VNFs[f].treatment_capacity)                
        elseif props(instance.physical_network, u)[:node_type] == "non_access" && f==1
            @constraint(my_model, w[s,f,n,u] ==  instance.setSlices[s].set_VNFs_to_install[f]*(sum(k["volume_of_data"] for k in instance.setSlices[s].set_commodities)*x[s,f,n,u])/instance.set_VNFs[f].treatment_capacity)                
        end
    end
                                           
    #---------------------------------------------
    # PLACEMENT - DISTRIBUTED DP NFSs
    #---------------------------------------------
    for s in 1:length(instance.setSlices) 
        O = Vector()   
        for k in instance.setSlices[s].set_commodities
             push!(O,k["origin_node"])                            
        end
        for f in 1:instance.number_of_AN_based_NFs,  u in 1: props(instance.physical_network)[:number_nodes]
            if u ∈ O 
                @constraint(my_model, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs)== instance.setSlices[s].set_VNFs_to_install[f]*(1 - z[s,f]))
            elseif props(instance.physical_network, u)[:node_type] == "access"
                @constraint(my_model, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs)== 0)
            end
        end
    end
            
  
    #---------------------------------------------
    # PLACEMENT - CENTRALIZED DP NFSs      --------------
    #---------------------------------------------
     for s in 1:length(instance.setSlices),  f in 1:instance.number_of_AN_based_NFs
        expression = zero(AffExpr)
        for u in 1: props(instance.physical_network)[:number_nodes]
            if props(instance.physical_network, u)[:node_type] != "access"
                add_to_expression!(expression, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs))                           
            end

        end
        @constraint(my_model, expression == instance.setSlices[s].set_VNFs_to_install[f]*z[s,f])  
                               
    end
                
    #---------------------------------------------
    # PLACEMENT - CP NFSs
    #---------------------------------------------
     for s in 1:length(instance.setSlices),  f in (1+instance.number_of_AN_based_NFs):(instance.number_of_AN_based_NFs + instance.number_of_CN_based_NFs)
        expression = zero(AffExpr)
         expression2 = zero(AffExpr)                            
        for u in 1: props(instance.physical_network)[:number_nodes]
            if props(instance.physical_network, u)[:node_type] == "non_access"
                add_to_expression!(expression, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs))
            else
                add_to_expression!(expression, sum(x[s,f,m,u] for m in 1:instance.number_of_NFs))
            end
        end       
        @constraint(my_model, expression == instance.setSlices[s].set_VNFs_to_install[f]) 
        @constraint(my_model, expression2 ==0) 

     end


    
    #---------------------------------------------
    # VIRTUAL ISOLATION
    #---------------------------------------------
    for s in 1:length(instance.setSlices),t in 1:length(instance.setSlices)
        if  s!=t
            for u in 1: props(instance.physical_network)[:number_nodes],  m in 1:instance.number_of_NFs,  f in 1:length(instance.set_VNFs),g in 1:length(instance.set_VNFs)            
                @constraint(my_model, x[s,f,m,u] + x[t,g,m,u]  <= 1 + instance.setSlices[s].VNF_sharing["$(t)"][f][g]*instance.setSlices[t].VNF_sharing["$(s)"][g][f])  
             end
        end
    end

    #---------------------------------------------
    # Physical ISOLATION
    #---------------------------------------------         
    for u in 1: props(instance.physical_network)[:number_nodes]
        if props(instance.physical_network, u)[:node_type] == "non_access"
           for  s in 1:length(instance.setSlices)-1, t in s+1:length(instance.setSlices), f in 1:length(instance.set_VNFs), g in 1:length(instance.set_VNFs)
                @constraint(my_model, sum(x[s,f,n,u] for n in 1:instance.number_of_NFs) + sum(x[t,g,m,u] for m in 1:instance.number_of_NFs)<=instance.setSlices[s].nodeSharing[t]*instance.setSlices[t].nodeSharing[s] +1 )
            end    
        end
    end
    #---------------------------------------------
    # PACKING
    #--------------------------------------------
    for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]
        @constraint(my_model, sum(w[s,f,m,u] for s in 1:length(instance.setSlices))  <= y[f,m,u])
        if parameters.objf== "minLINKload"                                
            @constraint(my_model, sum(w[s,f,m,u] for s in 1:length(instance.setSlices)) +0.99999999 >= y[f,m,u])
        end
    end
            
    #for s in 1:length(instance.setSlices)-1,t in s+1:length(instance.setSlices), f in 1:length(instance.set_VNFs), g in 1:length(instance.set_VNFs), u in 1:props(instance.physical_network)[:number_nodes]-1,v in u+1: props(instance.physical_network)[:number_nodes], n in 1:instance.number_of_NFs
     #   if u!=v
      #      @constraint(my_model, x[s,f,n,u] + x[t,g,n,v] <=1)
       # end
    #end       
    #---------------------------------------------
    # NODE CAPACITY
    #---------------------------------------------
    for u in 1: props(instance.physical_network)[:number_nodes]
        @constraint(my_model,  sum(instance.set_VNFs[f].ram_request*y[f,m,u] for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs) <= props(instance.physical_network,u)[:ram_capacity])
        @constraint(my_model,  sum(instance.set_VNFs[f].storage_request*y[f,m,u] for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs) <= props(instance.physical_network,u)[:storage_capacity])
        @constraint(my_model,  sum(instance.set_VNFs[f].cpu_request*y[f,m,u] for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs) <= props(instance.physical_network,u)[:cpu_capacity])
     end
                 
                      
    
#---------------------------------------------
    # FLOW - DP
    #---------------------------------------------
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities), f in 1:(instance.number_of_AN_based_NFs-1),u in 1: props(instance.physical_network)[:number_nodes]
            
        ingoing = zero(AffExpr)
        outgoing = zero(AffExpr)
        ingoing_target= zero(AffExpr)
        outgoing_target = zero(AffExpr)
        ingoing_origin = zero(AffExpr)
        outgoing_origin = zero(AffExpr)
        conttt=0
        for a in edges(instance.physical_network)
            if src(a) == u
               add_to_expression!(outgoing_target, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1])
               add_to_expression!(outgoing_origin, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1])
               add_to_expression!(outgoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1])
            elseif dst(a) == u
                add_to_expression!(ingoing_target, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1])
                add_to_expression!(ingoing_origin, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] )
                add_to_expression!(ingoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1])
            end
        end
        if props(instance.physical_network, u)[:node_type] != "access"
            @constraint(my_model, outgoing - ingoing == sum(x[s,f,m,u] for m in 1:instance.number_of_NFs) -sum(x[s,f+1,m,u] for m in 1:instance.number_of_NFs))
        elseif u!=instance.setSlices[s].set_commodities[k]["origin_node"]
            @constraint(my_model, outgoing - ingoing == 0)
        else               
             @constraint(my_model, outgoing  == z[s,f+1] - z[s,f])
        end  
        if  u==instance.setSlices[s].set_commodities[k]["target_node"]
            @constraint(my_model, outgoing_target - ingoing_target == -1)
            @constraint(my_model, outgoing_origin - ingoing_origin == 0)        
        
        elseif  u==instance.setSlices[s].set_commodities[k]["origin_node"]
            @constraint(my_model, outgoing_target - ingoing_target == 1-z[s,instance.number_of_AN_based_NFs])
            @constraint(my_model, outgoing_origin - ingoing_origin == z[s,1])

        
        elseif  props(instance.physical_network, u)[:node_type] == "non_access" 
            @constraint(my_model, outgoing_target - ingoing_target == sum(x[s,instance.number_of_AN_based_NFs,m,u] for m in 1:instance.number_of_NFs))
            @constraint(my_model, outgoing_origin - ingoing_origin == -sum(x[s,1,m,u] for m in 1:instance.number_of_NFs))
        
        else 
            @constraint(my_model, outgoing_target - ingoing_target == 0)
            @constraint(my_model, outgoing_origin - ingoing_origin == 0)
        end
    end
                
    #---------------------------------------------
    # FLOW - BETWEEN CP NFSs
    #---------------------------------------------
    for s in 1:length(instance.setSlices),  f in (instance.number_of_AN_based_NFs+1):(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),g in (instance.number_of_AN_based_NFs+1):(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),u in 1: props(instance.physical_network)[:number_nodes]
        if f!=g && instance.VNF_connection["$(s)"][f][g] == true && instance.setSlices[s].set_VNFs_to_install[g] == true && instance.setSlices[s].set_VNFs_to_install[f] == true
            ingoing = zero(AffExpr)
            outgoing = zero(AffExpr)
            for a in edges(instance.physical_network)
              if src(a) == u
                   add_to_expression!(outgoing, gamma[s,1,get_prop(instance.physical_network,a,:edge_id),f,g])         
                end
                if dst(a) == u
                   add_to_expression!(ingoing,gamma[s,1,get_prop(instance.physical_network,a,:edge_id),f,g])          
                end
            end
            @constraint(my_model, outgoing - ingoing == sum(x[s,f,m,u] for m in 1:instance.number_of_NFs) - sum(x[s,g,m,u] for m in 1:instance.number_of_NFs))
        end
    end
                
    #---------------------------------------------
    # FLOW - BETWEEN DP AND CP NFSs
    #---------------------------------------------               
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities), f in (instance.number_of_AN_based_NFs+1):(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),g in 1:instance.number_of_AN_based_NFs,u in 1: props(instance.physical_network)[:number_nodes]
        if instance.VNF_connection["$(s)"][f][g] == true && instance.setSlices[s].set_VNFs_to_install[g] == true && instance.setSlices[s].set_VNFs_to_install[f] == true
            ingoing = zero(AffExpr)
            outgoing = zero(AffExpr)
            for a in edges(instance.physical_network)
              if src(a) == u
                   add_to_expression!(outgoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])          
                end
                if dst(a) == u
                   add_to_expression!(ingoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])         
                end
            end
            if props(instance.physical_network, u)[:node_type] == "non_access"           
                @constraint(my_model, outgoing - ingoing == sum(x[s,f,m,u] for m in 1:instance.number_of_NFs) - sum(x[s,g,m,u] for m in 1:instance.number_of_NFs))

            elseif props(instance.physical_network, u)[:node_type] == "access" && props(instance.physical_network, u)[:node_id] ==  instance.setSlices[s].set_commodities[k]["origin_node"]           
                @constraint(my_model, outgoing - ingoing == z[s,g]-1)
            else
               @constraint(my_model, outgoing - ingoing == 0)
            end
        end
    end        

                    
  for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities),f in 1:instance.number_of_AN_based_NFs,g in (instance.number_of_AN_based_NFs+1):instance.number_of_AN_based_NFs+instance.number_of_CN_based_NFs,u in 1: props(instance.physical_network)[:number_nodes]
        if instance.VNF_connection["$(s)"][f][g] == true && instance.setSlices[s].set_VNFs_to_install[g] == true && instance.setSlices[s].set_VNFs_to_install[f] == true
            ingoing = zero(AffExpr)
            outgoing = zero(AffExpr)
            for a in edges(instance.physical_network)
              if src(a) == u
                   add_to_expression!(outgoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])         
                end
                if dst(a) == u
                   add_to_expression!(ingoing, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])         
                end
            end
            if props(instance.physical_network, u)[:node_type] == "non_access"           
                @constraint(my_model, outgoing - ingoing == sum(x[s,f,m,u] for m in 1:instance.number_of_NFs) - sum(x[s,g,m,u] for m in 1:instance.number_of_NFs))
            
            elseif props(instance.physical_network, u)[:node_type] == "access" && props(instance.physical_network, u)[:node_id] ==  instance.setSlices[s].set_commodities[k]["origin_node"]           
                 @constraint(my_model, outgoing - ingoing == 1 - z[s,f])
            else
                 @constraint(my_model, outgoing - ingoing == 0)            
            end
        end
    end            
        
    #---------------------------------------------
    #  LATENCY
    #---------------------------------------------
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities)
        delay_e2e = zero(AffExpr)
        for f in 1:(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs),g in 1:(instance.number_of_CN_based_NFs+instance.number_of_AN_based_NFs)
            delay_pair = zero(AffExpr)
            for a in edges(instance.physical_network)
                delay_a = get_prop(instance.physical_network,a,:delay)
                if f < instance.number_of_AN_based_NFs && g == f+1
                    add_to_expression!(delay_e2e, delay_a*gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g])
                end 
                if f == 1 && g == 1
                    add_to_expression!(delay_e2e, delay_a * (gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] +gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1]))
                 end 
                if instance.VNF_connection["$(s)"][f][g] == true
                    add_to_expression!(delay_pair, delay_a * gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]) 
                end
            end
            @constraint(my_model, delay_pair <= instance.maxLatencyBetweenFunctions[f][g])  
        end
        @constraint(my_model, delay_e2e <= instance.setSlices[s].maxLatencyDataLayer) 
    end

   
    #---------------------------------------------
    # LINK CAPACITY
                                                
    #---------------------------------------------
    for a in edges(instance.physical_network)           
        sum_gamma = zero(AffExpr)            
        for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities)
            add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]*instance.setSlices[s].set_commodities[k]["volume_of_data"])
            add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1]*instance.setSlices[s].set_commodities[k]["volume_of_data"]*instance.set_VNFs[instance.number_of_AN_based_NFs].compression)
            for f in 1:(instance.number_of_AN_based_NFs+instance.number_of_CN_based_NFs), g in 1:(instance.number_of_AN_based_NFs+instance.number_of_CN_based_NFs)
                if f!=g 
                    if f <= instance.number_of_AN_based_NFs && g <= instance.number_of_AN_based_NFs && f==g-1                      
                        add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]*instance.setSlices[s].set_commodities[k]["volume_of_data"]*instance.set_VNFs[f].compression)
                    end                            
                    if f>instance.number_of_AN_based_NFs && g>instance.number_of_AN_based_NFs
                        add_to_expression!(sum_gamma,gamma[s,1,get_prop(instance.physical_network,a,:edge_id),f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities)))
                    elseif f>instance.number_of_AN_based_NFs && g<=instance.number_of_AN_based_NFs
                        add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities)))
                    elseif f<=instance.number_of_AN_based_NFs && g>instance.number_of_AN_based_NFs
                        add_to_expression!(sum_gamma,gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities)))

                    end
                end

            end
        end
        if parameters.objf== "minNFS"                                
            @constraint(my_model, sum_gamma <= get_prop(instance.physical_network,a,:max_bandwidth))
        else                               
            @constraint(my_model, sum_gamma <= U*get_prop(instance.physical_network,a,:max_bandwidth))                      
        end
    end

      parameters.number_constraints_before_VI = sum(num_constraints(my_model, F, S) for (F,S) in list_of_constraint_types(my_model))

     
  parameters.vi_creation_time = @elapsed begin                                    
          #-------------------------- VALID INEQUALITIES-----------------------------------------


        if parameters.valid_inequalities == "SymmetryBreaking" || parameters.valid_inequalities=="all" || parameters.valid_inequalities=="both"
            #"SymmetryBreaking1" Inequalities
            for n in 1:(instance.number_of_NFs - 1)
                @constraint(my_model, sum(x[s,f,n+1,u] for s in 1:length(instance.setSlices),f in 1:length(instance.set_VNFs), u in 1:props(instance.physical_network)[:number_nodes]) <=instance.number_of_NFs*sum(x[t,g,n,v] for t in 1:length(instance.setSlices),g in 1:length(instance.set_VNFs), v in 1: props(instance.physical_network)[:number_nodes]))  
            end 
        end

                                                                                
        if parameters.valid_inequalities == "LB" || parameters.valid_inequalities=="all" || parameters.valid_inequalities=="both"
         dim_f = []

         for f in 1:length(instance.set_VNFs)
            dim = 0.0 
            for  s in 1:length(instance.setSlices)
                if f!=1 && instance.set_VNFs[f].typee != :cp 
                    dim += sum(instance.setSlices[s].set_VNFs_to_install[f]*instance.set_VNFs[f-1].compression*k["volume_of_data"]  for k in instance.setSlices[s].set_commodities)/instance.set_VNFs[f].treatment_capacity                
                elseif f== 1  && instance.set_VNFs[f].typee != :cp 
                    dim += sum(instance.setSlices[s].set_VNFs_to_install[f]*k["volume_of_data"] for  k in instance.setSlices[s].set_commodities)/instance.set_VNFs[f].treatment_capacity                
                else
                    dim += instance.setSlices[s].set_VNFs_to_install[f]*instance.set_VNFs[f].dataVolPerUE*instance.setSlices[s].TotalAmountUE/instance.set_VNFs[f].treatment_capacity
                end
            end
            push!(dim_f,ceil(dim))    
        end
                         

         LB =  sum(dim_f[i] for i in 1:length(dim_f))
         my_Ys = zero(AffExpr)
        add_to_expression!(my_Ys, sum(y[f,m,u] for f in 1:length(instance.set_VNFs),  m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]))
         @constraint(my_model, my_Ys >= LB)                                                                                   
    end                                                                         
                                                                                
                                                                                
                                                                            
        if parameters.valid_inequalities == "ShortestPath"|| parameters.valid_inequalities=="all"
            for a in edges(instance.physical_network)
                set_prop!(instance.physical_network, src(a) , dst(a) , :weight, get_prop(instance.physical_network,a,:delay))
            end
                                                                                #"ShortestPath"-based Inequalities
            for s in 1:length(instance.setSlices),k in 1:length(instance.setSlices[s].set_commodities)
                my_path = zero(AffExpr)
                add_to_expression!(my_path,sum(get_prop(instance.physical_network,a,:delay)*(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] 
                        + gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]) for a in edges(instance.physical_network))) 
                +sum(sum(get_prop(instance.physical_network,a,:delay)*gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for a in edges(instance.physical_network),f in 1:(instance.number_of_AN_based_NFs-1))) 
                
                 shortest_path_of_K = enumerate_paths(dijkstra_shortest_paths(instance.physical_network, instance.setSlices[s].set_commodities[k]["origin_node"]), instance.setSlices[s].set_commodities[k]["target_node"])    
                 shortest_latency = 0.0
                for a in 1:length(shortest_path_of_K)-1
                     shortest_latency+= get_prop(instance.physical_network,shortest_path_of_K[a],shortest_path_of_K[a+1],:delay)
                end
                @constraint(my_model, my_path >= shortest_latency)
            end 
        end                                                                    
 

        if parameters.valid_inequalities == "MinCut"|| parameters.valid_inequalities=="all"

           #"MinCut" inequalities
            numberNodes = props(instance.physical_network)[:number_nodes]
            flow_graph = DiGraph(numberNodes) 
            capacity_matrix = zeros(Float64, numberNodes, numberNodes) 
            for a in edges(instance.physical_network)
                add_edge!(flow_graph, src(a), dst(a))
                capacity_matrix[src(a),dst(a)] = get_prop(instance.physical_network,a,:max_bandwidth)
            end
            for s in 1:length(instance.setSlices),  k in length(instance.setSlices[s].set_commodities)

                f,F = LightGraphsFlows.maximum_flow(flow_graph, instance.setSlices[s].set_commodities[k]["origin_node"], instance.setSlices[s].set_commodities[k]["target_node"], capacity_matrix,LightGraphsFlows.EdmondsKarpAlgorithm()) # Run default maximum_flow with the capacity_matrix
                f2,F2 = LightGraphsFlows.maximum_flow(flow_graph, instance.setSlices[s].set_commodities[k]["origin_node"], instance.setSlices[s].set_commodities[k]["target_node"], capacity_matrix,LightGraphsFlows.DinicAlgorithm()) # Run default maximum_flow with the capacity_matrix
                f3,F3 = LightGraphsFlows.maximum_flow(flow_graph, instance.setSlices[s].set_commodities[k]["origin_node"], instance.setSlices[s].set_commodities[k]["target_node"], capacity_matrix,LightGraphsFlows.BoykovKolmogorovAlgorithm()) # Run default maximum_flow with the capacity_matrix

                     expression1 = zero(AffExpr)
                     expression2 = zero(AffExpr) 
                     expression3 = zero(AffExpr) 

                     expression1b = zero(AffExpr)
                     expression2b = zero(AffExpr) 
                     expression3b = zero(AffExpr)
                    for a in edges(instance.physical_network)
                        if F[src(a),dst(a)]!=0.0
                           add_to_expression!(expression1, sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] )
                           add_to_expression!( expression1b, get_prop(instance.physical_network,a,:max_bandwidth)*(sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]) )

                        end
                        if F2[src(a),dst(a)]!=0.0
                            add_to_expression!(expression2, sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]) 
                            add_to_expression!(expression2b, get_prop(instance.physical_network,a,:max_bandwidth)*(sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] ))

                        end
                        if F3[src(a),dst(a)]!=0.0
                           add_to_expression!( expression3, sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1] )
                             add_to_expression!(expression3b, get_prop(instance.physical_network,a,:max_bandwidth)*(sum(gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,f+1] for f in 1:(instance.number_of_AN_based_NFs-1))+ gamma[s,k,get_prop(instance.physical_network,a,:edge_id),instance.number_of_AN_based_NFs,length(instance.set_VNFs)+1] +  gamma[s,k,get_prop(instance.physical_network,a,:edge_id),length(instance.set_VNFs)+1,1]))

                        end
                     end 
                @constraint(my_model, expression1>=1)
                @constraint(my_model, expression2>=1)
                @constraint(my_model, expression3>=1)

                @constraint(my_model, expression1b>=instance.set_VNFs[instance.number_of_AN_based_NFs].compression*instance.setSlices[s].set_commodities[k]["volume_of_data"])
                @constraint(my_model, expression2b>=instance.set_VNFs[instance.number_of_AN_based_NFs].compression*instance.setSlices[s].set_commodities[k]["volume_of_data"])
                @constraint(my_model, expression3b>=instance.set_VNFs[instance.number_of_AN_based_NFs].compression*instance.setSlices[s].set_commodities[k]["volume_of_data"])

            end
        end
    end

    parameters.number_constraints_after_VI = sum(num_constraints(my_model, F, S) for (F,S) in list_of_constraint_types(my_model))
    
                                        
    function my_callback_cuts_complete(cb_data)
           time = @elapsed begin                                                                           
                if parameters.cut_class=="isolation_cuts" ||  parameters.cut_class=="all" #nao achei
                  for physical_node in 1:props(instance.physical_network)[:number_nodes]
                             aux_vector = Vector{Dict}()
                            if props(instance.physical_network, physical_node)[:node_type] == "non_access" 
                            for s in 1:length(instance.setSlices), f in 1:length(instance.set_VNFs), n in 1:instance.number_of_NFs
                                if callback_value(cb_data, x[s,f,n,physical_node]) >0.000001
                                    aux_dict = Dict()
                                    aux_dict["id"] = length(aux_vector)+1
                                    aux_dict["slice"] = s
                                    aux_dict["NF"] = n
                                    aux_dict["function"] = f
                                    push!(aux_vector, aux_dict)
                                    end #ends if
                                end # end for s

                            GraphNODEisolation = SimpleGraph(length(aux_vector))
                            for u in 1:length(aux_vector)-1
                                  for v in u+1:length(aux_vector)
                                    if  aux_vector[v]["slice"] != aux_vector[u]["slice"] && instance.setSlices[aux_vector[v]["slice"]].nodeSharing[aux_vector[u]["slice"]]*instance.setSlices[aux_vector[u]["slice"]].nodeSharing[aux_vector[v]["slice"]] ==0
                                        add_edge!(GraphNODEisolation, u, v)
                                        end # end if
                                    end#enf vor v
                                end#end for u

                            my_cliques = maximal_cliques(GraphNODEisolation)


                        for cl in my_cliques                
                                if length(cl) >= 3
                                    expres = zero(AffExpr)
                                     test = 0.0                                                               
                                    for node in cl
                                        add_to_expression!(expres,x[aux_vector[node]["slice"],aux_vector[node]["function"],aux_vector[node]["NF"],physical_node])
                                         test+=callback_value(cb_data,x[aux_vector[node]["slice"],aux_vector[node]["function"],aux_vector[node]["NF"],physical_node])
                                        end #end for
                                     if test >1                                                               
                                         con = @build_constraint(expres<=1)
                                         MOI.submit(my_model, MOI.UserCut(cb_data), con)
                                         parameters.number_of_cuts += 1
                                        end  #end if test                                                                  
                                    end #end if length
                                end #end for cliques
                            end #end if non access
                        end #end node 
                    for n in 1:instance.number_of_NFs
                    aux_vector = Vector{Dict}()
                    for s in 1:length(instance.setSlices), f in 1:length(instance.set_VNFs), u in 1:props(instance.physical_network)[:number_nodes]
                     if props(instance.physical_network, u)[:node_type] == "non_access"  && callback_value(cb_data, x[s,f,n,u]) >0.000001  
                            aux_dict = Dict()
                            aux_dict["id"] = length(aux_vector)+1
                            aux_dict["slice"] = s
                            aux_dict["node"] = u
                            aux_dict["function"] = f
                            push!(aux_vector, aux_dict)
                        end#end if
                    end#end fot

                    GraphNFIsolation = SimpleGraph(length(aux_vector))
                     for u in 1:length(aux_vector)-1
                        for v in u+1:length(aux_vector)
                            if aux_vector[v]["slice"] != aux_vector[u]["slice"]                                
                                if instance.setSlices[aux_vector[v]["slice"]].VNF_sharing["$(aux_vector[u]["slice"])"][aux_vector[u]["function"]][aux_vector[v]["function"]]*instance.setSlices[aux_vector[u]["slice"]].VNF_sharing["$(aux_vector[v]["slice"])"][aux_vector[v]["function"]][aux_vector[u]["function"]] ==0
                                    add_edge!(GraphNFIsolation, u, v)
                                end
                            end
                        end
                        end#end threads

                    my_cliques = maximal_cliques(GraphNFIsolation)
                     for cl in my_cliques
                        if length(cl) >= 3
                            expres = zero(AffExpr)
                            test = 0.0                                                                    
                            for node in cl
                                add_to_expression!(expres,x[aux_vector[node]["slice"],aux_vector[node]["function"],n,aux_vector[node]["node"]])
                                test +=  callback_value(cb_data, x[aux_vector[node]["slice"],aux_vector[node]["function"],n,aux_vector[node]["node"]])                                                                  
                            end #end for
                            if test > 1                                                                   
                                 con = @build_constraint(expres<=1)
                                 MOI.submit(my_model, MOI.UserCut(cb_data), con)
                                 parameters.number_of_cuts += 1
                            end  #end if
                        end  #end if lengh   
                    end #end for cliques
                end #end if cut     
                end
                                                                   
           if parameters.cut_class=="capacity_cuts" ||  parameters.cut_class=="all" 
                Threads.@threads for aa in 1:length(edges(instance.physical_network))
                    a = nothing
                    for b in edges(instance.physical_network)
                        if get_prop(instance.physical_network,b,:edge_id) ==aa
                            a = b
                            break
                        end
                    end
                        
                    aux_vector1 = Vector{Dict}()
                    for s in 1:length(instance.setSlices), f in 1:length(instance.set_VNFs)+1,g in 1:length(instance.set_VNFs)+1,k in length(instance.setSlices[s].set_commodities)
                       if  callback_value(cb_data, gamma[s,k,get_prop(instance.physical_network,a,:edge_id),f,g]) > 0.000001 
                             aux_dict = Dict()
                            aux_dict["id"] = length(aux_vector1)+1
                            aux_dict["slice"] = s
                            aux_dict["functionF"] = f
                             aux_dict["functionG"] = g
                            if f > instance.number_of_AN_based_NFs &&  f <= length(instance.set_VNFs)                            
                                aux_dict["commodity"] = 1
                            elseif g > instance.number_of_AN_based_NFs &&  g <= length(instance.set_VNFs)                            
                                aux_dict["commodity"] = 1  
                            else 
                                 aux_dict["commodity"] = k                                       
                            end
                            push!(aux_vector1, aux_dict)
                        end
                    end #end for s

                    GraphLinkCap = SimpleGraph(length(aux_vector1))
                  Threads.@threads for u in 1:length(aux_vector1)-1
                            for v in u+1:length(aux_vector1)
                            requestU = 0.0
                            requestV = 0.0
                                if  aux_vector1[u]["functionF"] < instance.number_of_AN_based_NFs && aux_vector1[u]["functionG"] ==  aux_vector1[u]["functionF"] + 1 
                                    requestU = instance.setSlices[aux_vector1[u]["slice"]].set_commodities[aux_vector1[u]["commodity"]]["volume_of_data"]*instance.set_VNFs[aux_vector1[u]["functionF"]].compression

                                elseif aux_vector1[u]["functionF"] == length(instance.set_VNFs)+1 && aux_vector1[u]["functionG"] == 1 
                                     requestU = instance.setSlices[aux_vector1[u]["slice"]].set_commodities[aux_vector1[u]["commodity"]]["volume_of_data"]

                                elseif aux_vector1[u]["functionF"] == instance.number_of_AN_based_NFs && aux_vector1[u]["functionG"] == length(instance.set_VNFs)+1
                                    requestU = instance.setSlices[aux_vector1[u]["slice"]].set_commodities[aux_vector1[u]["commodity"]]["volume_of_data"]*instance.set_VNFs[aux_vector1[u]["functionF"]].compression
                                else
                                    requestU =  instance.set_VNFs[aux_vector1[u]["functionF"]].amount_of_traffic_sent_to_g[aux_vector1[u]["functionG"]]*instance.setSlices[aux_vector1[u]["slice"]].TotalAmountUE                                             
                                end

                                if  aux_vector1[v]["functionF"] < instance.number_of_AN_based_NFs && aux_vector1[v]["functionG"] ==  aux_vector1[v]["functionF"] + 1 
                                    requestU = instance.setSlices[aux_vector1[v]["slice"]].set_commodities[aux_vector1[v]["commodity"]]["volume_of_data"]*instance.set_VNFs[aux_vector1[v]["functionF"]].compression

                                elseif aux_vector1[v]["functionF"] == length(instance.set_VNFs)+1 && aux_vector1[v]["functionG"] == 1 
                                     requestU = instance.setSlices[aux_vector1[v]["slice"]].set_commodities[aux_vector1[v]["commodity"]]["volume_of_data"]

                                elseif aux_vector1[v]["functionF"] == instance.number_of_AN_based_NFs && aux_vector1[v]["functionG"] == length(instance.set_VNFs)+1
                                    requestU = instance.setSlices[aux_vector1[v]["slice"]].set_commodities[aux_vector1[v]["commodity"]]["volume_of_data"]*instance.set_VNFs[aux_vector1[v]["functionF"]].compression
                                else
                                    requestU =  instance.set_VNFs[aux_vector1[v]["functionF"]].amount_of_traffic_sent_to_g[aux_vector1[v]["functionG"]]*instance.setSlices[aux_vector1[v]["slice"]].TotalAmountUE                                             
                                end                     

                             if (requestV+requestU) > get_prop(instance.physical_network,a,:max_bandwidth)       
                                add_edge!(GraphLinkCap, u, v)
                            end                 
                        end #end for v
                    end #end for u

                    my_cliques = maximal_cliques(GraphLinkCap)
                    for cl in my_cliques
                        if length(cl) >= 3
                            cliqueEx= zero(AffExpr)
                            test = 0.0                                                                                                    
                            for node in cl
                                test+=callback_value(cb_data,gamma[aux_vector1[node]["slice"],aux_vector1[node]["commodity"],get_prop(instance.physical_network,a,:edge_id),aux_vector1[node]["functionF"],aux_vector1[node]["functionG"]])                                                                                                         
                                add_to_expression!(cliqueEx,gamma[aux_vector1[node]["slice"],aux_vector1[node]["commodity"],get_prop(instance.physical_network,a,:edge_id),aux_vector1[node]["functionF"],aux_vector1[node]["functionG"]])          
                            end
                             if test > 1                                                                                                   
                                 con = @build_constraint(cliqueEx<=1)
                                 MOI.submit(my_model, MOI.UserCut(cb_data), con)
                                 parameters.number_of_cuts += 1

                            end
                        end
                    end
                end# for a
            
                resources = ["ram_capacity","storage_capacity","cpu_capacity"]

                Threads.@threads for physical_node in 1:props(instance.physical_network)[:number_nodes] 
                    aux_vector = Vector{Dict}()
                    for s in 1:length(instance.setSlices), f in 1:length(instance.set_VNFs), n in 1:instance.number_of_NFs
                        if callback_value(cb_data, x[s,f,n,physical_node]) >0.00001
                            aux_dict = Dict()
                            aux_dict["id"] = length(aux_vector)+1
                            aux_dict["slice"] = s
                            aux_dict["node"] = physical_node
                            aux_dict["NFS"] = f
                            aux_dict["NF"] = n                                                                
                            push!(aux_vector, aux_dict)
                        end
                    end # for s

                                                            GraphNodeCap = SimpleGraph(length(aux_vector))
                                                            Threads.@threads for u in 1:length(aux_vector)-1
                                                                for v in u+1:length(aux_vector), resource in resources
                                                                    cap_u = callback_value(cb_data, w[aux_vector[u]["slice"],aux_vector[u]["NFS"],aux_vector[u]["NF"],physical_node]) / callback_value(cb_data, x[aux_vector[u]["slice"],aux_vector[u]["NFS"],aux_vector[u]["NF"],physical_node])                                                                    
                                                                    cap_v = callback_value(cb_data, w[aux_vector[v]["slice"],aux_vector[v]["NFS"],aux_vector[v]["NF"],physical_node]) / callback_value(cb_data, x[aux_vector[v]["slice"],aux_vector[v]["NFS"],aux_vector[v]["NF"],physical_node])                                                                    
                                                                    if resource == "ram_capacity"
                                                                        test = cap_u*instance.set_VNFs[aux_vector[u]["NFS"]].ram_request + cap_v*instance.set_VNFs[aux_vector[v]["NFS"]].ram_request
                                                                        if test > props(instance.physical_network,physical_node)[:ram_capacity]
                                                                            add_edge!(GraphNodeCap, u, v)
                                                                        end 
                                                                    end

                                                                    if resource == "storage_capacity"
                                                                        test = cap_u*instance.set_VNFs[aux_vector[u]["NFS"]].storage_request + cap_v*instance.set_VNFs[aux_vector[v]["NFS"]].storage_request
                                                                        if test > props(instance.physical_network,physical_node)[:storage_capacity]
                                                                            add_edge!(GraphNodeCap, u, v)
                                                                        end 
                                                                    end

                                                                    if resource == "cpu_capacity"
                                                                        test = cap_u*instance.set_VNFs[aux_vector[u]["NFS"]].cpu_request + cap_v*instance.set_VNFs[aux_vector[v]["NFS"]].cpu_request
                                                                        if test > props(instance.physical_network,physical_node)[:cpu_capacity]
                                                                            add_edge!(GraphNodeCap, u, v)
                                                                        end 
                                                                    end
                                                                end
                                                            end #for thread u
                                                                my_cliques = maximal_cliques(GraphNodeCap)
                                                                 for cl in my_cliques
                                                                    if length(cl) >= 3
                                                                        expres = zero(AffExpr)
                                                                        test = 0.0                                                                                                    
                                                                        for node in cl
                                                                            add_to_expression!(expres,x[aux_vector[node]["slice"],aux_vector[node]["NFS"],aux_vector[node]["NF"],aux_vector[node]["node"]])
                                                                            test+=callback_value(cb_data, x[aux_vector[node]["slice"],aux_vector[node]["NFS"],aux_vector[node]["NF"],aux_vector[node]["node"]])                                                                                                   
                                                                        end
                                                                        if test > 1                                                                                                   
                                                                             con = @build_constraint(expres<=1)
                                                                             MOI.submit(my_model, MOI.UserCut(cb_data), con)
                                                                             parameters.number_of_cuts += 1
                                                                        end  
                                                                    end    
                                                                end #end for  cl
                end   #end for pphy node
                end
        end # end time
        parameters.cut_time += time      
      end #end funciton

        if parameters.cut_class !="none" && parameters.relaxation == false
                                                                                                
            MOI.set(my_model, MOI.UserCutCallback(), my_callback_cuts_complete)

        end                               
    #-------------------------- objective-----------------------------------------
    #-------------------------- objective-----------------------------------------
    if parameters.objf== "minNFS"
        @objective(my_model, Min , sum((props(instance.physical_network,u)[:ram_cost]*instance.set_VNFs[f].ram_request+props(instance.physical_network,u)[:storage_cost]*instance.set_VNFs[f].storage_request+props(instance.physical_network,u)[:cpu_cost]*instance.set_VNFs[f].cpu_request)*y[f,m,u] for f in 1:length(instance.set_VNFs),  m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]))                    
    else
         @objective(my_model, Min, U) 
    end
    return my_model

end

          

function get_MILP_LOG(folder::String,parameters::Parameters,instance::Instance)
    parameters.root_gap = parameters.final_lower_bound
    open(joinpath(folder,"LOG_MILP_$(parameters.instance_name).log")) do file
        aux2 = ["none"]
        #we read line by line
        for ln in eachline(file)

            #Here, we storage each word / number in a position of aux_string vector
            aux_string = split(ln)

            #error treatment: if it is a line with nothing written, we return to the first isntruction of this loop
            if length(aux_string) == 0
                continue #going dic=rectly to the first line of the loop    
            
            elseif aux_string[1] == "*" && aux_string[2] != "Starting"
                node = split(aux_string[2],"+")
                parameters.node_last_int_sol = parse(Int64,node[1])
            
            elseif length(aux_string) >= 3 && aux_string[1] == "0" && aux_string[2] == "2" 
                    parameters.root_gap = parse(Float16,aux_string[length(aux_string)-2])
                if length(aux_string) >= 7
                        parameters.start_branching_upper_bound = parse(Float16,aux_string[length(aux_string)-3])
                    end
            elseif length(aux_string) >= 3 && aux_string[1] == "0" &&  aux_string[2] == "2+" 
                    parameters.root_gap = parse(Float16,aux_string[length(aux_string)-2])
                    if length(aux_string) >= 7
                        parameters.start_branching_upper_bound = parse(Float16,aux_string[length(aux_string)-3])
                    end
            elseif length(aux_string) >= 3 &&  aux_string[2] == "0" && aux_string[3] == "2" 
                    parameters.root_gap = parse(Float16,aux_string[length(aux_string)-2])
                if length(aux_string) >= 7
                        parameters.start_branching_upper_bound = parse(Float16,aux_string[length(aux_string)-3])
                    end
             elseif length(aux_string) >= 3 && aux_string[2] == "0" && aux_string[3] == "2+"
                    parameters.root_gap = parse(Float16,aux_string[length(aux_string)-2])
                if length(aux_string) >= 7
                        parameters.start_branching_upper_bound = parse(Float16,aux_string[length(aux_string)-3])
                    end
                                    
            elseif length(aux_string) >= 3 && aux_string[1] == "0+" && aux_string[2] == "2" 
                    parameters.root_gap = parse(Float16,aux_string[length(aux_string)-2])
                    if length(aux_string) >= 7
                        parameters.start_branching_upper_bound = parse(Float16,aux_string[length(aux_string)-3])
                    end
            elseif length(aux_string) >= 3 && aux_string[1] == "0+" &&  aux_string[2] == "2+" 
                    parameters.root_gap = parse(Float16,aux_string[length(aux_string)-2])
                if length(aux_string) >= 7
                        parameters.start_branching_upper_bound = parse(Float16,aux_string[length(aux_string)-3])
                    end
            elseif length(aux_string) >= 3 &&  aux_string[2] == "0+" && aux_string[3] == "2" 
                    parameters.root_gap = parse(Float16,aux_string[length(aux_string)-2])
                if length(aux_string) >= 7
                        parameters.start_branching_upper_bound = parse(Float16,aux_string[length(aux_string)-3])
                    end
             elseif length(aux_string) >= 3 && aux_string[2] == "0+" && aux_string[3] == "2+"
                    parameters.root_gap = parse(Float16,aux_string[length(aux_string)-2])
                if length(aux_string) >= 7
                        parameters.start_branching_upper_bound = parse(Float16,aux_string[length(aux_string)-3])
                    end
                        
            elseif aux_string[1] == "Root" && aux_string[2] == "relaxation"
                parameters.MILP_relaxation_time = parse(Float16,aux_string[6])
            elseif aux2[1] == "Root" && aux2[2] == "node"
                parameters.root_node_processing_time = parse(Float16,aux_string[4])
            
            elseif aux_string[1] == "Total" && aux_string[2] == "(root+branch&cut)"
                parameters.branch_and_bound_time = parse(Float16,aux_string[4])
            end
             aux2=aux_string         
        end#end for - we have read all lines from file
    end#closing file
end
        


function get_final_LP_solution_parameters(parameters::Parameters,my_model::Model,instance::Instance)
  
    epslon = 0.00000001
    sol_gamma = JuMP.value.(getindex(my_model,:gamma)) 
    sol_z = JuMP.value.(getindex(my_model,:z)) 
    sol_x = JuMP.value.(getindex(my_model,:x)) 
    sol_y = JuMP.value.(getindex(my_model,:y)) 
    sol_w = JuMP.value.(getindex(my_model,:w)) 
    

       
    parameters.number_of_base_relax_variables_gamma =0    
    parameters.number_of_base_relax_variables_x = 0    
    parameters.number_of_base_relax_variables_y = 0    
    parameters.number_of_base_relax_variables_z = 0
    parameters.number_of_base_relax_variables_w  = 0  

    parameters.number_of_frac_variables_gamma =0    
    parameters.number_of_frac_variables_x = 0    
    parameters.number_of_frac_variables_y = 0    
    parameters.number_of_frac_variables_z = 0
    parameters.number_of_frac_variables_w  = 0 
    
    for s in 1:length(instance.setSlices),f in 1:instance.number_of_AN_based_NFs
        if sol_z[s,f] > 0 + epslon
             parameters.number_of_base_relax_variables_z= parameters.number_of_base_relax_variables_z+1 
        end
         if sol_z[s,f] < 1 - epslon
             parameters.number_of_frac_variables_z= parameters.number_of_frac_variables_z+1 
        end
    end
    
    for f in 1:length(instance.set_VNFs), s in 1:length(instance.setSlices), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]
        
      if sol_x[s,f,m,u] > 0 + epslon
             parameters.number_of_base_relax_variables_x= parameters.number_of_base_relax_variables_x+1 
            if sol_x[s,f,m,u] < 1 - epslon
                 parameters.number_of_frac_variables_x= parameters.number_of_frac_variables_x+1 
            end
        end
        if sol_w[s,f,m,u] > 0+ epslon
             parameters.number_of_base_relax_variables_w= parameters.number_of_base_relax_variables_w+1 
        end
    end
    
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities), a in 1: props(instance.physical_network)[:number_of_arcs],f in 1:length(instance.set_VNFs)+1, g in 1:length(instance.set_VNFs)+1
          if sol_gamma[s,k,a,f,g] > 0 + epslon
             parameters.number_of_base_relax_variables_gamma= parameters.number_of_base_relax_variables_gamma+1 
            if sol_gamma[s,k,a,f,g] <  1 - epslon 
                 parameters.number_of_frac_variables_gamma= parameters.number_of_frac_variables_gamma+1 
            end
        end
    end
    
    for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]
        
      if sol_y[f,m,u] > 0 + epslon
             parameters.number_of_base_relax_variables_y= parameters.number_of_base_relax_variables_y+1 
            if sol_y[f,m,u] - floor(sol_y[f,m,u] ) + 1 != 1
                parameters.number_of_frac_variables_y= parameters.number_of_frac_variables_y+1 
            end
        end
    end
    
    parameters.LP_solution = objective_value(my_model)
    parameters.simplex_iterations = MOI.get(my_model, MOI.SimplexIterations())
   


end

function get_final_MILP_solution_parameters(parameters::Parameters,my_model::Model,instance::Instance)
    if result_count(my_model) > 0
 
    parameters.tree_size = MOI.get(my_model, MOI.NodeCount())
    sol_gamma = JuMP.value.(getindex(my_model,:gamma)) 
    sol_z = JuMP.value.(getindex(my_model,:z)) 
    sol_x = JuMP.value.(getindex(my_model,:x)) 
    sol_y = JuMP.value.(getindex(my_model,:y)) 
    sol_w = JuMP.value.(getindex(my_model,:w)) 
    
    parameters.number_of_variables_gamma = length(sol_gamma)
    parameters.number_of_variables_x = length(sol_x)
    parameters.number_of_variables_y =  length(sol_y)
    parameters.number_of_variables_w = length(sol_w)
    parameters.number_of_variables_z = length(sol_z)
       
    parameters.number_of_base_final_variables_gamma =0    
    parameters.number_of_base_final_variables_x = 0    
    parameters.number_of_base_final_variables_y = 0    
    parameters.number_of_base_final_variables_z = 0
    parameters.number_of_base_final_variables_w  = 0  

    for s in 1:length(instance.setSlices),f in 1:instance.number_of_AN_based_NFs
        if sol_z[s,f] == 1
             parameters.number_of_base_final_variables_z= parameters.number_of_base_final_variables_z+1 
        end
    end
    
    for f in 1:length(instance.set_VNFs), s in 1:length(instance.setSlices), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]
        
      if sol_x[s,f,m,u] >= 0.9991
             parameters.number_of_base_final_variables_x= parameters.number_of_base_final_variables_x+1 

                
        end
        if sol_w[s,f,m,u] > 0
             parameters.number_of_base_final_variables_w= parameters.number_of_base_final_variables_w+1 
        end
    end
    
    for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities), a in 1: props(instance.physical_network)[:number_of_arcs],f in 1:length(instance.set_VNFs)+1, g in 1:length(instance.set_VNFs)+1
          if sol_gamma[s,k,a,f,g] >= 0.9991
             parameters.number_of_base_final_variables_gamma= parameters.number_of_base_final_variables_gamma+1 
        end
    end
    
    for f in 1:length(instance.set_VNFs), m in 1:instance.number_of_NFs, u in 1: props(instance.physical_network)[:number_nodes]
        
    if sol_y[f,m,u] - floor(sol_y[f,m,u] ) + 1 != 1
             parameters.number_of_base_final_variables_y= parameters.number_of_base_final_variables_y+1 
        end
    end
        
    parameters.MILP_solution = objective_value(my_model) 
    parameters.final_upper_bound = parameters.MILP_solution
    parameters.final_lower_bound = objective_bound(my_model)
    parameters.first_upper_bound = objective_value(my_model,result = 1)
    parameters.number_of_int_solutions = result_count(my_model)
    parameters.start_branching_upper_bound = objective_value(my_model,result = 1)
        
end
    

end

function get_quali_solution(my_model::Model, instance::Instance,folder::String,split_::Int64, pol::String)
  
#-------------------- GETTING SOLUTION -----------------------------------      
    sol_gamma = JuMP.value.(getindex(model,:gamma)) 
    sol_z = JuMP.value.(getindex(model,:z)) 
    sol_x = JuMP.value.(getindex(model,:x)) 
    sol_y = JuMP.value.(getindex(model,:y)) 
    sol_w = JuMP.value.(getindex(model,:w)) 
    
   
    Objective_value =  objective_value(my_model)

#-------------------- GETTING NODE LOAD AND NUMBER OF NFS -----------------------------------      
    #auxiliary variables
    charge_node = Array{Float64}(undef, props(instance.physical_network)[:number_nodes])
    total_nodes_capacity = 0.0
    most_loaded = 0.0 
    total_load = 0.0
    number_NFSs = 0.0
    ppn = 0.0
    number_active_node = 0
    dis_NFS = 0
    cent_NFS = 0

    for u in 1: props(instance.physical_network)[:number_nodes]
        charge_node[u] = 0.0
        total_nodes_capacity+=props(instance.physical_network,u)[:cpu_capacity]
        if props(instance.physical_network, u)[:node_type] != "app"
            for f in 1:length(instance.set_VNFs),  m in 1:instance.number_of_NFs
                charge_node[u] += (sol_y[f,m,u]*instance.set_VNFs[f].cpu_request)/props(instance.physical_network,u)[:cpu_capacity]
                total_load += (sol_y[f,m,u]*instance.set_VNFs[f].cpu_request)
                number_NFSs += sol_y[f,m,u]
                if props(instance.physical_network, u)[:node_type] == "non_access"
                    cent_NFS += sol_y[f,m,u]
                else
                    dis_NFS += sol_y[f,m,u]
                end
            end
        end
        if charge_node[u] > 0
            number_active_node += 1
            ppn+=charge_node[u]
            if charge_node[u] > most_loaded
                most_loaded = charge_node[u]
            end
        end
    end
    total_load = total_load/total_nodes_capacity


  #-------------------- GETTING LINK LOAD  -----------------------------------      
    #auxiliary variables
    charge_arc = Array{Float64}(undef, props(instance.physical_network)[:number_of_arcs])
    total_arc_capacity = 0.0
    most_loaded_arc = 0.0 
    total_arc_load = 0.0
    number_active_links = 0
    ppa = 0.0
    ver = 0.0
    
        for a in edges(instance.physical_network) 
        edge_id = get_prop(instance.physical_network,a,:edge_id)
        charge_arc[edge_id] = 0.0
        total_arc_capacity+= get_prop(instance.physical_network,a,:max_bandwidth)

        for s in 1:length(instance.setSlices), k in 1:length(instance.setSlices[s].set_commodities)

            charge_arc[edge_id] += sol_gamma[s,k,edge_id,length(instance.set_VNFs)+1,1]*instance.setSlices[s].set_commodities[k]["volume_of_data"]

            charge_arc[edge_id] += sol_gamma[s,k,edge_id,length(instance.set_VNFs),length(instance.set_VNFs)+1]*instance.setSlices[s].set_commodities[k]["volume_of_data"]*instance.set_VNFs[instance.number_of_AN_based_NFs].compression

            for f in 1:(instance.number_of_AN_based_NFs+instance.number_of_CN_based_NFs), g in 1:(instance.number_of_AN_based_NFs +instance.number_of_AN_based_NFs)
                if f!=g 
                    if f<=instance.number_of_AN_based_NFs&&g<=instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g] >0.9
                    charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*instance.setSlices[s].set_commodities[k]["volume_of_data"]*instance.set_VNFs[f].compression
                    end
                    if f>instance.number_of_AN_based_NFs && g>instance.number_of_AN_based_NFs && sol_gamma[s,1,edge_id,f,g]>=0.9
                        charge_arc[edge_id] += sol_gamma[s,1,edge_id,f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities))
                    elseif f>instance.number_of_AN_based_NFs && g<=instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g]>=0.9
                     charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities))
                    elseif f<=instance.number_of_AN_based_NFs && g>instance.number_of_AN_based_NFs && sol_gamma[s,k,edge_id,f,g]>=0.9
                        charge_arc[edge_id] += sol_gamma[s,k,edge_id,f,g]*instance.set_VNFs[f].amount_of_traffic_sent_to_g[g]*(instance.setSlices[s].TotalAmountUE/length(instance.setSlices[s].set_commodities))

                    end
                end
            end
        end


        total_arc_load+= charge_arc[edge_id]
         charge_arc[edge_id] =  charge_arc[edge_id]/get_prop(instance.physical_network,a,:max_bandwidth) 
        if charge_arc[edge_id] > 0
            number_active_links += 1
            ppa+=charge_arc[edge_id]
            if charge_arc[edge_id]  > most_loaded_arc
                most_loaded_arc = charge_arc[edge_id] 
            end
        end
    end
    total_arc_load = total_arc_load/total_arc_capacity


#-------------------- PRINTING -----------------------------------      
        
    open(joinpath(result_folder,"load_on_arcs.csv"), "a") do iooo
        open(joinpath(result_folder,"final_statistics.csv"), "a") do io
            open(joinpath(result_folder,"load_on_nodes.csv"), "a") do ioo              

                if split_ == 7
                    write(io,"$(pol)OptionFlex;$(pol);flex;")
                else
                    write(io,"$(pol)Option$(split_);$(pol);$(split_);")
                end


                write(io,"$(Objective_value);")   
                write(io,"$(number_NFSs);")
                write(io,"$(dis_NFS);")
                write(io,"$(cent_NFS);")


                write(io,"$(props(instance.physical_network)[:number_nodes]);")
                write(io,"$(number_active_node);")
                write(io,"$(number_active_node/props(instance.physical_network)[:number_nodes]);")

                write(io,"$(total_nodes_capacity);")
                write(io,"$(total_load);")
                write(io,"$(ppn/number_active_node);")
                write(io,"$(most_loaded);")


                write(io,"$(props(instance.physical_network)[:number_of_arcs]);")
                write(io,"$(number_active_links);")
                write(io,"$(number_active_links/props(instance.physical_network)[:number_of_arcs]);")

                write(io,"$(total_arc_capacity);")
                write(io,"$(total_arc_load);")
                write(io,"$(ppa/number_active_links);")
                write(io,"$(most_loaded_arc)\n")

                for u in 1:props(instance.physical_network)[:number_nodes]
                  node_type = props(instance.physical_network, u)[:node_type] 
                    if split_ == 7
                        write(ioo,"$(u);$(node_type);$(pol);flex;$(charge_node[u])\n")
                    else
                        write(ioo,"$(u);$(node_type);$(pol);$(split_);$(charge_node[u])\n")
                    end
                end


                for a in edges(instance.physical_network) 
                    edge_id = get_prop(instance.physical_network,a,:edge_id)
                    edge_type = ""           
                    if props(instance.physical_network, src(a))[:node_type] == "access" &&  props(instance.physical_network, dst(a))[:node_type]  == "non_access"         
                        edge_type = "fronthaul"
                    elseif props(instance.physical_network, src(a))[:node_type] == "non_access" &&  props(instance.physical_network, dst(a))[:node_type]  == "non_access"         
                        edge_type = "backhaul"
                    elseif props(instance.physical_network, src(a))[:node_type] == "non_access" &&  props(instance.physical_network, dst(a))[:node_type]  == "app"         
                        edge_type = "external"
                    elseif props(instance.physical_network, dst(a))[:node_type] == "non_access" &&  props(instance.physical_network, src(a))[:node_type]  == "app"         
                        edge_type = "external"  
                    elseif props(instance.physical_network, dst(a))[:node_type] == "access" &&  props(instance.physical_network, src(a))[:node_type]  == "non_access"         
                        edge_type = "fronthaul"
                    end               
                    if split_ == 7
                        write(iooo,"$(edge_id);$(edge_type);$(pol);flex;$(charge_arc[edge_id])\n")
                    else
                        write(iooo,"$(edge_id);$(edge_type);$(pol);$(split_);$(charge_arc[edge_id])\n")
                    end
                end
            end 
        end   
    end

                                                    
        
end

function solve_NSDP_model(my_model::Model,parameters::Parameters,instance::Instance,file_warmup::String)
    #exporting CPLEX log
    #calling solver
    if parameters.relaxation==false
        #CPXsetlogfilename(backend(my_model), "LOG_MILP.log", "w")

        parameters.total_MILP_time = @elapsed begin  
            optimize!(my_model)
        end
        parameters.final_status = termination_status(my_model)
        
       if parameters.warming_up==true
            open("$(file_warmup).txt","w") do file
                write(file,"wp $(termination_status(my_model))")
            end
        else
            get_final_MILP_solution_parameters(parameters,my_model,instance)
        end
        #CPLEX.close_CPLEX(mockModel.inner.env)  
        #mockModel = nothing

    else
        for v in all_variables(my_model)
            if is_integer(v)
                unset_integer(v)
                set_lower_bound(v, 0)
            elseif is_binary(v)
                unset_binary(v)
                set_upper_bound(v, 1)
                set_lower_bound(v, 0)
            end
        end
        mockModel = nothing
        mockModel=JuMP.backend(my_model).optimizer.model
        CPLEX.set_logfile(mockModel.inner.env,joinpath(result_folder,"LOG_LP_$(parameters.instance_name).log"))
        parameters.total_LP_time = @elapsed begin   
            
                        optimize!(my_model)
        end
        parameters.final_status = termination_status(my_model)
        get_final_LP_solution_parameters(parameters,my_model,instance)
        CPLEX.close_CPLEX(mockModel.inner.env)
        mockModel = nothing

    end
 
end

function open_file_stat(test::Int64)    
   
    open(joinpath(result_folder,"quantitative_analyses_test$(test).csv"), "w") do io

        write(io,"instance_name;")
        write(io,"instance_type;")
        write(io,"objective_function;")
        write(io,"model;")
         write(io,"instance_class;")

        
        write(io,"number_of_nodes;") 
        write(io,"graph_density;") 
        write(io,"link_capacity;") 
        write(io,"node_capacity;") 
        write(io,"slice_requests;")
        write(io,"commodities_per_NS;")
        write(io,"CP_NFSs;")
        write(io,"DP_NFSs;")
        write(io,"sharing_NFS;")
        write(io,"sharing_node;")
        
        write(io,"solver;")
        write(io,"boost;")
        write(io,"valid_inequalitiy_class;")
        
        
        write(io,"vi_creation_time;")
        write(io,"number_constraints_without_VI;")
        write(io,"number_constraints_with_VI;")
        write(io,"increase_on_number_constr_with_vis;")
        write(io,"number_of_variables;")
        
        write(io,"number_of_lazy_cuts;")
        write(io,"lazy_time_creation;")
         write(io,"cut_class;")
        write(io,"cut_creation_time;")
        write(io,"number_of_cuts_added;")
        
        write(io,"total_LP_solution_time;")
        write(io,"gap_related_to_best_solution_MILP;")
        write(io,"number_of_simplex_iterations;")

        
        write(io,"total_number_of_gamma_variables;")
        write(io,"ratio_number_of_activeLP_relax_gamma_variables;")
        write(io,"ratio_number_of_frac_gamma_variables;")
        write(io,"ratio_number_of_activeMILP_gamma_variables;")


        write(io,"total_number_of_x_variables;")
        write(io,"ratio_number_of_activeLP_relax_x_variables;")
        write(io,"ratio_number_of_frac_x_variables;")
        write(io,"ratio_number_of_activeMILP_x_variables;")
        

        write(io,"total_number_of_y_variables;")
        write(io,"ratio_number_of_activeLP_relax_y_variables;")
        write(io,"ratio_number_of_frac_y_variables;")
        write(io,"ratio_number_of_activeMILP_y_variables;")
        
        write(io,"total_number_of_z_variables;")
        write(io,"ratio_number_of_activeLP_relax_z_variables;")
        write(io,"ratio_number_of_frac_z_variables;")
        write(io,"ratio_number_of_activeMILP_z_variables;")


        write(io,"total_number_of_w_variables;")
        write(io,"ratio_number_of_activeLP_relax_w_variables;")
        write(io,"ratio_number_of_activeMILP_w_variables;")

        write(io,"total_MILP_solve_time;")
        write(io,"first_MILP_gap;")
        write(io,"root_node_processing_time;")
        write(io,"root_gap;")
        write(io,"start_branching_upper_bound;")
        write(io,"branching_time;")
        write(io,"tree_size;")
        write(io,"number_of_int_solutions;")
        write(io,"node_last_int_solution;")
        write(io,"final_ralative_gap;final_UB;final_LB;")
        write(io,"final_status\n")
        

    end
    
    open(joinpath(result_folder,"quantitative_analyses_test_HEU$(test).csv"), "w") do io

        write(io,"instance_name;")
        write(io,"instance_type;")
        write(io,"objective_function;")
        write(io,"model;")
         write(io,"instance_class;")

        
        write(io,"number_of_nodes;") 
        write(io,"graph_density;") 
        write(io,"link_capacity;") 
        write(io,"node_capacity;") 
        write(io,"slice_requests;")
        write(io,"commodities_per_NS;")
        write(io,"CP_NFSs;")
        write(io,"DP_NFSs;")
        write(io,"sharing_NFS;")
        write(io,"sharing_node;")
        
        
        write(io,"Total_runtime;")
        write(io,"numb_sol_found;")
        write(io,"runtime_per_round;")
        write(io,"final_solution;")
         write(io,"round_final_solution;")
        write(io,"time_final_solution;")
        write(io,"av_solution_cost;")
        write(io,"final_gap;")
        write(io,"av_gap\n")
        
       
        

    end
end